# Maze and path finding

For searching, we have Depth-First Search, Breadth-First Search, Dijkstra, Best-First Search and A\* algorithms.

Using searching to solve a problem, we should know the target doesn't only means a specific location on the graph, but also can be a state reached during the searching process. Which means even you work on the same location, but it's not the same node in abstraction since you are in different state.

In [36]:
import collections
import heapq
class Solution(object):
    def shortestPathAllKeys(self, grid):
        M = len(grid)
        N = len(grid[0])
        
        location = {v: (m, n) 
                    for m, row in enumerate(grid)
                    for n, v in enumerate(row) 
                    if v not in '.#'}
    
        def neighbors(m, n):
            for nm, nn in ((m-1, n), (m+1, n), (m, n-1), (m, n+1)):
                if 0<=nm<M and 0<=nn<N:
                    yield nm, nn
    
        def bfs_from(source):
            m, n = location[source]
            visited = [[False]*N for _ in range(M)]
            visited[m][n] = True
            queue = [(m, n, 0)]
            dist = {}
            while queue:
                m, n, d = queue.pop(0)
                if grid[m][n] != source and grid[m][n]!= '.':
                    dist[grid[m][n]] = d
                    continue # We don't want a path containing two points
                for nm, nn in neighbors(m, n):
                    if grid[nm][nn] != '#' and not visited[nm][nn]:
                        visited[nm][nn] = True
                        queue.append((nm, nn, d+1))
            return dist
        dists = {place: bfs_from(place) for place in location}
        target_state = 2**sum(p.islower() for p in location)-1
        pq = [(0, '@', 0)]
        cost_sofar = collections.defaultdict(lambda: float('inf'))
        cost_sofar['@', 0] = 0
        while pq:
            d, place, state = heapq.heappop(pq)
            if cost_sofar[place, state] < d: 
                continue
            if state == target_state:
                return d
            
            for nextstation in dists[place]:
                nextd = dists[place][nextstation]
                tempstate = state
                if nextstation.islower():
                    tempstate |= (1<< (ord(nextstation)-ord('a')))
                elif nextstation.isupper():
                    if not (state & (1<< (ord(nextstation)-ord('A')))):
                        continue
                
                if d+nextd < cost_sofar[nextstation, tempstate]:
                    cost_sofar[nextstation, tempstate] = d+nextd
                    heapq.heappush(pq, (d+nextd, nextstation, tempstate))
        return -1
                
    

In [37]:
s = Solution()

In [38]:
grid = ["@.a.#","###.#","b.A.B"]
s.shortestPathAllKeys(grid)

8

Example [Link](https://leetcode.com/problems/longest-increasing-path-in-a-matrix/description/):

In [4]:
class Solution(object):
    def longestIncreasingPath(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: int
        """
        if not matrix:
            return 0
        M = len(matrix)
        N = len(matrix[0])
        adj = [[[] for _ in range(N)] for _ in range(M)]
        for m in range(M):
            for n in range(N):
                for i, j in ((m+1, n), (m, n+1)):
                    if i<M and j<N:
                        if matrix[i][j]> matrix[m][n]:
                            adj[m][n].append((i,j))
                        elif matrix[i][j] < matrix[m][n]:
                            adj[i][j].append((m,n))
        state = [[0]*N for _ in range(M)]
        number = [[1]*N for _ in range(M)]
        maxnum = 1
        stack= [(0,0)]
        while stack:
            while stack:
                m, n = stack[-1]
                v = matrix[m][n]
                s = state[m][n]
                if s == len(adj[m][n]):
                    if s > 0:
                        i,j = adj[m][n][-1]
                        if number[i][j]+1> number[m][n]:
                            number[m][n] = number[i][j]+1
                        if number[m][n] > maxnum:
                            maxnum = number[m][n]
                    stack.pop()
                else:
                    while True:
                        if s > 0:
                            pi,pj = adj[m][n][s-1]
                            if number[pi][pj]+1> number[m][n]:
                                number[m][n] = number[pi][pj]+1
                        i, j = adj[m][n][s]
                        state[m][n] += 1
                        s = state[m][n]
                        if state[i][j] < len(adj[i][j]):
                            stack.append((i, j))
                            break
                        if state[m][n] == len(adj[m][n]):
                            break
            for nid in range(M*N):
                curm = int(nid/N)
                curn = int(nid%N)
                if state[curm][curn] < len(adj[curm][curn]):
                    stack.append((curm, curn))
                    break
        return maxnum

In [5]:
s=Solution()

In [6]:
matrix = [[7,7,5],[2,4,6],[8,2,0]]
s.longestIncreasingPath(matrix)

4

A recursive version:

In [1]:
class Solution(object):
    def longestIncreasingPath(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: int
        """
        if not matrix:
            return 0
        self.matrix = matrix
        M = len(matrix)
        self.M = M
        N = len(matrix[0])
        self.N = N
        self.adj = [[[] for _ in range(N)] for _ in range(M)]
        for m in range(M):
            for n in range(N):
                for i, j in ((m+1, n), (m, n+1)):
                    if i<M and j<N:
                        if matrix[i][j]> matrix[m][n]:
                            self.adj[m][n].append((i,j))
                        elif matrix[i][j] < matrix[m][n]:
                            self.adj[i][j].append((m,n))
        self.number = [[0]*N for _ in range(M)]
        maxnum = 1
        for m in range(M):
            for n in range(N):
                length = self.dfs(m, n)
                maxnum = max(maxnum, length)
        return maxnum
    
    def dfs(self, m, n):
        if self.number[m][n] > 0:
            return self.number[m][n]
        maxn = 1
        for i, j in self.adj[m][n]:
            length = 1+self.dfs(i, j)
            maxn = max(maxn, length)
        self.number[m][n] = maxn
        return maxn

In [2]:
s=Solution()

In [3]:
matrix = [[7,7,5],[2,4,6],[8,2,0]]
s.longestIncreasingPath(matrix)

4

**Summay**

A comparison between recursive version and the iteration version equiped with the a stack.
It doesn't means recursive will be slower all the time, these code for this problem is a good example.

In python, we may spend some time unintensionally revisiting some nodes, which can be avoid during the recursive version. We need to realize that this is also where we complain about for using recursive algorithm, but if we can solve the problem of revising some nodes in recursive, like here using extra memory, using recursive may be even more convinent and faster.

New Problem [link](https://leetcode.com/problems/reachable-nodes-in-subdivided-graph/solution/)

In [91]:
import heapq
class Solution(object):
    def reachableNodes(self, edges, M, N):
        """
        :type edges: List[List[int]]
        :type M: int
        :type N: int
        :rtype: int
        """
        adj = {}
        for i in range(N):
            adj[i] = {}
        for i, j, n in edges:
            adj[i][j] = n
            adj[j][i] = n
        total = 1
        stack = {}
        stack[0] = M
        use = [False]*N
        use[0] = True
        while stack:
            cur, left = max(stack.items(), key=lambda x:x[1])
            todelete = []
            for neigh in adj[cur]:
                nodes = adj[cur][neigh]
                if nodes >= left:
                    total += left
                    if cur in adj[neigh]:
                        adj[neigh][cur] -= left
                elif nodes + 1 == left:
                    total += left
                    if use[neigh]:
                        total -= 1
                    use[neigh] = True
                    todelete.append(neigh)
                else:
                    total += nodes + 1
                    if use[neigh]:
                        total -= 1
                    todelete.append(neigh)
                    use[neigh] = True
                    if neigh in stack:
                        stack[neigh] = max(stack[neigh], left-nodes-1)
                    else:
                        stack[neigh] = left-nodes-1
            adj[cur] = {}
            for neigh in todelete:
                if cur in adj[neigh]:
                    adj[neigh].pop(cur)
            stack.pop(cur)
        return total
        

In [92]:
so = Solution()

In [93]:
edges=[[830,2162,4869],[2290,2727,3371],[2054,2889,322],[1800,2925,3997],[1579,2611,8505],[617,1585,9381],[2075,2180,1901],[1970,2703,8359],[1473,2659,292],[1450,2658,3113],[1449,2299,7084],[2087,2803,4419],[1156,2749,7667],[2441,2997,6413],[1213,1554,4548],[1621,2222,3039],[756,1174,6764],[1522,2882,4513],[1073,2263,8782],[353,1278,786],[2344,2478,2689],[202,1447,6096],[2681,2900,2924],[836,2997,7361],[82,1650,3614],[377,388,8153],[2930,2955,4206],[1559,2386,8468],[2394,2929,469],[2291,2681,7117],[338,2269,4021],[2439,2790,7224],[38,2067,8096],[395,2479,2644],[1937,2269,694],[181,937,757],[1937,2151,8083],[317,2360,8053],[590,2743,4271],[1389,2840,481],[2563,2715,7371],[2703,2802,1390],[96,1732,9583],[639,770,4666],[546,1996,83],[1615,2273,1843],[2868,2934,8861],[1538,2876,4061],[576,990,2829],[190,1923,1206],[1306,1893,5531],[2642,2744,7406],[2760,2974,8680],[1336,2157,3049],[591,1611,5997],[1515,2760,8279],[566,2541,9101],[2252,2631,5154],[1430,2583,8015],[2458,2663,3420],[2751,2952,6022],[1528,2519,8902],[1387,1812,2387],[1994,2980,1457],[1525,1602,7315],[2252,2741,2970],[460,2505,809],[644,932,2533],[1334,2803,2325],[1125,1851,1634],[1358,2236,5431],[2523,2680,7269],[2295,2473,7930],[1265,2569,4895],[2060,2774,8166],[668,1416,6175],[2832,2910,7886],[1153,2968,6384],[1770,2226,6912],[1247,1281,6911],[492,978,4961],[1868,2678,3580],[1160,1807,7578],[2476,2586,1837],[2815,2883,102],[1406,2671,7057],[393,1697,8121],[434,1946,4649],[1888,2646,6236],[2870,2915,3875],[2805,2896,3017],[1630,2161,7160],[1561,2939,9393],[2093,2586,3384],[867,1869,8315],[1764,2001,4225],[1404,1990,1996],[2499,2759,3228],[618,1243,5566],[585,2391,8598],[116,1638,7141],[1609,1787,5806],[6,644,536],[1741,2400,8371],[2265,2916,5934],[2258,2833,6317],[1879,2468,7866],[1830,2910,3602],[1370,2350,6711],[2217,2339,1356],[1908,2064,2315],[1072,1664,2527],[2398,2665,7605],[1520,2330,9498],[2949,2998,8379],[153,1352,4416],[337,2182,9963],[1718,2269,9843],[313,1789,7118],[1538,1679,3728],[1945,2653,7614],[1364,2290,1946],[2191,2902,8544],[2546,2817,5694],[908,1976,7063],[453,2632,2523],[1190,1827,6779],[1600,2259,3747],[862,1380,7063],[1093,1838,8623],[2487,2518,9657],[2083,2663,2639],[702,1391,1674],[1333,2774,1278],[2665,2805,4277],[1024,1555,1014],[2520,2548,3434],[598,1758,6724],[660,2393,7031],[1329,2360,3009],[716,785,9312],[1494,2201,7888],[2865,2970,240],[930,1535,8734],[916,2411,8404],[324,2166,3266],[554,2873,3465],[2136,2638,2304],[2796,2934,2849],[2029,2413,2672],[2652,2944,9408],[2538,2718,7244],[644,2549,9172],[2296,2567,3995],[123,1843,3774],[409,1881,6864],[2552,2745,89],[758,2335,6579],[1531,1795,9752],[1017,1813,8985],[2750,2904,942],[1458,1566,1740],[2202,2331,6528],[1676,2445,133],[1610,2203,4215],[2064,2359,7245],[856,1172,3610],[1300,1570,5805],[978,2993,2892],[1347,2332,8270],[2731,2979,1918],[2517,2891,7920],[2073,2532,755],[1244,2396,8441],[1861,2920,5143],[1321,2250,5364],[459,1920,5206],[2355,2416,255],[1448,1489,6989],[2714,2972,5950],[1229,1370,6190],[2250,2517,7346],[491,1863,9204],[2074,2454,2235],[1649,1915,7194],[1972,2134,6322],[68,2747,5043],[1115,2122,999],[386,1125,4215],[826,1939,4832],[2271,2915,5200],[2938,2996,6809],[1024,2804,5],[2241,2711,763],[2804,2954,2457],[988,2656,7029],[431,2179,4623],[938,2483,5347],[144,1572,5878],[2617,2740,3624],[1969,2468,1072],[1073,2702,6883],[141,2168,6745],[2325,2615,4768],[2631,2867,5989],[1469,1658,6691],[1851,2094,7401],[1533,2858,6137],[2762,2842,9052],[1237,2880,6247],[2266,2964,786],[1942,2584,5109],[1688,1972,322],[773,2021,1722],[1344,1438,825],[1971,2910,2821],[1189,2851,8606],[2426,2602,111],[1853,2240,1400],[368,1893,4787],[1165,2657,1400],[1474,2421,3264],[2151,2430,140],[1175,2666,5154],[2503,2762,899],[2874,2907,3862],[48,564,9461],[2802,2951,6904],[2296,2618,3697],[804,2295,3087],[279,1786,5349],[981,2338,8874],[1579,1964,9709],[865,2307,5495],[44,1916,2770],[2703,2738,4705],[1778,2515,6942],[448,1475,971],[532,1497,1519],[1239,2843,2041],[2057,2229,1403],[2105,2854,7149],[2455,2867,5090],[2324,2844,8125],[2599,2953,7418],[1586,2287,9243],[1607,2884,8083],[1863,2599,3855],[2780,2792,5073],[1098,2952,5045],[405,761,6721],[770,2238,9334],[51,779,6326],[2781,2815,193],[2589,2678,772],[2165,2894,2233],[2284,2922,710],[392,1681,2613],[2563,2901,4322],[2415,2934,9205],[296,1370,826],[2362,2480,5703],[327,1611,9292],[1816,1838,4820],[884,1420,534],[211,1946,6573],[1764,2594,5180],[2108,2432,673],[1785,1830,1033],[764,2278,8869],[1485,1652,3586],[195,571,8755],[2714,2881,6866],[573,1304,2381],[746,1118,9158],[1766,1814,9937],[278,1887,8845],[638,2124,1342],[641,1205,4299],[439,1942,7463],[1257,1793,5571],[2095,2634,6589],[1271,2679,1961],[1074,2279,8740],[444,542,6529],[763,1196,2622],[1696,2711,3135],[53,2763,5997],[2933,2983,7284],[2309,2959,82],[2475,2608,6362],[2143,2442,3306],[1906,2297,6676],[2687,2810,8030],[1539,2395,4412],[2003,2583,9765],[65,2515,9304],[1243,2072,3608],[404,2147,4432],[210,2082,6720],[92,235,4364],[2826,2943,4271],[2874,2884,2312],[343,2034,5335],[1597,2668,6841],[2202,2788,9480],[2652,2836,5844],[1182,2448,6257],[2407,2563,5899],[2117,2673,116],[717,1758,2887],[1723,2832,7392],[2018,2839,7764],[1393,1970,8116],[2995,2996,4738],[36,884,4974],[1280,1867,478],[239,1020,3576],[933,2355,1714],[221,1908,7019],[2759,2944,8681],[772,1418,5473],[353,1431,5450],[2367,2475,2689],[2479,2561,9756],[899,2185,8461],[2884,2988,9281],[1242,1384,11],[2561,2686,1324],[454,2894,1619],[1502,2552,1556],[2145,2254,7808],[2996,2998,3702],[1088,1155,1908],[2008,2972,3980],[1890,2244,3838],[2401,2924,5748],[996,1876,6615],[1907,2571,5646],[1878,2686,867],[1761,2295,432],[161,1179,5450],[1236,2725,8440],[1421,2201,6249],[1507,2373,9699],[883,1413,8778],[1917,2921,4333],[2856,2986,2079],[355,2396,3231],[1848,2743,3268],[583,1751,5087],[1918,2481,5723],[254,1779,7073],[2879,2951,7586],[903,1591,6662],[1505,1774,8520],[2659,2833,9955],[2730,2821,5994],[2358,2609,1521],[2462,2737,7969],[222,747,2420],[2439,2650,8198],[1209,1420,9684],[814,2772,4980],[865,1009,7265],[1815,2342,9029],[2763,2781,8215],[559,1154,3470],[1262,2375,8940],[557,1842,6788],[1439,2801,2725],[1326,2478,8044],[17,2112,8811],[2794,2800,4816],[2337,2503,378],[762,1291,3263],[113,2339,5242],[1293,1306,5008],[2066,2256,1452],[1577,2164,9170],[1860,2085,217],[2096,2127,56],[1284,2419,8409],[2720,2877,8274],[2334,2528,4312],[2900,2925,552],[1753,2179,9715],[2264,2456,4555],[2408,2664,261],[719,2168,7241],[2169,2540,9628],[620,2508,7111],[698,2122,5179],[536,2300,7501],[2153,2311,5841],[1465,1692,4916],[1929,2049,2512],[1565,1569,224],[480,1688,4003],[2053,2577,4915],[1569,2680,1086],[1432,2046,1759],[19,52,7252],[2130,2874,4274],[534,2307,3812],[1889,2789,5470],[1918,2642,9474],[2364,2712,2868],[1879,2965,8491],[527,1582,4477],[905,1300,6386],[457,473,8855],[599,2991,8457],[2916,2982,1619],[1513,2648,8077],[1553,2556,3869],[796,1945,7946],[754,1019,2086],[2776,2805,884],[1204,1711,7799],[1470,2297,1095],[74,2386,633],[325,1966,4745],[311,2575,3386],[837,2450,7726],[2432,2820,4944],[1830,2065,3013],[383,2600,4231],[1253,2576,65],[1583,2478,9889],[230,761,6650],[2486,2940,2646],[686,2596,8574],[1220,1847,3248],[1125,1813,656],[2586,2788,3985],[1951,2988,628],[1966,2281,300],[497,1886,1295],[432,1579,3978],[2429,2743,6871],[418,925,7604],[244,1517,6917],[1119,2151,5938],[975,1743,7519],[2726,2746,5488],[2037,2952,9133],[2846,2859,6928],[167,836,1541],[2601,2845,440],[685,2975,2777],[2524,2709,9762],[372,603,5700],[2166,2267,3370],[2015,2406,7409],[115,1031,8059],[2668,2875,8771],[1477,1907,8761],[1062,1747,7181],[1477,2252,3486],[877,2447,7118],[1584,2041,4716],[468,2274,8337],[413,2777,8696],[2857,2988,4397],[10,839,6704],[2405,2651,6058],[2933,2984,3518],[1109,1592,8582],[1400,2175,6800],[1707,2818,292],[1885,1907,3508],[1238,1268,4229],[286,725,1190],[206,871,6686],[1380,1683,3750],[2072,2559,5676],[818,1127,7874],[2519,2871,318],[1956,2953,8653],[1300,1432,4322],[681,1146,6779],[722,940,7561],[1048,2662,3206],[363,2339,6028],[2570,2607,6878],[556,956,6338],[1749,1966,5666],[891,2882,1211],[1022,1638,2419],[1363,2740,3886],[588,777,3678],[2931,2962,6792],[670,1610,3298],[968,1629,5794],[2100,2391,5848],[2222,2575,5442],[663,1851,3172],[1007,1751,9756],[2016,2783,4422],[805,1611,8490],[2303,2759,5952],[717,991,8703],[1469,1993,4039],[1084,2447,1994],[1619,2045,9421],[2668,2897,2437],[1980,2440,6984],[470,1283,7124],[2833,2920,9847],[223,624,313],[1508,2566,1229],[2132,2898,4215],[1038,1396,4025],[2920,2955,545],[1056,1804,3618],[1012,1272,8031],[1799,2334,4011],[847,1282,7895],[1333,2156,3160],[1514,1526,4113],[662,2752,2898],[916,1765,10],[1213,2725,5249],[120,513,8306],[2752,2994,4274],[127,2435,9724],[356,2583,7413],[2625,2952,8734],[2294,2696,3157],[447,515,9250],[774,2633,1182],[1404,2466,4124],[257,1167,6618],[548,2886,1597],[1977,1997,6639],[590,1533,2909],[291,2172,584],[991,2293,7903],[2209,2765,4224],[340,1704,5319],[1669,2386,8018],[2014,2836,444],[2121,2125,7281],[1856,2876,8392],[2490,2999,9726],[1760,1999,769],[125,1012,1396],[2542,2782,2527],[1206,2317,4006],[2922,2981,7103],[2047,2404,8608],[2659,2848,3645],[2325,2889,2304],[2356,2470,1252],[1950,2569,1912],[1878,2774,924],[1901,2008,9134],[1076,2298,8627],[1640,2508,9468],[1442,2359,4998],[2610,2622,3327],[586,685,9557],[1221,1668,3176],[2575,2712,8037],[688,2670,9374],[774,996,2867],[874,1237,5996],[2715,2867,3890],[2955,2994,9317],[426,723,6266],[2360,2956,7150],[281,1842,2407],[950,2776,9212],[2820,2997,5795],[29,2107,3492],[2156,2815,4882],[2553,2637,2441],[1483,1816,5408],[970,1951,3670],[2771,2880,781],[429,1709,1858],[685,2737,2132],[1666,1694,5365],[1768,2828,8298],[1402,1910,6629],[1425,2795,4956],[1093,2278,8074],[1776,2028,4295],[1384,2467,9804],[1167,1598,5482],[1565,2245,71],[330,526,6744],[175,2504,9459],[1954,2111,6344],[299,1502,71],[2920,2957,5155],[1531,2716,7999],[2774,2988,7528],[643,1993,466],[831,1213,4967],[1602,1940,5139],[497,1532,6136],[2082,2624,4938],[961,2079,5251],[2761,2805,3107],[897,1891,1401],[203,2487,4498],[4,2099,4738],[261,1364,974],[2043,2413,614],[1171,1236,1152],[1964,2085,4908],[35,2239,4684],[2386,2417,9470],[778,1763,1037],[2769,2940,2279],[660,2072,5044],[2280,2647,6600],[2127,2430,7987],[2928,2965,3927],[2334,2874,7047],[2003,2198,1540],[2691,2751,7553],[2212,2496,955],[1552,2389,6546],[1546,1776,5479],[1716,1744,5059],[374,2960,3360],[314,2890,3947],[602,1757,3382],[1441,2109,1962],[2487,2695,8019],[941,1154,3039],[1644,2342,5724],[1450,2399,7836],[240,2691,7101],[1913,2720,5173],[1600,2618,2376],[899,2883,5014],[1519,2029,8052],[2908,2999,9164],[1362,1828,8471],[853,2402,5224],[1669,2143,6581],[2228,2390,1368],[2679,2859,4017],[2774,2929,469],[1654,2275,9324],[42,365,8290],[2196,2700,9764],[287,1950,1890],[2240,2966,9948],[1232,2139,8228],[1821,2395,3455],[790,2362,1359],[2194,2817,8852],[316,2045,9262],[2409,2975,1000],[16,1853,7974],[683,744,6116],[378,593,7913],[1878,2369,1343],[1340,1584,4221],[20,2637,21],[844,2542,1420],[2894,2916,5463],[1794,2412,4],[2231,2352,5751],[2379,2583,4967],[15,252,4897],[1660,1771,8965],[1789,2795,461],[2954,2987,9423],[125,1069,8704],[936,1142,7143],[212,2501,5524],[1006,2544,9698],[83,1492,405],[288,1170,8372],[959,1110,142],[798,2135,2955],[651,2214,1603],[1735,2247,4043],[1606,1683,9462],[1501,1579,9707],[919,1351,5485],[2052,2953,6423],[2108,2459,8671],[2366,2783,5860],[2410,2992,1610],[2103,2812,1855],[2971,2998,813],[2827,2981,4655],[2173,2838,8056],[395,1920,4066],[2496,2514,5721],[1770,2410,357],[2227,2979,5334],[2353,2974,7912],[639,1061,1852],[2690,2694,1290],[1568,2200,784],[1979,1992,6972],[1017,2442,1135],[1315,2024,7557],[100,2762,2857],[1853,2074,5009],[1529,1746,5856],[30,1380,808],[883,1039,7900],[1550,2569,7407],[1845,2145,6680],[2820,2876,6594],[1955,2463,3722],[2415,2766,5589],[327,808,3080],[23,1451,882],[2661,2863,1559],[1601,2799,1908],[2616,2650,9115],[2622,2976,251],[151,1107,4418],[866,2734,3640],[528,1013,5528],[135,2438,9964],[2720,2929,2123],[319,2251,4059],[2259,2984,7291],[1501,1663,4022],[935,2942,8336],[1927,2953,3472],[2525,2907,2085],[1606,1973,4198],[799,2433,5463],[522,924,7753],[2203,2983,3512],[2245,2341,6736],[1249,1648,2773],[1382,2760,1867],[1306,1399,5902],[2380,2922,3402],[2139,2993,2283],[1047,1980,2583],[1724,2607,5587],[1646,2143,5331],[2351,2463,9355],[481,2527,1305],[2525,2563,7436],[1078,2616,6542],[2974,2986,448],[2834,2857,2523],[1199,1545,8679],[836,1941,4665],[272,971,2849],[153,2717,9181],[2380,2856,1368],[513,2752,3316],[1822,2360,6898],[1414,2420,1421],[2817,2900,4949],[930,2673,797],[2873,2980,8740],[1472,2741,1598],[411,1280,8491],[1776,2928,6444],[2560,2894,1420],[249,2734,9516],[2219,2495,5784],[1348,2031,1335],[848,2727,6194],[852,2158,1293],[2176,2511,10000],[1028,2056,9405],[12,545,2516],[2836,2929,5914],[1979,2756,5638],[2830,2845,5261],[1499,1736,9747],[952,2556,6255],[689,2954,9491],[2506,2558,164],[453,1581,5464],[2814,2963,5881],[933,2598,3687],[497,2215,8173],[1351,1855,2794],[911,1215,3826],[1647,1952,3546],[1123,1799,5624],[2637,2700,8565],[100,2175,8987],[1273,1860,8448],[531,713,4553],[2065,2807,3726],[1175,1481,38],[2359,2731,7093],[434,2696,9421],[416,2665,2705],[2241,2772,4068],[2771,2973,3211],[2326,2567,4690],[11,1967,4073],[2417,2745,5483],[1072,2476,1826],[1321,2063,5583],[318,957,3290],[541,1693,7549],[1469,1738,6116],[112,641,9546],[2686,2703,2715],[2456,2935,3817],[1872,2536,6017],[696,1751,5941],[911,2209,2322],[2349,2522,8294],[1665,2112,5210],[1699,1888,3063],[305,2802,6882],[997,2772,3161],[385,465,2323],[2072,2366,5414],[1711,2099,5214],[919,1665,988],[795,1261,6338],[656,842,8560],[1768,2292,8239],[164,2163,2466],[2250,2354,955],[2219,2665,3847],[1196,1402,4248],[2764,2806,3639],[1470,1586,3134],[118,1210,5361],[2729,2790,501],[1335,2069,310],[2436,2759,5403],[2434,2629,4838],[1128,1192,9810],[2137,2692,4748],[217,1946,3593],[77,787,5111],[665,2738,4234],[416,1229,2503],[119,869,9145],[2830,2929,9053],[637,1496,8451],[690,967,3428],[982,2002,6066],[1926,2231,1226],[1272,1477,3181],[2179,2565,2244],[1833,2298,9832],[2096,2498,795],[2173,2350,1488],[416,2244,1410],[2137,2412,2905],[2035,2878,2951],[1150,1827,914],[1742,2023,3345],[1007,2998,6745],[1035,2381,4336],[2291,2377,2359],[2694,2842,3599],[2423,2480,6343],[1412,2853,3041],[1357,1543,1866],[2586,2631,6572],[1358,2877,8319],[2771,2782,1086],[2828,2941,8613],[2654,2729,3494],[1893,2885,2726],[1890,2925,9222],[857,1351,3442],[1627,2994,3921],[15,389,7349],[683,1772,8661],[791,1881,3250],[1625,1920,7409],[2227,2542,8190],[271,2184,4038],[1987,2796,27],[525,2790,4840],[403,594,965],[161,1854,1479],[2352,2725,582],[2840,2904,3534],[2410,2990,3561],[2135,2261,9979],[2738,2952,9465],[3,2620,9466],[1809,2262,1256],[2151,2335,8376],[524,777,4994],[2724,2765,3679],[364,2271,1106],[145,2613,3687],[2214,2433,2651],[1953,2889,7554],[2344,2760,368],[1337,2551,681],[1854,2107,5747],[1837,2700,2448],[2020,2571,5812],[1538,2589,1052],[638,1151,1183],[78,1944,233],[2720,2996,2941],[1416,2026,2282],[945,1124,1741],[2220,2360,7225],[2529,2674,5719],[2939,2946,4488],[608,2187,468],[1716,2292,1578],[2688,2911,7546],[1093,1613,9629],[2978,2992,2973],[2298,2304,2673],[165,1178,929],[1925,2489,3034],[2497,2811,972],[289,330,787],[2276,2943,3409],[2403,2998,8505],[601,2185,2718],[469,2315,3093],[1495,2743,8098],[1245,2860,2490],[1612,1778,6999],[2616,2916,548],[2049,2926,9052],[1912,2988,7419],[155,1594,9503],[516,2066,6752],[1948,2494,4185],[695,882,4263],[2415,2521,6144],[1977,2602,9262],[482,2929,7142],[1025,2059,7756],[826,2343,2607],[1583,2300,1805],[523,2966,8653],[1650,2876,6436],[2852,2927,7690],[1791,2137,2592],[2806,2838,2238],[2835,2868,6176],[1447,1980,1217],[1763,2676,6397],[2614,2678,5779],[336,756,8587],[6,2582,128],[406,2290,8958],[1196,2116,4785],[326,1538,4608],[964,2101,7175],[822,2428,6050],[788,2294,4869],[1243,2877,2250],[105,2679,3894],[2843,2937,2097],[1400,2129,5171],[2081,2136,2326],[864,2494,4336],[1384,2458,1229],[861,1550,5635],[2801,2936,479],[2911,2982,3764],[1654,2140,151],[467,2336,9411],[2356,2988,6815],[739,1668,3001],[2311,2751,5524],[472,2975,7059],[2624,2647,5073],[1919,2864,5476],[170,968,1940],[756,1589,2121],[2339,2834,4184],[1132,1869,4742],[1829,2931,6894],[404,1373,865],[1242,2579,8670],[2161,2918,2862],[284,2895,988],[1628,1733,7239],[1438,1933,3301],[2206,2294,8763],[592,1355,663],[639,1070,9552],[525,1228,1202],[635,2070,8314],[1167,2917,4916],[1348,1910,1958],[2090,2784,8494],[1420,1752,1022],[2370,2831,9902],[716,2289,1426],[311,947,9032],[811,2862,6347],[595,770,9269],[2757,2864,202],[1858,2392,2639],[817,1834,1403],[1939,2392,9659],[2604,2858,7243],[358,402,3731],[1707,1719,5205],[2682,2867,5937],[1529,2584,4198],[1122,1540,7370],[2131,2474,2031],[2624,2625,5372],[1200,1805,9883],[1863,2898,9560],[343,2374,7581],[2229,2928,3025],[782,1175,4790],[2562,2924,7097],[870,1598,5589],[1910,2947,7671],[16,324,527],[2136,2975,8275],[688,2815,118],[644,2587,1382],[807,2560,9513],[1961,2241,5419],[1229,1810,2967],[2069,2434,5597],[598,1645,88],[1820,2901,9272],[163,1567,5865],[60,607,6643],[1934,2090,9587],[402,1163,4789],[351,1371,134],[1997,2832,4864],[447,728,6241],[1783,2657,8672],[2275,2405,9392],[2451,2732,9002],[1492,2094,9594],[895,1985,385],[822,1327,8638],[500,2615,477],[1597,2166,751],[497,2848,1792],[2543,2755,5468],[1240,1789,4731],[2259,2951,2232],[2253,2965,2848],[884,2947,1789],[1563,2892,4096],[2646,2859,8837],[1894,2840,926],[2719,2740,6160],[2260,2442,9673],[851,1077,2328],[2943,2965,741],[2348,2884,7032],[2550,2816,743],[622,1299,1863],[2916,2918,3230],[2707,2806,6927],[294,2893,551],[1137,2310,3423],[683,2979,1791],[946,2959,4778],[742,1889,1452],[2950,2952,7600],[2751,2800,7120],[2454,2592,3802],[849,2670,4283],[1966,2209,4594],[2036,2903,9997],[421,2771,919],[934,2560,2318],[2049,2160,2416],[2705,2919,4949],[1529,2924,2492],[2475,2798,3722],[1538,1958,7740],[1563,2627,88],[1084,1958,2100],[495,1467,7343],[1481,2543,5918],[43,1698,9885],[1907,2144,4912],[1680,2496,5004],[2538,2927,1710],[2710,2865,427],[113,2272,5236],[2843,2935,7332],[90,1260,1161],[920,2506,5582],[2981,2986,1144],[2593,2668,8671],[1907,2357,3170],[14,1085,8327],[126,1991,8033],[2465,2606,636],[1316,1995,6203],[2883,2990,7139],[2897,2980,9443],[578,2356,4499],[2164,2246,9939],[1074,2909,1693],[357,2172,2257],[675,1154,8772],[156,218,5173],[31,1906,4498],[1073,2649,1398],[927,2048,4478],[2890,2899,1096],[127,650,2720],[1373,2174,3277],[2714,2768,2386],[471,913,6199],[184,2184,8492],[1898,2278,3978],[1727,2845,5265],[1094,2518,5347],[772,2624,559],[861,927,3336],[1163,1565,8286],[86,211,6210],[775,1055,2964],[813,1568,9194],[1499,2031,5838],[1795,1971,4131],[1711,2346,5601],[1613,2175,683],[637,2094,5409],[2532,2659,8481],[485,536,2309],[72,751,8370],[2528,2704,1081],[2579,2984,5420],[2668,2881,9872],[802,2329,3968],[791,907,9367],[750,1752,6833],[1191,2729,6455],[1696,1744,9188],[1367,2093,3108],[2397,2836,9456],[2533,2582,5754],[1667,2490,3052],[1799,2018,3606],[157,2729,1870],[2000,2307,1915],[718,2877,7339],[1877,2537,1068],[2610,2915,2593],[2954,2969,9161],[817,2623,6994],[233,629,4944],[2980,2984,3580],[733,2290,442],[2655,2816,2419],[1896,2739,95],[97,1473,6700],[1741,2456,9738],[2063,2529,9599],[1278,2720,9230],[486,1602,638],[2157,2988,7823],[327,2378,5533],[354,1971,1002],[2628,2825,3443],[2309,2682,1310],[1886,2350,2990],[2015,2034,8443],[394,2287,9132],[2641,2818,7479],[424,1397,7368],[2192,2741,7645],[2183,2962,7935],[1527,2572,7535],[6,2975,7169],[2023,2431,149],[2704,2907,405],[1518,2382,7278],[1312,2010,7976],[2725,2912,6259],[2811,2989,912],[1,693,8020],[2844,2857,1296],[229,1334,7090],[2764,2777,6782],[2921,2971,1538],[1237,2493,7302],[1147,1824,9042],[2578,2639,7135],[2698,2757,7961],[2920,2976,7432],[2161,2726,6414],[1192,1707,9948],[2404,2690,1516],[2629,2979,1231],[1341,2705,189],[311,1298,4754],[263,1276,4808],[2408,2708,458],[1165,1999,3273],[882,2738,2492],[2368,2931,4411],[1003,1374,3982],[2535,2563,9065],[1881,2319,9574],[1996,2523,537],[2018,2350,3029],[974,1703,9060],[2689,2726,1528],[1755,2156,7816],[1145,2974,6164],[1661,1960,8898],[2870,2884,209],[2618,2779,8925],[1385,1509,6726],[1900,1923,715],[441,2246,483],[1463,1985,9792],[503,1087,499],[330,2341,4567],[2732,2747,7557],[449,1369,9197],[2163,2347,9174],[2343,2895,1415],[542,607,2863],[523,1496,2604],[167,1853,3849],[1335,1410,4921],[2836,2996,385],[2800,2987,7740],[2860,2928,8392],[2880,2936,3802],[1682,2156,9888],[2216,2915,2802],[2845,2964,1902],[2141,2787,2208],[2743,2957,7028],[1293,2970,5237],[1825,1991,1709],[238,1529,3404],[1165,2716,2285],[144,2849,5905],[1777,2130,5353],[1634,2004,1193],[1349,2462,4134],[794,2989,9277],[2555,2865,9512],[405,473,851],[1000,2418,8700],[2916,2946,49],[1588,2774,3542],[956,2259,7069],[1456,2699,7397],[2802,2967,8403],[2112,2313,9079],[2009,2821,9011],[1966,2542,1118],[188,1966,4596],[1403,2604,9797],[2519,2743,2652],[1255,2351,5376],[2885,2986,6948],[985,2492,536],[2002,2537,7139],[1312,2505,6395],[2385,2781,1719],[1003,2862,9874],[310,1110,6309],[2935,2964,3507],[686,1111,6777],[1474,1890,8684],[222,1247,5855],[1163,1927,6678],[1905,2302,3340],[155,1074,8893],[103,2225,6812],[252,2339,6356],[2493,2795,8828],[2506,2947,2239],[365,2574,6005],[1386,2449,3194],[2392,2596,1081],[1052,1156,778],[2907,2951,1442],[1617,2846,6535],[1804,2125,8972],[2098,2591,3964],[2798,2888,9729],[155,910,6976],[1742,2989,5365],[2859,2879,5255],[1907,2850,5200],[365,594,6889],[2183,2506,5974],[1527,2476,4652],[704,2712,9399],[726,1710,1779],[2414,2576,7252],[2657,2960,9500],[1199,2863,3862],[2278,2560,3978],[663,2872,7486],[1602,1774,2173],[2029,2686,5020],[1699,2358,7880],[204,2530,7671],[2218,2976,6232],[1027,1859,8317],[828,1224,143],[2131,2922,1899],[997,2595,1969],[1062,1543,8033],[2078,2442,6917],[28,325,5012],[1222,1811,2595],[737,877,6823],[2275,2576,4746],[2170,2529,7952],[2427,2767,2113],[1856,2352,660],[1881,2813,5566],[2740,2944,9735],[2982,2985,6141],[1687,2651,6499],[643,809,6061],[830,1320,1326],[2814,2869,4718],[2838,2943,82],[762,1793,2202],[2783,2885,1710],[2779,2998,8131],[851,2989,2270],[1675,2727,3205],[1385,1510,5750],[1057,1639,6135],[1950,2535,2602],[625,1335,9644],[1415,1802,3537],[1148,1329,1790],[2717,2799,4014],[788,1984,7736],[2458,2631,9168],[1021,1940,2368],[2115,2426,3814],[252,904,9216],[589,2675,3030],[2137,2797,5352],[2238,2937,7143],[2430,2580,8441],[2556,2567,8548],[1567,1628,4254],[1925,2788,3909],[1089,2183,1383],[2357,2822,5087],[1744,1897,1448],[1710,2552,4155],[1059,2978,423],[797,2057,7224],[2315,2823,2875],[616,1618,2487],[2403,2410,787],[1821,2918,5259],[2486,2891,7028],[478,645,3843],[498,2280,3984],[1787,2010,681],[1585,2383,6973],[314,1717,84],[2778,2911,4501],[672,2699,8807],[1022,1492,4042],[554,2581,4012],[792,1661,7764],[445,528,8728],[347,950,2029],[1305,1479,2556],[103,974,1995],[1855,1894,4350],[2576,2872,34],[1835,2934,4112],[176,1647,4698],[1047,1420,7476],[1303,1596,4697],[1853,2439,7444],[1174,1920,4975],[117,195,2034],[178,492,8533],[2121,2299,442],[1364,2605,1983],[1697,2358,7316],[201,1767,7743],[2169,2255,1555],[1739,1758,4700],[2856,2911,3799],[2382,2683,4756],[2071,2102,8662],[1982,2706,8136],[34,942,4408],[1627,2309,9771],[1176,1739,7796],[637,2463,8047],[388,2978,7535],[1798,2849,5981],[1334,1732,3797],[1197,1198,9713],[2849,2944,2133],[174,601,2560],[448,1008,3083],[1169,2053,1673],[205,1416,8837],[2780,2910,1519],[153,1739,744],[2794,2863,4905],[39,116,7292],[1100,1149,8518],[394,686,1382],[1417,1935,8133],[1587,1995,7741],[1708,1854,5003],[1432,2474,3346],[237,2100,7947],[1859,2037,7686],[786,879,6247],[2684,2937,8606],[1864,1967,4259],[2070,2556,8938],[336,906,6981],[2298,2655,485],[1413,2620,5767],[1710,2762,2554],[1623,2900,7477],[139,1351,1550],[2141,2419,4933],[1794,2529,2400],[1289,2268,8189],[2352,2873,9072],[656,1410,6344],[2302,2928,4666],[1587,1643,2856],[1998,2247,3645],[1105,2110,5978],[2118,2670,2352],[2668,2946,3038],[271,582,9601],[748,1035,3962],[513,1615,801],[360,1411,4827],[971,2520,8018],[1123,1980,2570],[2069,2565,3480],[2192,2800,5271],[1696,2924,9161],[2180,2539,3452],[848,896,9260],[387,2622,9672],[2505,2999,1859],[2210,2629,544],[776,2229,8533],[1512,1740,5882],[1989,2552,5699],[636,1423,3146],[2020,2449,5858],[97,2388,2505],[2547,2570,1348],[349,1856,4639],[2628,2719,4843],[2018,2752,6652],[1419,1835,1044],[2500,2598,7962],[1620,2075,2255],[336,1393,8795],[468,1697,1654],[1309,2841,6906],[1376,2225,262],[2716,2759,201],[2040,2295,4600],[1034,2600,5358],[298,1524,278],[429,630,1390],[412,2056,9903],[266,1349,6385],[357,558,3346],[1011,1164,6365],[1364,2980,3181],[1886,2900,5689],[1553,2345,3764],[2260,2780,8787],[2077,2213,9759],[1266,2133,8785],[1882,2969,2632],[359,953,9088],[193,2156,4306],[19,2961,9109],[1813,1911,5082],[2878,2968,9127],[1129,1288,8881],[33,2398,6647],[2493,2583,3201],[1444,1667,9677],[1351,2783,6630],[2276,2406,9116],[1035,1485,1775],[2295,2681,9442],[1208,2851,9978],[1513,2956,9823],[621,2686,2369],[607,1694,9974],[1366,2470,8619],[1787,2413,7785],[2965,2991,4215],[1459,1999,1781],[1486,2369,6472],[1041,1736,3947],[399,2986,4501],[2020,2950,1129],[1118,2090,6307],[2589,2908,9049],[2796,2858,5580],[1630,2300,2429],[582,1927,5097],[954,1891,5362],[2453,2967,5358],[2358,2504,7986],[2091,2664,8570],[1540,2665,8146],[2315,2325,3283],[1643,2059,4090],[300,995,3915],[794,1485,7027],[1194,1311,4635],[2011,2224,3574],[390,2928,3475],[303,1901,5493],[2311,2544,6966],[223,1172,1767],[2079,2353,3592],[247,2635,6377],[594,950,7716],[1229,1837,3091],[2325,2807,361],[2945,2947,3606],[163,528,5757],[179,1348,9561],[1495,2577,8762],[250,2263,5981],[923,2336,9989],[2997,2998,919],[2584,2995,6327],[1032,1898,4836],[1166,2411,8251],[1549,2506,1509],[2476,2528,3740],[1124,1323,1481],[142,1150,2443],[110,1747,4411],[322,2472,3670],[332,2851,7375],[1576,2380,9765],[2475,2736,2423],[221,915,512],[2600,2787,8927],[2497,2524,2281],[2498,2983,2300],[1914,2108,4875],[2892,2946,6223],[1656,2470,5316],[1468,2214,2954],[2209,2711,5064],[418,2813,4755],[2551,2763,6972],[697,2084,5447],[1151,2900,2449],[1036,2353,8703],[314,1471,6389],[2311,2883,232],[1876,2083,2733],[1553,1576,3487],[2653,2663,9041],[261,1875,5965],[2569,2675,7116],[2498,2944,6821],[2926,2932,6669],[1307,2299,9740],[2381,2588,6152],[788,2297,4447],[544,1589,2837],[1092,2193,3298],[1402,2593,9103],[1672,2563,839],[2824,2965,8800],[827,1263,6539],[2425,2703,8808],[2865,2983,3384],[1050,2895,8587],[227,989,8852],[1505,2042,5299],[2691,2889,3962],[1679,2870,1299],[2430,2573,5245],[371,498,5404],[2583,2978,9284],[1588,2763,5815],[2511,2603,263],[1498,2128,8946],[523,1486,8095],[916,2135,6719],[1156,1174,5760],[1839,2692,7303],[2154,2598,9942],[1394,2805,679],[2143,2309,5748],[92,2513,76],[2352,2372,3018],[330,2114,4226],[2848,2859,740],[516,2712,4057],[2740,2835,1620],[1419,2501,9125],[2869,2968,4300],[2800,2986,3441],[1546,1719,8278],[2848,2958,4046],[1386,1474,8159],[1651,1709,7195],[2736,2826,3973],[2747,2849,6362],[556,1963,1085],[200,2427,7124],[2636,2846,243],[372,949,475],[1837,2861,1429],[1450,2950,669],[332,2374,2042],[801,2942,3922],[2375,2666,1050],[23,1209,8812],[1300,2171,210],[898,1051,5865],[81,1735,5544],[2191,2699,413],[2265,2688,9923],[1801,2440,989],[2629,2884,1414],[1514,2516,3797],[128,2332,6446],[52,1859,9600],[1562,1912,5001],[2447,2543,9583],[2468,2820,2860],[786,2246,9851],[2883,2902,3789],[358,2573,685],[1524,2004,3343],[1410,2306,4622],[109,878,3636],[1786,2424,245],[1673,2223,6994],[2972,2982,6296],[811,1166,8289],[2866,2946,5614],[1751,2662,3903],[1490,2719,3947],[2639,2928,5666],[961,2858,1505],[2209,2256,6133],[2358,2716,3699],[2893,2918,9806],[302,1145,3462],[2286,2773,6586],[1767,2517,3278],[2989,2992,3347],[1274,2953,4068],[1922,2160,9376],[332,1600,5294],[1053,1573,8258],[1118,1151,2046],[106,306,8518],[2273,2306,1350],[443,1919,47],[1225,1986,975],[2615,2821,5663],[2227,2537,6062],[1240,2252,7949],[305,1508,4774],[1585,2703,3843],[2929,2970,8145],[1632,1893,6440],[1622,1885,1505],[69,2612,6330],[684,2746,7610],[1427,2364,7477],[772,1504,8144],[1364,1682,3275],[1588,2039,5682],[2212,2293,5587],[2963,2994,8906],[872,1855,5067],[2289,2517,9521],[1078,2416,7422],[180,1662,1010],[820,2323,8512],[2365,2983,9752],[381,2480,675],[2938,2960,7916],[2276,2452,7572],[2296,2471,4306],[1525,1948,5046],[261,1497,2328],[2871,2936,791],[2466,2767,6454],[2420,2463,1684],[1363,1748,9474],[1822,2601,7083],[2532,2714,5691],[2245,2745,5504],[1817,2437,9156],[495,1491,3356],[805,1480,5311],[1381,2565,1973],[2746,2837,1765],[1980,2398,1447],[121,996,9386],[821,2881,9578],[199,533,1586],[794,1887,8205],[1186,1632,1926],[1532,2077,6705],[2589,2630,9041],[2176,2539,8832],[2795,2849,4267],[509,2498,2327],[627,2999,4120],[1318,2798,5105],[2844,2873,2990],[1523,1637,3917],[2480,2580,5691],[813,2305,494],[927,1165,8669],[2800,2937,7165],[2003,2555,8490],[61,1825,434],[1927,2945,4317],[423,622,3265],[313,2288,5919],[20,1058,5700],[2737,2850,7106],[111,1761,1257],[1928,2499,1855],[2602,2657,5770],[1006,1511,8916],[280,2701,6193],[1611,2536,904],[1459,2425,1923],[921,2297,456],[2357,2473,5549],[453,1876,472],[1648,2975,3552],[2655,2809,4517],[967,1939,1700],[1143,2829,3803],[257,653,2816],[1281,1403,9991],[468,2350,1658],[970,978,9995],[1098,2359,2484],[1693,1819,8189],[2503,2580,5878],[1288,1769,1203],[79,2080,2718],[2395,2819,4217],[98,1030,1286],[248,1916,5925],[2251,2427,6661],[2732,2895,9815],[1528,2103,5797],[441,2217,3693],[49,2393,349],[2370,2863,7819],[577,1266,6595],[1700,2303,6774],[1821,1876,3650],[221,784,5648],[2459,2484,9343],[445,1767,3678],[1069,1232,4331],[2030,2775,4997],[2432,2435,2008],[1852,2374,1683],[2241,2936,8667],[2468,2869,5411],[2606,2684,4850],[791,2434,8406],[2688,2766,8290],[2356,2575,8151],[628,1237,2563],[85,370,6442],[387,1790,1690],[2681,2701,3550],[2418,2894,6936],[2078,2521,9663],[2795,2992,5273],[2899,2914,7132],[2694,2923,1978],[665,1661,8856],[481,713,5261],[1294,1305,8946],[1495,2700,4180],[1894,2828,9915],[2455,2551,9264],[2921,2989,4229],[606,1380,1612],[842,2109,2574],[1850,2790,4471],[47,2141,9193],[55,891,4208],[1304,1973,3990],[1460,2770,7788],[2370,2805,661],[2726,2992,342],[752,1091,6134],[591,2876,4777],[768,2210,6809],[1270,1498,9717],[1787,2617,8827],[754,2889,7510],[196,1193,2176],[2405,2543,6231],[1870,2103,2008],[2391,2663,183],[2825,2978,513],[767,1027,9308],[2296,2965,8970],[2445,2806,930],[823,2606,411],[2612,2683,4194],[1659,2060,4975],[841,926,498],[2036,2847,1425],[1066,2310,1923],[1709,2764,7979],[1679,2768,1879],[1273,1527,717],[1396,1800,9000],[1900,2000,1790],[2089,2208,1506],[1360,2836,5573],[2697,2764,3742],[311,2830,3675],[343,1568,5692],[84,2060,3790],[1563,1843,3835],[106,2451,9890],[2816,2964,1502],[723,2558,6778],[1768,2405,355],[1623,2268,14],[585,2046,2193],[2270,2474,9803],[870,1675,5248],[1443,1621,9880],[2815,2820,8166],[120,2366,107],[1066,1476,4554],[859,2291,1332],[33,740,1580],[1610,2491,7263],[827,2105,3326],[2789,2803,3563],[367,2566,2844],[2873,2917,4548],[875,1285,2571],[1346,2970,8553],[377,2767,3436],[2771,2819,946],[134,2861,5431],[2943,2968,1812],[1259,2618,4714],[1668,2192,4921],[558,1376,4972],[2097,2951,1211],[1936,2281,2939],[2066,2427,2644],[161,1150,8962],[428,1649,6496],[1797,2562,6565],[2899,2916,6539],[170,1835,381],[2386,2850,2236],[519,1359,8738],[1671,2781,8257],[64,473,943],[1585,1740,5782],[1735,2884,6603],[972,1171,3034],[1126,2982,4236],[2801,2970,6198],[1723,2678,8584],[1862,2785,6170],[2183,2909,3830],[863,2040,9308],[1055,1821,2409],[76,2337,2868],[2280,2465,9612],[2683,2825,7066],[898,1557,5496],[22,1641,9732],[2194,2405,9642],[2520,2984,3224],[1723,2617,3340],[2798,2834,9195],[2699,2767,4148],[1674,1895,3076],[865,2401,9276],[2324,2496,2137],[906,1873,7367],[269,1288,3368],[1446,2775,1209],[2199,2385,374],[2805,2909,8415],[1922,2799,9488],[1759,2635,3607],[1226,1671,7135],[1534,1949,512],[147,1852,3209],[1001,2218,7978],[2176,2440,6217],[1585,1882,4799],[2343,2547,6198],[2503,2643,8276],[929,1115,7485],[1733,1857,4187],[1869,2423,6681],[2086,2630,8572],[787,1998,6387],[608,1439,5904],[1757,2481,6163],[1599,2845,4160],[163,990,4735],[1005,2518,4657],[572,1215,6234],[2284,2578,2540],[1032,2320,1407],[190,1575,9174],[2359,2892,399],[2093,2676,705],[686,1137,2328],[2083,2980,1738],[1090,1763,831],[902,2039,3706],[1568,2991,9275],[2919,2969,6343],[598,2063,3334],[1188,2091,5549],[187,1398,174],[128,355,9166],[1824,2539,756],[2635,2852,5103],[1815,2918,3790],[446,613,3864],[2785,2842,1766],[579,1346,3891],[1917,2934,5436],[2094,2359,8005],[1393,2342,2127],[1298,2387,7027],[1602,1933,571],[2465,2618,8802],[1288,2742,8524],[539,859,6110],[907,2028,958],[914,2512,3077],[560,930,3888],[2073,2764,4085],[1886,2902,575],[996,1391,4874],[1913,2985,2795],[1538,2867,9866],[2751,2980,4305],[441,800,125],[1665,2499,7960],[1113,2205,3972],[1795,2735,8897],[2763,2800,1225],[1390,1783,7814],[2913,2992,3400],[841,1982,2560],[1981,2360,6865],[2266,2815,3329],[1113,2047,2587],[1154,2467,6883],[218,1907,2608],[1430,1930,7534],[1951,2673,1451],[1791,1996,2798],[25,734,9296],[2912,2955,4700],[1052,1362,7465],[1966,2084,2162],[1313,2530,5231],[1284,1834,8264],[2729,2787,1430],[1635,2003,6781],[2320,2915,3770],[2289,2738,196],[588,2571,4121],[1297,1578,9855],[782,1684,5895],[992,1026,7567],[2319,2779,9373],[2072,2082,3133],[1247,1878,7138],[2893,2990,1439],[1611,1896,5477],[2009,2456,1393],[1479,2980,7734],[2510,2833,2198],[2262,2963,9975],[852,1329,1299],[1390,1473,9940],[2670,2954,702],[11,1480,2398],[1678,2639,1631],[1121,2627,7136],[2576,2818,5333],[2177,2915,2687],[2953,2992,1281],[2058,2608,3991],[2358,2863,1495],[1695,2920,8956],[1659,2414,1876],[573,1862,827],[382,2604,2448],[1323,1905,2865],[243,2961,7259],[2346,2768,7818],[1845,2852,6378],[429,1286,8031],[1298,2910,3039],[2871,2915,8118],[1538,1865,1337],[253,2928,6692],[2863,2985,4848],[1131,2950,3288],[2942,2953,539],[117,2363,9305],[2600,2805,3836],[1969,2432,1413],[2482,2846,365],[1274,1716,10],[1145,2571,5505],[1184,2292,6749],[389,706,9515],[2446,2805,9727],[2706,2961,1876],[2226,2944,8892],[681,2181,8346],[2735,2884,7678],[638,1161,9422],[2241,2427,7348],[2150,2175,2715],[2967,2994,9354],[549,1245,2819],[1267,1428,5636],[2010,2199,5092],[1384,1568,3021],[1907,2947,1035],[2795,2953,7354],[2575,2796,3957],[154,729,1035],[1355,2040,1084],[2218,2865,7821],[1391,1514,3034],[1236,1907,2513],[2492,2655,3406],[2688,2954,4263],[1834,2855,4422],[410,2538,3589],[1498,2690,1899],[177,2270,9822],[973,1499,8677],[924,1717,3260],[1068,2737,1072],[2555,2655,4145],[2388,2763,3086],[745,2824,5000],[2354,2368,2349],[1195,1963,6281],[2004,2133,7757],[1576,2729,387],[2770,2907,7717],[133,212,7417],[784,2482,2949],[2106,2293,7746],[1926,2430,9192],[1503,2116,7310],[535,1336,452],[1115,2720,2160],[2065,2963,7631],[1202,2839,5228],[2617,2803,90],[2102,2240,2628],[2923,2992,8041],[1057,1108,3399],[2706,2804,7484],[269,1276,1174],[2012,2182,3050],[2229,2561,2915],[884,1253,4257],[2341,2582,9446],[1055,1561,8078],[727,792,3576],[1310,2124,7992],[1761,2472,1101],[761,2649,8855],[750,1910,5070],[159,2993,7144],[2903,2947,967],[240,844,8481],[440,2144,4185],[998,1146,1954],[90,979,8198],[1863,2480,3384],[1871,2687,1402],[1727,2506,1292],[1833,2595,4008],[1140,2227,5465],[1807,2248,7083],[111,393,1205],[2212,2267,9951],[609,701,20],[2265,2660,8786],[2796,2863,2046],[1,1013,1235],[395,1888,6255],[316,2320,2843],[504,970,1250],[934,2084,8081],[649,2810,8861],[925,2464,9492],[170,1745,6023],[2873,2898,8261],[693,884,3782],[735,1294,8304],[2813,2826,2701],[997,1730,6142],[1486,2463,8191],[1635,1899,8506],[2888,2946,2628],[2454,2548,2106],[774,2969,9186],[303,2071,7636],[1419,2235,8277],[2694,2705,2628],[2539,2736,1325],[284,2453,2729],[2048,2592,9108],[2093,2425,5937],[1169,2359,9867],[2099,2156,2413],[2311,2368,5290],[2096,2099,2522],[1409,1662,3390],[1977,2579,9929],[1903,2430,3276],[1014,2924,9746],[2632,2679,8607],[2868,2894,1550],[2430,2766,3873],[2606,2696,4046],[979,1426,6171],[768,2136,1429],[2239,2490,8413],[698,1910,8135],[2167,2704,2202],[2295,2932,1620],[967,1358,3761],[148,2427,1481],[2922,2945,3706],[1564,2593,5764],[2526,2763,2268],[2500,2510,5472],[1430,2454,8405],[1543,2766,8182],[696,1464,7747],[1187,2358,2409],[2831,2982,4909],[369,2857,7670],[1772,1869,730],[1733,2137,2330],[2343,2471,7054],[294,2729,3508],[1974,2172,9575],[2807,2904,1174],[604,1301,3073],[2721,2952,5912],[1203,2069,1831],[2848,2938,9556],[2365,2867,5378],[2826,2840,4237],[880,2706,5774],[2056,2669,3774],[20,93,7869],[963,1321,126],[1006,1528,8638],[1067,2493,4428],[197,552,5631],[2790,2842,2566],[1034,1869,4557],[2157,2726,3914],[160,544,6414],[2128,2783,7721],[1926,1969,9470],[2800,2942,4991],[2790,2941,6858],[1269,2556,5710],[1452,1520,6613],[2679,2934,6557],[554,1585,7815],[45,2184,8309],[2786,2999,9800],[2264,2532,9118],[906,1811,4984],[1177,2301,2542],[1274,2762,8488],[587,1742,616],[1346,1420,1572],[2768,2979,7134],[1110,2009,7294],[1264,2343,8945],[1630,2889,9120],[633,2805,4387],[1273,2418,2092],[1490,2587,1439],[128,2927,5090],[1423,2925,9178],[1287,2961,4595],[2714,2767,6310],[835,2246,6498],[2972,2989,6270],[1496,1769,2146],[2081,2372,8877],[232,1339,1408],[330,1217,5125],[859,2681,5900],[985,2258,5238],[2204,2968,6586],[616,1889,617],[1323,2081,9876],[2352,2975,5113],[99,2466,5105],[929,2574,5566],[2348,2812,3696],[997,1753,8593],[2543,2990,3648],[2533,2899,561],[1471,2782,4885],[2868,2964,6956],[2105,2783,8736],[374,2863,127],[2471,2545,4135],[517,2502,8127],[2436,2443,5664],[1485,1707,8637],[2881,2972,1923],[817,2239,8310],[2136,2245,6305],[885,2018,7319],[1310,1412,8285],[1584,2823,3412],[2566,2961,5604],[1059,2348,627],[1949,2639,5738],[921,1278,8470],[2864,2973,5181],[2843,2922,540],[667,2178,2854],[1951,2966,8115],[773,1525,930],[578,582,863],[2664,2821,8094],[226,864,7070],[1522,1804,2243],[2787,2991,374],[2372,2876,6903],[2039,2524,6121],[191,2277,370],[222,2432,5992],[2059,2444,8105],[2412,2454,1556],[2726,2977,6740],[2677,2877,7869],[2775,2823,292],[2123,2141,5825],[1564,2647,1771],[2290,2474,4863],[815,1287,7589],[2825,2958,7365],[1509,2902,3652],[1869,2490,4034],[2470,2661,7083],[2516,2638,8933],[997,2119,3357],[1378,2300,1714],[2843,2975,3837],[2392,2490,6780],[1162,2233,2780],[1242,1257,8608],[25,2781,1377],[612,1216,718],[300,689,6902],[2294,2370,127],[34,480,2368],[2815,2934,469],[1191,2992,7810],[1288,2166,8058],[153,2345,4404],[1687,2989,1702],[1893,2229,3523],[1155,1390,2427],[2021,2492,5337],[1929,2220,6024],[31,1204,6340],[311,2148,7184],[812,1897,8312],[821,1618,3660],[1175,1980,1196],[1762,1909,9580],[518,1128,812],[889,1503,3856],[2012,2252,329],[2603,2901,7549],[1983,2178,8170],[1306,1424,7937],[2426,2887,4877],[529,2514,4987],[1486,2502,8411],[2855,2967,9938],[1276,1450,242],[449,2427,9046],[2693,2921,9627],[2289,2724,8332],[2481,2576,3447],[163,1731,5592],[592,2414,8146],[2792,2956,7010],[657,1314,9562],[220,1511,6564],[210,1749,4074],[2369,2772,5934],[2195,2545,2059],[2106,2718,4546],[2040,2741,798],[1673,1834,5778],[1083,1751,9733],[1571,2235,9988],[539,1208,6679],[444,2215,941],[578,2730,9619],[2033,2792,9431],[165,2159,9282],[2836,2900,2100],[2173,2915,3956],[813,933,1819],[2711,2837,9246],[2997,2999,2032],[2164,2387,4957],[1289,2392,1006],[2418,2531,6269],[1995,2569,2382],[1433,2267,8400],[2715,2866,5664],[28,788,9484],[184,2363,6376],[2636,2996,2915],[1718,2002,9188],[1559,2541,2162],[808,2586,3988],[549,1296,6041],[1661,2977,5973],[561,2146,8491],[2728,2918,149],[1874,1919,4803],[1992,2915,4876],[1374,2650,6390],[304,536,9793],[1179,1601,623],[803,1995,5171],[785,2078,3073],[705,1758,7930],[1630,2120,8625],[2817,2993,3823],[1969,2558,2072],[2341,2402,6613],[1075,1662,7446],[2429,2481,6621],[71,608,7526],[1127,2470,1080],[2784,2990,8668],[758,1420,6036],[1109,2621,6451],[2117,2606,4463],[1481,1787,4451],[562,2838,1883],[2109,2697,1047],[2329,2877,2782],[1989,2668,526],[2542,2731,3385],[2133,2172,2394],[2907,2992,8382],[2666,2734,8623],[1708,2857,3045],[2577,2795,8516],[794,2462,8202],[1014,2898,6668],[1999,2182,1568],[220,2322,3973],[389,1169,621],[102,578,897],[1531,2754,3071],[1934,2334,7543],[879,1107,5396],[1641,1835,5448],[150,536,1505],[2384,2423,2154],[2213,2365,1534],[2228,2307,7089],[2199,2888,2802],[1026,1101,6899],[866,1614,3830],[2841,2896,2898],[2576,2672,7685],[908,1363,7175],[426,2963,415],[1296,2034,8977],[1252,2174,2196],[339,1360,6024],[600,2618,8730],[813,2615,7360],[2018,2199,86],[2801,2902,1369],[2665,2739,1729],[2374,2647,1973],[1395,2891,4480],[1655,2538,4621],[687,1014,3553],[1751,2334,9175],[2099,2144,7196],[2786,2972,5224],[762,1269,7142],[1032,2883,1740],[25,2608,9761],[1321,1896,1908],[1535,1961,1590],[1971,2809,8214],[1941,2918,7862],[90,105,7981],[1595,2058,6475],[2643,2710,7126],[574,1109,1000],[264,1002,4175],[2115,2702,7792],[2284,2385,3413],[2255,2268,186],[894,2497,6958],[705,2758,6424],[1119,1386,2766],[958,1984,4033],[2369,2589,1072],[191,2822,5345],[2166,2902,8372],[229,2739,1900],[1498,2954,9942],[1053,1421,5998],[2664,2999,7655],[2020,2119,3929],[855,1818,7985],[1020,2010,7134],[2421,2778,8135],[650,866,3819],[1876,2526,1133],[1126,2875,9193],[2781,2828,3840],[1346,1899,6084],[589,719,7823],[951,1042,7862],[1526,2023,5504],[1304,2865,5620],[2333,2822,5582],[144,2213,8298],[139,1594,1791],[22,2004,5583],[159,1158,7144],[1788,2287,9289],[519,2269,2225],[1976,2113,6755],[1973,2835,9643],[1922,2889,4425],[754,2702,4420],[2993,2994,838],[608,2481,2448],[753,2228,6449],[1560,1782,4351],[249,2114,2387],[817,833,679],[374,494,7011],[863,1695,5453],[527,1749,7787],[1169,2946,5871],[241,2996,4155],[576,2023,2160],[1705,2122,9893],[2319,2740,4522],[2516,2960,3066],[196,2780,764],[454,1986,7599],[1031,1859,1978],[325,1721,9871],[1059,2747,5346],[1659,2812,4121],[887,1157,1511],[1267,1268,5836],[2448,2949,1638],[932,1007,2881],[1290,2088,6045],[667,2243,7555],[864,1640,3818],[1494,2132,618],[2993,2997,7239],[184,866,4189],[2291,2581,4803],[256,1607,4693],[2893,2929,7185],[728,1470,3934],[1465,2029,2068],[1742,2750,3026],[1832,2175,3432],[1492,2683,4255],[474,2433,3915],[201,1329,3448],[2984,2993,517],[1010,1552,82],[1697,2964,7553],[2119,2458,5251],[804,1387,8767],[280,2375,3607],[2708,2947,2902],[2737,2941,570],[483,1757,4685],[1900,2768,3080],[16,1763,5141],[1285,2689,3772],[769,1493,5915],[359,934,556],[719,1656,8145],[2323,2429,6851],[703,2916,7699],[74,1367,6812],[284,1887,2138],[1017,1643,9858],[2400,2761,5162],[2497,2876,1912],[2357,2772,4769],[1705,1740,766],[1704,2179,1572],[2505,2798,134],[114,791,7842],[592,1696,2028],[2239,2744,8884],[2739,2931,9847],[42,2441,571],[1796,2004,655],[2030,2314,5199],[1621,1940,8614],[1356,1559,3957],[2962,2979,3139],[553,838,2211],[443,2786,2197],[1275,2818,6509],[2314,2315,4824],[899,2975,6886],[209,467,9421],[1390,2380,3873],[1496,1754,2758],[158,913,7802],[971,2011,6494],[1145,2657,2077],[44,2979,6468],[1056,1459,2055],[692,2423,3843],[2690,2883,2000],[2881,2918,2745],[1645,1718,3342],[2331,2533,4776],[375,692,5775],[602,1616,4702],[1901,2974,8398],[2482,2962,8791],[1455,2110,1751],[755,1304,2657],[587,1690,5356],[1053,1693,493],[2809,2905,1935],[1966,2407,6128],[2507,2665,7909],[1533,2961,3100],[2009,2317,6478],[2758,2988,3288],[1097,2344,1584],[2295,2476,6220],[2300,2800,527],[1089,2874,8943],[2057,2541,4008],[1750,2938,5912],[2411,2530,234],[53,202,5417],[991,2680,4583],[175,2900,6056],[2573,2798,5640],[1283,1744,7859],[1650,2586,723],[1809,2608,2624],[2319,2474,3980],[1247,1389,9509],[433,1109,8681],[945,2141,3995],[203,1364,5986],[208,1232,7851],[1285,1981,1175],[2299,2421,7662],[2328,2719,4465],[2927,2990,9731],[368,1669,5245],[2419,2532,9311],[1746,2839,1550],[860,1895,6058],[2728,2934,5334],[421,2929,90],[118,1297,322],[1115,2093,1935],[1730,2319,9800],[785,870,5616],[106,2541,104],[2543,2661,5633],[2944,2949,8296],[574,1446,8346],[1692,2430,3919],[2864,2908,3511],[2449,2661,3202],[1187,2041,9249],[1341,1940,281],[587,2214,8682],[131,1012,764],[715,2413,3180],[632,1473,6371],[527,922,7788],[360,1183,7371],[2330,2507,5701],[649,1981,1488],[730,1768,509],[1877,2974,5383],[2830,2969,6403],[809,2739,640],[1728,1859,4186],[1843,2547,5520],[2138,2756,463],[481,1609,8413],[153,2441,4403],[2123,2426,2899],[2082,2862,8241],[1711,2062,9286],[666,2268,5962],[1664,2915,6245],[105,690,2382],[2485,2745,9949],[2013,2541,3400],[1543,2741,1423],[1575,1703,1035],[859,2064,7385],[2724,2778,436],[1909,2236,8085],[1514,2596,8871],[1715,2214,2056],[2913,2980,5834],[2417,2932,2480],[560,2044,854],[247,2200,8560],[2680,2806,1119],[1549,2156,6843],[1199,2075,8506],[2968,2977,2370],[547,853,345],[92,2069,5549],[726,2846,5253],[142,1384,7195],[304,2024,8908],[1946,2466,7347],[800,994,9859],[2090,2718,2824],[2267,2963,540],[2771,2970,5087],[1663,2230,9317],[1955,2800,9374],[895,1645,968],[2123,2454,7079],[338,1610,9005],[512,895,4452],[2863,2951,6139],[1718,2961,4260],[1029,2011,7777],[2169,2886,6237],[2757,2844,242],[1087,2517,9042],[1384,1391,9245],[1800,2240,5049],[188,519,86],[1779,2938,9456],[519,1279,1555],[1103,2246,3784],[1299,2277,1125],[1180,1851,8171],[2601,2794,5871],[2653,2954,4893],[577,1516,5606],[456,2489,6040],[1338,2992,971],[1402,1768,9486],[1458,1884,8730],[1915,2878,8393],[574,1141,7932],[2851,2885,2954],[517,1412,5058],[2122,2171,6064],[2611,2741,6006],[1020,2692,7292],[1906,2355,7988],[360,2003,1387],[2843,2918,5553],[125,2395,4439],[1342,1387,1360],[1960,2642,4784],[2733,2748,3122],[1979,2839,2690],[1336,2134,6455],[2706,2973,1521],[2675,2908,9527],[2046,2436,54],[2082,2842,7560],[2362,2408,4048],[2637,2683,9981],[941,2375,8386],[2278,2518,8623],[1321,1902,1129],[76,2549,7139],[1254,2791,833],[2182,2660,2911],[740,1715,6901],[2013,2486,4865],[402,2787,8193],[1672,2720,9075],[906,2973,4392],[1391,1671,74],[1827,2688,7478],[309,2117,4687],[1453,2929,4426],[2451,2744,3748],[701,2171,3153],[2188,2321,4662],[995,1024,5795],[2927,2945,3644],[1126,1690,341],[1879,1957,1979],[1875,2127,2631],[90,282,4858],[2839,2973,2200],[1480,1747,2722],[1306,2824,2741],[2737,2913,6801],[1425,1614,2298],[2282,2752,2483],[236,1623,3805],[2718,2813,6424],[1339,1822,5867],[2721,2732,8634],[2659,2780,9124],[1435,1743,9274],[94,2727,150],[1153,2599,8603],[2023,2483,97],[2088,2304,2997],[1394,2422,1702],[2607,2635,2564],[1128,1307,6908],[984,2169,6350],[777,1352,1561],[50,126,1155],[580,2242,2304],[2004,2238,5239],[1028,2516,1052],[676,886,8199],[133,2418,9499],[1748,2265,5478],[2791,2908,2573],[1384,2494,42],[1053,2258,6189],[1204,2228,4957],[186,2500,8022],[1289,2491,6946],[2706,2862,1951],[2424,2745,5193],[1247,1737,4392],[1644,1646,6943],[399,1777,3176],[2154,2173,2394],[284,563,3643],[1558,2710,7745],[1379,1823,8370],[94,2324,9143],[1773,2959,7973],[1454,2691,6494],[1331,2342,8700],[1312,2537,5825],[2166,2400,863],[2982,2986,7099],[1431,1652,6356],[1142,2262,8524],[466,1521,599],[2700,2966,345],[2224,2225,1470],[1135,2301,4],[769,1465,7419],[2012,2033,6259],[568,1907,2456],[2636,2898,5659],[1142,2079,1795],[2554,2589,3670],[936,1190,9851],[685,1026,2322],[2482,2801,2481],[1438,1976,1554],[1862,2563,5075],[2771,2845,7156],[2294,2409,4981],[230,2381,7422],[1757,2910,1494],[2810,2931,1202],[1256,2535,2941],[2029,2194,6249],[2380,2996,1409],[779,1168,9235],[223,1678,6593],[2813,2985,6774],[1390,1887,2392],[357,1619,7917],[746,1990,4682],[2241,2929,8284],[2542,2783,5716],[906,1889,3633],[266,1744,9598],[93,577,8635],[1679,2701,6293],[2593,2788,5281],[1530,2178,1817],[663,2331,4222],[244,2000,1876],[832,2264,5208],[2665,2862,4605],[863,1328,9184],[91,1092,3383],[2344,2959,3406],[2730,2823,1429],[2054,2225,8580],[1243,1251,2883],[2102,2979,5984],[2054,2558,5906],[1320,2484,2694],[1308,1317,6789],[2211,2987,6221],[852,1130,3639],[1761,2306,6044],[335,1100,9168],[1115,1451,583],[417,1014,3151],[369,2846,8216],[2067,2676,2228],[2606,2805,1269],[1727,2567,6062],[428,1328,5674],[1069,1659,506],[136,1385,6880],[1946,1975,7485],[2489,2868,401],[2110,2759,821],[1379,2047,3091],[1806,2705,7917],[625,2362,9795],[583,1461,6401],[32,2595,3830],[1465,2770,1294],[1732,2125,5826],[1796,2939,2275],[528,2254,5663],[26,394,7706],[1107,2968,8135],[308,2604,3614],[1210,2671,8599],[2898,2925,3133],[270,454,982],[128,2438,3235],[2644,2655,6049],[1105,1997,2582],[69,1830,3959],[1807,2337,6642],[2966,2982,4544],[543,974,1793],[708,2363,1085],[2698,2719,5531],[652,2259,5428],[1835,2505,8174],[940,1685,800],[2531,2913,6446],[2486,2982,4091],[56,779,1042],[460,2674,9998],[217,249,5202],[1184,2480,9153],[517,1657,5444],[2254,2321,8730],[947,1901,4889],[318,1808,5167],[670,2290,4841],[2972,2998,5075],[128,2377,7407],[2546,2935,2002],[2271,2907,5446],[2842,2908,7075],[1946,2472,1336],[2194,2848,711],[1255,2241,7576],[1900,2607,3843],[2306,2866,1323],[2262,2796,2789],[2573,2826,714],[762,2192,5990],[1564,2406,9889],[441,685,7053],[1239,2418,2573],[2918,2990,5787],[243,2764,3200],[2635,2961,4461],[1711,2769,4311],[137,2686,206],[2342,2602,1084],[2126,2149,1296],[1540,2474,1721],[887,1479,7079],[725,1292,3773],[2365,2950,4201],[285,1807,4455],[1792,2772,2705],[1385,1728,3093],[2278,2885,8182],[2425,2556,4723],[1827,1943,238],[242,379,3246],[607,884,7378],[2651,2911,7473],[435,1623,9091],[342,1419,747],[2349,2937,3114],[2073,2281,3630],[1508,2652,6655],[2200,2371,6509],[2889,2905,9915],[2686,2958,4992],[1037,2665,436],[2820,2884,2765],[404,450,9418],[1071,2712,8513],[1164,1815,8346],[184,1805,1483],[1461,1870,102],[2726,2771,9838],[1470,2865,9463],[1940,1955,2200],[361,2121,821],[559,1261,6360],[2341,2399,1344],[2748,2803,9841],[1838,2563,9077],[838,1861,7496],[1673,2239,8187],[2670,2822,9457],[2038,2416,9643],[828,2420,9501],[863,2773,8535],[2888,2963,6501],[774,2379,2406],[883,1829,3406],[2100,2248,8495],[1952,2075,3472],[1537,2099,9030],[2779,2842,6267],[2048,2481,9793],[2930,2999,2686],[382,726,3581],[2046,2828,4286],[2608,2857,4787],[2067,2225,3452],[1117,1311,6442],[2610,2957,1396],[1436,2836,5406],[638,1430,5132],[2305,2788,7774],[1354,2034,720],[1879,2103,4404],[827,1104,9368],[85,2290,4693],[2809,2853,4220],[296,2209,7380],[1352,2925,4396],[482,2392,8148],[2141,2585,5159],[2007,2591,5662],[17,2463,9716],[2516,2771,8419],[2596,2694,189],[2532,2880,1954],[2232,2698,7159],[1053,1502,9300],[2655,2919,7089],[1804,2266,1586],[1552,2613,6508],[1728,2971,5725],[1773,2504,1810],[1574,2711,4544],[2322,2819,5532],[1592,2948,6150],[2811,2910,6559],[2472,2735,8053],[2449,2854,1104],[1859,1889,2971],[279,2567,7880],[2499,2909,2711],[1500,1961,6867],[1824,2244,6500],[372,968,9853],[893,2591,7],[1722,2988,6320],[2510,2639,1474],[161,2916,469],[2474,2616,3486],[2681,2745,3821],[244,1901,4852],[2635,2726,1987],[363,1867,7070],[2951,2979,3065],[1778,2054,4049],[2930,2965,7288],[1490,1732,4036],[524,2412,4842],[1559,2225,6114],[2939,2978,1426],[1516,2915,1585],[1025,2253,9502],[1500,2062,4595],[116,334,5248],[1363,2747,1169],[822,2955,5936],[1107,2680,4925],[1455,2124,7792],[1717,1962,5767],[1831,2437,290],[2726,2981,4882],[1133,1725,9024],[2940,2990,9979],[411,2130,7095],[753,1464,1060],[1902,2080,2762],[1853,1882,1453],[54,2701,6780],[451,1791,1782],[630,1079,1738],[1552,2683,8731],[2356,2644,3221],[2195,2443,3488],[2068,2587,2644],[2503,2933,8594],[673,805,9600],[479,1032,1798],[434,1519,8432],[1234,2256,3277],[2972,2991,1273],[1029,1925,5240],[115,2299,8064],[1712,2376,700],[1418,1888,4136],[978,987,3901],[972,1849,1800],[2557,2987,8645],[2369,2535,4930],[1654,2125,7317],[2212,2288,303],[198,2769,7181],[2359,2554,6776],[993,1829,5110],[1582,2875,8573],[2780,2798,2858],[2634,2725,7020],[221,1001,3113],[358,2360,6862],[350,1409,7852],[1463,2801,342],[585,2556,7780],[2080,2792,9227],[2443,2576,527],[1380,1471,7681],[61,2207,4398],[1139,1430,8628],[2028,2372,9714],[2477,2514,5655],[2016,2927,1170],[1709,2612,6358],[1719,2754,9631],[1147,1741,253],[2378,2997,9793],[2607,2647,7283],[359,838,7760],[928,2927,3355],[1917,2724,3704],[2910,2919,4043],[1258,1641,7162],[1858,2050,1551],[493,1502,1166],[2754,2896,5579],[1318,1448,7938],[2844,2957,4946],[1004,1355,150],[772,2334,6061],[1172,1659,9882],[2850,2891,9160],[1972,2913,7671],[374,2358,7179],[2816,2957,134],[619,2011,8544],[823,882,8307],[1690,2591,9220],[2992,2997,1663],[2629,2872,1983],[2241,2917,2288],[1076,1368,3581],[1206,1860,9978],[2130,2212,2410],[1203,2195,4562],[2288,2764,6533],[149,2169,30],[2048,2199,4879],[635,950,626],[87,1911,5618],[2496,2816,6490],[1562,1878,2202],[2334,2375,3340],[1323,2770,6652],[1736,1926,4147],[87,2420,9104],[452,1954,5397],[2334,2824,5946],[235,1157,2272],[2071,2846,2469],[2449,2493,6088],[105,1776,3218],[1017,1700,6501],[921,2796,5337],[868,2166,5642],[167,191,7743],[307,471,3189],[621,2123,9366],[2276,2646,3597],[2457,2475,5342],[1350,1559,3333],[1218,2964,5595],[949,2128,1977],[2167,2999,5536],[1080,2069,2000],[1225,2018,734],[2094,2813,8754],[1225,2854,3391],[714,1228,5483],[1676,2484,8821],[1457,1587,3338],[2409,2699,119],[2501,2899,278],[2065,2271,9722],[2603,2714,1003],[494,557,2464],[72,2698,1466],[326,847,3219],[168,2799,1671],[2983,2989,6621],[68,744,9390],[1286,1540,3465],[2870,2887,6729],[1608,2453,8190],[2183,2426,8310],[2117,2989,5104],[90,2463,1229],[1713,2542,9101],[618,756,9497],[292,1366,8082],[2037,2065,4782],[459,1110,9329],[1728,2381,4874],[2316,2861,8668],[2296,2442,3030],[2782,2802,9750],[2776,2925,2043],[1907,2636,3900],[305,408,2403],[873,1237,3503],[1272,2361,7801],[2430,2880,9571],[874,2636,5865],[2994,2999,6470],[1581,2599,5221],[822,1303,3937],[2397,2670,4659],[1680,2022,9262],[1365,1773,7964],[305,2677,1717],[2514,2916,5608],[2629,2760,6668],[2599,2887,2587],[2739,2795,745],[2307,2708,3311],[2670,2830,7594],[752,2588,5766],[1701,2189,9257],[2518,2610,9491],[282,1392,2700],[457,864,3720],[749,1087,7568],[2491,2897,9733],[555,862,5799],[2121,2278,3245],[1535,2595,4195],[834,2293,8529],[846,1123,6344],[2943,2954,6270],[1375,2498,8775],[2042,2263,3027],[330,572,755],[1545,2387,9440],[1748,2868,325],[1772,2804,7972],[83,225,6682],[1290,2199,9465],[2412,2788,3612],[2741,2931,4381],[2490,2624,3617],[1910,2021,7153],[1672,1871,4420],[1956,2274,5910],[2436,2516,9090],[1378,2133,9188],[1820,2672,1767],[615,2937,9454],[302,1890,4938],[586,2693,7951],[1602,2607,3827],[1673,1914,8231],[1313,1559,9049],[2771,2955,1396],[1537,2443,8542],[2554,2812,3849],[294,2771,844],[2099,2225,885],[223,2428,5694],[503,2054,4614],[92,596,9493],[1208,1329,3056],[1171,2589,5576],[496,1360,6139],[19,2951,7680],[2814,2857,3744],[333,2297,1317],[2591,2636,9853],[2367,2821,6959],[96,321,457],[546,1741,7833],[1404,1488,8646],[1601,1833,5601],[2768,2843,2732],[2765,2779,9135],[2801,2963,2440],[2410,2532,5352],[329,1360,7418],[2062,2734,5726],[2500,2807,2564],[2824,2909,1601],[275,1814,3483],[2919,2998,6659],[933,1376,8347],[2405,2434,5207],[2015,2404,8825],[517,2611,6334],[1450,2900,5686],[2518,2810,3736],[1307,2020,3029],[1857,1951,2977],[2120,2886,7615],[2633,2737,8424],[2260,2513,9258],[514,694,2906],[2844,2879,1004],[1985,2850,9359],[696,1795,6712],[1086,1622,4918],[1505,2974,97],[134,2048,3693],[1325,1726,1496],[1377,1462,7478],[414,2783,7520],[779,2332,7201],[366,2478,3787],[359,1902,8206],[942,2905,4008],[2610,2686,791],[402,2117,3746],[2289,2743,7624],[2765,2869,6455],[2161,2795,629],[2221,2936,3787],[2038,2550,8696],[732,1442,7494],[2779,2840,9703],[1351,2519,9355],[1907,2997,3842],[1231,1915,8034],[2249,2263,790],[1781,2280,2874],[1330,1815,5470],[2060,2471,9221],[852,1243,81],[2334,2679,7160],[2659,2957,5063],[286,1373,4358],[928,1646,7090],[1971,2466,2672],[970,1304,7018],[900,1839,917],[1601,2522,9639],[2422,2450,6948],[2059,2823,5151],[777,1164,6317],[182,597,6286],[1322,2805,4901],[1268,1682,6606],[1143,2041,7300],[1093,2437,1826],[635,1770,1005],[1021,2139,4517],[2243,2844,6372],[2463,2513,5983],[73,1373,581],[2158,2649,6473],[6,2716,8954],[2829,2852,1550],[924,2765,2423],[2405,2870,94],[1770,2377,5310],[854,2356,4738],[1762,1775,5997],[2294,2523,4962],[2126,2487,2729],[704,1108,9720],[1525,2055,6335],[1112,2903,8232],[2960,2990,1753],[376,1044,926],[447,707,9163],[1431,1871,697],[2067,2547,2876],[2031,2588,7365],[2339,2342,1276],[307,1219,9542],[851,1226,6646],[1896,2854,8988],[1369,2366,5725],[596,1759,1466],[2305,2934,2965],[1945,2107,7450],[186,2683,2797],[511,1391,4657],[2573,2653,5791],[175,2746,703],[1191,1222,8724],[1496,2793,4938],[1014,1240,6193],[1437,2136,1466],[1021,1212,1350],[1373,2305,1240],[1319,1902,599],[2307,2319,5038],[588,1855,7638],[2157,2194,8014],[2605,2790,7395],[1607,1764,8394],[2792,2853,7656],[546,2389,6428],[866,1580,2968],[1263,2369,720],[1609,2799,7294],[1296,2586,1464],[1781,2767,5262],[1419,1556,559],[641,788,7444],[2,2710,8026],[193,2221,232],[1652,2034,5934],[967,2896,5807],[2696,2935,1445],[1722,2970,1156],[1522,2770,9971],[217,2867,9899],[1409,2450,5760],[2432,2495,578],[2977,2983,8817],[1961,2722,9995],[1575,2477,5813],[1494,2084,7280],[58,2074,6910],[2966,2968,5750],[1584,2063,1264],[2898,2942,7772],[1230,2302,5787],[141,166,7193],[2888,2911,5744],[2126,2538,8268],[2860,2871,2640],[502,2201,1169],[2926,2974,1263],[2190,2312,4455],[2684,2898,246],[115,2469,4546],[2984,2988,2193],[1607,1727,8430],[1753,2663,8356],[614,1707,649],[880,1434,3267],[2894,2989,5674],[1868,2208,6466],[2400,2840,7641],[1609,2306,2677],[2366,2592,2999],[1735,2371,7560],[432,2571,4447],[571,1612,3186],[879,2657,9176],[10,56,8463],[1419,1647,4110],[1369,1648,2287],[1206,1339,9642],[5,1326,1421],[2037,2368,8612],[807,1643,3151],[1494,1902,9095],[2787,2814,7548],[2151,2455,8631],[1400,2695,6241],[331,409,1855],[845,1422,9608],[754,1670,345],[1933,2881,3909],[961,1421,8396],[1145,2336,5067],[46,2237,9677],[2824,2882,3794],[466,2321,7303],[892,2189,2436],[2969,2998,3455],[2403,2635,1131],[843,1647,4242],[899,1100,4139],[1552,2537,3746],[601,851,4563],[312,2291,5092],[1435,2352,5900],[1611,2664,4521],[44,2723,136],[1352,2352,5745],[2011,2747,3431],[397,1752,8599],[4,2966,1958],[2572,2980,4745],[2155,2993,2418],[529,1641,3721],[162,829,3701],[1519,2597,8987],[2393,2555,6588],[2015,2204,4730],[1094,2376,113],[2275,2502,9536],[2846,2912,1256],[1043,1291,6798],[1503,1513,3538],[1662,1755,7731],[2223,2445,3868],[1842,1921,5877],[1413,1512,1479],[1791,2117,4055],[642,2659,4419],[2663,2670,1547],[1587,2049,1623],[2894,2987,2931],[541,1161,2514],[893,1537,973],[2327,2405,3381],[843,1882,262],[388,1305,3011],[1110,2311,5450],[2040,2907,9259],[858,2026,1757],[1061,2997,742],[904,1833,2554],[421,1444,934],[361,1821,4058],[298,2734,8483],[729,1880,7444],[468,477,4868],[649,2744,6697],[1041,1718,3376],[1997,2065,1228],[31,1315,3906],[1250,2879,4028],[357,2030,2377],[1426,2439,9791],[23,344,1339],[594,2296,6976],[447,1437,2544],[1639,2839,2073],[1242,2826,60],[715,1931,3613],[2346,2965,3989],[1758,2953,3429],[914,1836,4342],[1154,1164,3914],[2662,2853,7021],[572,2010,5681],[2638,2663,2018],[2906,2911,2944],[1141,1603,3285],[621,1726,6769],[1268,2148,4820],[331,1913,6158],[1542,2848,7453],[2149,2508,8907],[1803,2239,2622],[190,1860,5997],[1292,2846,6785],[547,1415,3226],[1790,2582,6417],[146,1508,6842],[2331,2376,1047],[2762,2788,1072],[2417,2803,2151],[707,1829,1943],[1100,2264,413],[1595,1869,3025],[79,952,4670],[2896,2921,1967],[2901,2980,9901],[160,1948,9647],[2943,2970,666],[1704,1883,2929],[577,2357,6437],[544,1770,1280],[248,2020,5108],[2680,2821,9255],[1891,2269,6018],[1762,2925,9402],[2149,2439,84],[306,589,2868],[161,387,6945],[966,1408,6087],[2790,2998,1031],[2338,2436,9055],[1887,2389,1334],[206,299,6020],[1963,2280,4547],[1725,2215,7184],[2261,2446,5956],[56,1648,2242],[2756,2759,6575],[2688,2968,3540],[1036,1064,428],[691,703,4941],[174,2608,8343],[2566,2622,5581],[1754,2946,9555],[512,987,4686],[2799,2810,2903],[1123,2572,1882],[1141,2670,777],[236,1937,2416],[1023,1274,6579],[1266,1965,7295],[2627,2784,7401],[34,2361,1044],[2044,2307,3333],[340,2827,3864],[905,930,5736],[2496,2685,1139],[311,2836,5685],[1024,2948,4455],[1599,2784,7929],[717,2728,1439],[2794,2943,8425],[2863,2896,3856],[731,2801,3906],[298,904,7077],[2601,2961,3590],[2689,2714,3263],[1176,1591,4053],[2846,2967,9515],[1549,1825,1890],[2300,2935,9537],[603,1975,4193],[1620,2890,1536],[2707,2819,8326],[1797,2401,8076],[1620,1984,217],[1545,1739,8143],[2657,2894,9252],[2653,2788,4309],[1032,2552,8609],[1286,1465,3617],[1130,2695,2658],[1603,2704,4103],[2437,2540,8721],[220,1973,1510],[758,902,3366],[1094,1761,7390],[2922,2926,2099],[328,2651,7858],[105,2276,9404],[2915,2928,1195],[2284,2938,4085],[2124,2849,9050],[1525,2643,7082],[356,458,4941],[1993,2156,2506],[2285,2432,4265],[865,2746,5567],[1328,2729,2308],[1295,1659,6366],[2028,2985,4824],[2243,2295,4164],[2250,2803,32],[2897,2927,1025],[2315,2516,9371],[151,2956,1335],[636,2803,9192],[1141,2725,5329],[28,907,405],[2678,2946,3037],[2000,2241,2719],[2956,2975,3222],[2664,2797,4091],[833,1987,3196],[2386,2642,7976],[2847,2949,8595],[2147,2951,4883],[1280,1918,3757],[1791,2551,88],[369,2508,4259],[2201,2868,9879],[517,1007,1527],[2651,2784,7849],[296,456,626],[1143,1406,6265],[36,2157,8214],[2494,2556,7811],[450,2851,128],[1151,1532,2815],[1736,2246,1094],[2703,2770,3701],[1529,1827,8941],[1417,1567,2535],[2678,2998,3937],[679,1189,4720],[457,2482,369],[2308,2715,5050],[746,1411,4637],[2124,2373,5789],[424,1813,6487],[410,1447,6575],[1860,2992,7178],[1628,2129,472],[2708,2987,2297],[1859,2315,9645],[1485,1925,3406],[1619,1906,2703],[134,797,6766],[1838,2676,9806],[1388,2091,4143],[1655,2670,1042],[2801,2997,8131],[2212,2999,4109],[2254,2424,3424],[2699,2986,4625],[1832,2454,9395],[1055,2664,7024],[2491,2870,8960],[817,1730,4740],[1497,2962,5120],[1884,2966,3048],[161,191,4084],[1580,1963,7099],[829,1108,3642],[1643,2688,387],[511,789,6932],[2955,2967,109],[2883,2979,7114],[416,1621,3642],[956,2416,3172],[1310,1520,300],[2246,2810,7838],[2535,2953,421],[130,1199,1069],[2894,2975,8381],[1051,1153,7636],[213,1260,5805],[1318,2443,9878],[725,1133,4308],[916,2180,6202],[207,1951,7144],[630,2296,5510],[260,2313,5824],[2717,2980,1444],[1179,2712,5779],[2419,2807,8711],[458,2937,4016],[545,1245,4202],[984,2107,317],[1608,1700,5892],[1786,2010,4967],[2643,2931,2540],[2064,2248,6371],[1939,2837,4023],[2509,2670,8452],[823,2534,1830],[927,1785,4960],[2536,2872,4222],[1616,1891,4527],[1941,2947,9075],[948,2915,3108],[1747,2145,5754],[1760,2968,1044],[2234,2558,6757],[2876,2900,4565],[1497,2591,5097],[1267,1406,1165],[629,1916,7362],[613,955,3614],[762,2434,5310],[2947,2950,4031],[1444,1648,7398],[2529,2691,6880],[990,1755,16],[902,2742,5642],[2930,2959,8546],[2112,2176,4169],[1369,1403,8301],[2134,2380,6880],[2507,2652,2254],[2331,2601,7689],[1863,2530,7295],[2511,2942,3181],[1050,2044,7509],[186,704,5317],[2853,2947,1888],[2728,2993,6013],[2424,2949,3937],[2529,2873,2276],[2582,2968,5932],[571,1053,9014],[1392,2906,5061],[544,2000,3944],[2669,2697,6570],[886,1638,1789],[2934,2936,2064],[248,1885,6886],[780,2118,3673],[2783,2903,9376],[2685,2928,4929],[2893,2946,41],[2281,2647,5664],[2757,2950,4645],[2781,2812,2025],[2180,2742,7404],[1645,2275,5281],[80,1762,206],[1269,1802,2519],[1555,2121,8589],[761,2699,4202],[2596,2801,3863],[2711,2798,8159],[2874,2976,8239],[1153,2447,1791],[1482,2139,6869],[2899,2941,8953],[2801,2915,5159],[2821,2865,5891],[2528,2961,5550],[556,1543,9203],[1862,2471,1459],[2758,2966,6712],[86,2234,7420],[168,1238,5452],[1377,2717,9037],[2359,2720,4314],[1754,2642,6909],[194,2876,9842],[1845,2469,27],[714,1650,4686],[2161,2259,135],[1778,2157,277],[1615,2089,2810],[869,2789,9710],[872,1146,2713],[2568,2608,7093],[2304,2544,3552],[72,2210,7204],[2258,2753,6995],[242,1298,410],[2730,2834,4830],[1819,1952,8264],[2071,2880,4809],[1197,2728,8893],[1313,1613,4047],[1154,1862,7492],[2399,2748,4339],[1393,2272,3964],[57,2144,6513],[799,1938,4088],[1362,2907,7234],[2549,2967,5797],[2456,2572,9208],[1326,2732,1487],[2771,2847,7776],[1364,2213,353],[375,2112,7858],[2772,2969,6099],[1545,2603,964],[2700,2734,7680],[2456,2657,6768],[1876,2202,2318],[1510,2887,9758],[2178,2744,4939],[319,1641,6502],[2625,2647,3614],[362,2188,6778],[1926,2943,9631],[778,1707,671],[518,1746,5545],[1060,2366,6208],[2490,2884,8834],[1457,1662,5313],[1252,2404,878],[2098,2197,9524],[1747,2528,7953],[2868,2965,1993],[2086,2421,5335],[776,1379,7149],[1178,2137,7627],[39,495,1840],[1361,2111,3050],[2657,2726,5994],[940,2200,7075],[391,2491,1647],[1575,2223,2964],[2115,2519,1001],[2267,2972,7139],[2916,2994,4860],[1893,2057,8517],[49,496,8765],[656,1447,842],[790,807,7750],[1004,2398,3278],[103,533,112],[2665,2891,4184],[1755,2719,9895],[622,1875,5784],[2350,2392,3442],[1477,2716,2607],[444,776,5977],[2053,2943,8036],[309,700,9497],[2720,2915,5108],[2362,2661,3697],[2044,2692,3837],[224,467,5714],[381,482,1060],[474,2855,1779],[623,2237,7742],[1728,2416,1215],[2779,2918,1673],[2768,2962,5363],[2617,2769,7125],[1031,2162,2485],[1215,1374,9071],[1170,2665,5402],[399,1533,1052],[2246,2725,548],[564,2918,6209],[2373,2541,5644],[2168,2288,4908],[2772,2932,2475],[1115,1838,8635],[982,1314,9442],[1473,2211,5116],[23,2814,9673],[2812,2917,7497],[1233,2854,5912],[2486,2609,6835],[2122,2166,5082],[2666,2866,687],[1938,2016,640],[894,2805,9117],[1611,2904,7110],[1396,2926,8723],[1243,2182,9254],[293,2220,7320],[522,1435,7254],[1954,2142,8779],[855,1577,6581],[2968,2971,8036],[175,1488,200],[385,2798,6956],[948,1864,248],[1297,1755,8820],[1954,2072,6622],[2150,2494,9375],[2046,2935,679],[902,2441,6637],[547,2128,8541],[2670,2820,2352],[2526,2695,2048],[1621,1964,4683],[2952,2976,8398],[1782,2584,1781],[1180,1733,2203],[2750,2857,2416],[681,1633,6890],[998,2576,1029],[2411,2471,5664],[1566,2305,6421],[939,1399,7358],[2027,2105,4193],[449,503,8616],[2632,2901,4854],[1145,2141,6062],[2804,2932,6],[1782,2039,4643],[1032,2111,5600],[1818,2924,5967],[2232,2389,6979],[1879,1933,7659],[671,2577,7887],[1727,2081,8431],[1000,1987,880],[1614,2413,4256],[2637,2844,5978],[1372,2479,3251],[777,913,290],[507,2744,9910],[2928,2929,6977],[1303,1531,8555],[254,2664,9708],[1158,1987,5499],[2137,2300,5447],[227,420,832],[134,2992,1960],[229,530,932],[2669,2937,3963],[2626,2838,6825],[1476,2362,681],[2600,2898,3140],[396,864,5190],[734,1725,9147],[2602,2701,5477],[2350,2862,4254],[2966,2978,6691],[703,1798,3943],[783,1133,6697],[1061,1494,5369],[2574,2648,2322],[2491,2792,325],[2370,2992,7932],[347,1799,6714],[1000,1260,8690],[86,2596,1643],[2284,2585,5410],[1916,2901,5150],[2876,2932,9004],[1692,2521,6560],[407,700,3535],[2259,2831,8194],[2478,2970,4256],[805,1277,3520],[1257,2774,3201],[151,158,3506],[890,1304,5707],[571,580,4676],[1054,2638,1734],[238,1503,9331],[1203,2016,7311],[2651,2930,8955],[2778,2787,9183],[1111,2836,4119],[1914,2388,8917],[2814,2946,4125],[2631,2775,4043],[1645,2073,9468],[2703,2934,5482],[1352,1438,6527],[818,863,7349],[254,1640,7489],[2110,2372,9812],[1080,1388,4048],[246,2962,3429],[2220,2529,9109],[625,1129,8598],[2837,2902,5057],[250,752,7031],[2698,2838,6769],[674,2909,7939],[1427,2157,3031],[2499,2943,1281],[2707,2940,8250],[1610,2957,5447],[2840,2882,3805],[2069,2295,1716],[574,1413,4361],[981,1655,3167],[242,2143,4570],[1409,1599,861],[647,1321,1330],[56,2017,2567],[2951,2957,9473],[1938,2712,9960],[1095,1715,9869],[1902,2914,6043],[2942,2980,4280],[2684,2910,6054],[2223,2465,1799],[906,2954,8372],[1844,1891,6208],[724,2659,5546],[1618,2730,6641],[1762,2928,8758],[1446,1853,6636],[130,1470,702],[340,2497,4273],[256,2530,3537],[1273,2008,7027],[2717,2948,9706],[1258,2368,9803],[1658,2235,8219],[2774,2849,6647],[148,2032,9957],[2356,2533,4894],[464,2417,3104],[2762,2971,8969],[2245,2560,7455],[1115,1306,6036],[1434,2842,5853],[483,1229,1297],[2126,2782,4400],[2793,2961,217],[18,529,549],[2576,2625,5247],[2729,2836,8669],[1024,2895,1621],[1076,1623,8220],[1654,1752,1324],[1847,2933,6032],[2773,2962,5736],[1296,2159,9107],[846,1876,3719],[46,852,7315],[904,2447,2731],[2877,2967,2038],[1843,2873,7655],[517,2862,5588],[1871,2464,6841],[1568,1609,8972],[2103,2548,3957],[2586,2620,214],[467,2263,9633],[1192,2842,5740],[2988,2989,1501],[2698,2906,8326],[820,1862,4044],[57,1614,7817],[1047,1805,4077],[376,1511,3143],[2832,2897,9150],[1323,2148,7063],[499,2828,1878],[2803,2963,7459],[2750,2764,528],[31,2320,4864],[2986,2990,7105],[2078,2517,3809],[1201,2192,1676],[1053,1062,3778],[1332,2794,7118],[801,2277,4860],[211,2606,6135],[2558,2765,2050],[2458,2636,4910],[662,1005,5394],[2912,2956,1643],[724,2800,9001],[638,1922,4068],[677,2088,6146],[1106,2307,1676],[1405,2804,4105],[2412,2672,4492],[175,1918,1375],[2913,2982,9325],[1611,2077,5896],[2480,2880,5943],[1080,2093,1585],[324,584,3673],[2935,2974,9294],[1404,2772,8980],[2446,2793,6734],[397,2177,5491],[1582,2796,9698],[296,2633,3331],[1013,1108,3697],[2342,2962,9140],[907,1657,847],[943,1339,140],[582,2204,5617],[2959,2988,859],[1684,2652,4591],[326,2170,2930],[1060,1328,7647],[968,1456,8353],[995,1946,4236],[1858,2916,9974],[2648,2695,3047],[2252,2357,4059],[691,2909,8764],[954,963,7358],[198,2090,9149],[1502,2371,168],[287,1476,4126],[535,2034,1365],[2875,2902,9417],[739,1435,7235],[1538,2184,8918],[1685,2971,7271],[902,2939,8097],[322,2831,3392],[523,1435,1840],[249,1656,4482],[1383,2322,700],[587,2555,7923],[1057,1148,6387],[1872,2272,3030],[1764,2199,9770],[2434,2673,4965],[2928,2964,9738],[1831,2500,8162],[2753,2782,5541],[289,1223,2298],[2149,2327,5377],[2752,2937,3584],[1117,1222,3308],[132,279,7998],[2604,2873,7577],[637,2576,8657],[2881,2958,6547],[267,1012,9114],[809,2500,472],[1248,1262,7396],[692,1020,8019],[1551,2157,5111],[2705,2811,4256],[1345,1430,3057],[867,1831,9226],[2377,2821,5952],[2991,2998,2713],[501,2116,2016],[2883,2992,9777],[1638,1893,8732],[2069,2819,4877],[547,1861,4552],[2617,2837,9096],[2073,2783,2611],[567,1215,3008],[1647,2311,6004],[113,468,6853],[1258,1576,3187],[2477,2598,9338],[1312,1314,3267],[2648,2806,1162],[2983,2990,389],[1999,2076,1733],[1408,2600,2161],[559,2276,6598],[2761,2947,5520],[1503,2689,7705],[1875,2213,3750],[2256,2925,3160],[2188,2434,8294],[2440,2497,5860],[2010,2287,4418],[2619,2876,753],[2919,2993,5781],[2838,2971,1977],[1022,2811,6523],[1908,2382,647],[1447,2062,5293],[1453,2071,9536],[599,731,938],[582,740,6681],[2747,2996,7590],[2737,2945,7616],[2302,2333,6439],[988,1750,7061],[1375,2261,3937],[2568,2991,9454],[649,2660,7937],[931,2202,9884],[734,1015,6001],[1989,2135,365],[766,811,1057],[89,2105,4372],[266,1691,4619],[1583,2181,7423],[310,622,8247],[526,2788,7449],[2874,2966,7503],[1037,1122,3191],[505,893,4644],[327,698,9194],[927,1101,4639],[1639,2418,2842],[901,2398,6941],[2221,2251,2173],[663,1157,4650],[14,1693,107],[839,2112,473],[2209,2680,6730],[113,2141,4212],[638,1103,1029],[2176,2546,9278],[1373,2626,9912],[2093,2230,885],[1143,1475,9677],[1187,2766,9533],[2827,2844,2949],[1051,2587,7252],[2199,2595,2215],[389,685,7407],[2007,2133,2478],[2028,2561,8543],[518,1869,6945],[2250,2592,1004],[556,690,1613],[382,2748,2112],[2746,2874,9025],[953,1832,2572],[1940,2442,9567],[1215,2496,3353],[1495,2132,7322],[2187,2257,9524],[1711,1805,9159],[1068,1978,1161],[937,1037,5536],[1479,1634,9646],[129,2970,8934],[588,1329,5537],[1204,2124,5475],[2541,2669,7529],[1069,2250,4203],[2576,2864,1278],[2987,2992,428],[726,2976,309],[2174,2319,286],[2584,2864,6463],[2407,2984,6542],[2677,2831,6529],[2931,2941,8480],[1992,2788,5906],[958,1540,5997],[1440,2703,9888],[2336,2887,3085],[1705,1923,5583],[2614,2816,2824],[473,1124,5700],[1737,2034,7689],[1165,1257,3160],[2778,2921,805],[732,1871,4823],[1128,1349,1975],[1660,2442,7273],[955,984,9639],[570,1631,5849],[1089,2595,2074],[2480,2531,2039],[290,2408,2663],[510,626,707],[2619,2683,8269],[774,827,8379],[2081,2868,5643],[1008,2072,7512],[889,2073,6212],[2367,2381,1452],[1745,1777,7741],[2432,2535,3379],[1067,1543,2334],[349,2768,6970],[2610,2682,6619],[1764,2712,782],[2033,2443,6623],[164,1291,5933],[1662,2108,8053],[2793,2831,5167],[267,1259,9759],[2223,2769,1271],[203,485,5419],[207,1461,3718],[2807,2897,8202],[1323,2717,991],[2463,2820,2160],[251,552,8927],[1241,2504,4978],[149,2495,8858],[1296,2084,2733],[2980,2989,5956],[2580,2654,2877],[716,1972,7156],[519,2017,2309],[1537,1787,452],[1585,2693,4983],[2506,2915,1436],[1600,2808,4294],[1598,2873,3474],[329,1948,7612],[1372,2441,8174],[1732,2028,9352],[108,1120,727],[439,2213,8722],[275,1062,7204],[2785,2997,4845],[1460,1462,8261],[2886,2910,1108],[322,2858,3861],[1421,1560,392],[1411,2252,3346],[1531,2086,12],[2135,2967,7482],[664,667,8856],[2925,2940,958],[2499,2555,6474],[489,2261,8323],[2943,2977,6862],[1758,2006,94],[2109,2729,3213],[1102,1785,1986],[1864,2141,724],[1670,2853,5938],[531,826,3111],[406,1164,7619],[763,2098,2811],[773,1074,5337],[2131,2960,4167],[2945,2981,6617],[149,2738,3460],[317,511,8469],[2839,2869,7401],[656,1344,459],[2562,2855,1895],[2158,2856,7426],[1003,2392,9836],[891,2006,9965],[2774,2906,9846],[1728,1921,3403],[1347,1728,9044],[1634,2977,2938],[488,1282,4975],[1575,1973,6338],[2006,2374,8981],[2782,2832,6511],[2160,2881,5768],[995,2834,8452],[665,2804,8744],[2931,2953,88],[980,1449,5465],[962,2509,9886],[1163,2757,1696],[798,2118,8473],[2694,2872,5463],[50,1364,3826],[706,1446,6701],[439,1493,8190],[1074,1128,612],[249,2678,6544],[1021,1861,4069],[1755,2406,5710],[1896,2500,7370],[1856,2237,7466],[2313,2897,5328],[1323,2935,1506],[271,1720,8475],[1807,2801,3288],[624,823,747],[1088,1651,7065],[2935,2990,602],[473,2908,7867],[181,2867,3308],[2548,2573,6100],[766,2422,3386],[1375,2386,7703],[879,2879,4078],[1505,2151,6156],[2412,2437,7050],[2958,2997,3026],[1720,2635,6375],[824,1450,281],[646,1504,9785],[70,1756,8711],[2766,2876,1003],[2344,2411,2683],[1201,1353,7984],[2729,2878,3747],[2079,2649,5436],[1887,2725,2419],[2427,2952,958],[1732,2506,6714],[2021,2596,1798],[2029,2082,8294],[1489,2003,9683],[2983,2988,6882],[84,2686,4605],[31,719,885],[1971,2471,9978],[1771,1777,6354],[704,1084,7849],[493,1861,4654],[2292,2826,4863],[1805,2887,2073],[2092,2700,193],[2679,2996,2573],[1381,2702,2027],[2341,2828,1115],[1700,2737,8846],[1359,1510,2153],[2327,2950,5793],[2699,2802,5264],[2342,2836,7479],[2928,2992,556],[724,930,8861],[1004,2078,3693],[896,2576,631],[151,1399,2048],[1339,2202,1370],[840,2719,8659],[2292,2472,8019],[1004,1581,3183],[2781,2802,9567],[2021,2231,689],[2196,2572,7872],[453,2633,114],[1518,2218,3543],[845,1265,3304],[2716,2829,1887],[735,2773,9653],[2673,2843,2606],[2720,2864,1250],[1593,2731,2096],[1929,2018,1275],[412,1730,233],[954,1568,3783],[443,1420,2211],[217,2351,5833],[1337,2361,5097],[2203,2445,3189],[1512,2792,3608],[1500,2930,5423],[2523,2927,8640],[576,2574,5966],[442,2200,8020],[2815,2874,8032],[309,2995,6068],[794,2463,7759],[1968,2926,7425],[2032,2396,772],[2779,2983,5062],[2752,2801,9609],[1995,2838,2332],[2271,2509,2607],[753,775,338],[1429,1567,4959],[1654,2961,7440],[2544,2899,3932],[1,417,3110],[496,2902,3863],[2323,2861,5158],[1155,1702,2576],[279,1164,1705],[2985,2993,6816],[636,1154,4565],[2866,2995,2686],[1383,2021,6483],[2916,2952,9748],[1426,2213,9151],[1458,1630,5418],[2810,2914,1762],[2309,2480,5162],[935,2715,5946],[1636,2949,1036],[1115,2491,4164],[1567,1974,2941],[354,1827,6657],[2515,2706,2641],[2166,2859,6749],[343,2014,3257],[708,839,9223],[1352,1842,1627],[2535,2669,8515],[1959,2024,9249],[1229,2520,5931],[2448,2881,6491],[2935,2945,9044],[346,2561,7952],[1675,2106,6660],[465,2452,3844],[1834,2201,7515],[2547,2970,5457],[2798,2979,4507],[876,2026,2702],[1802,2546,9577],[2426,2878,4293],[2493,2578,2260],[552,1675,1095],[2314,2879,5028],[330,1519,3166],[2540,2954,4856],[851,1517,937],[2114,2945,987],[1897,2805,2870],[667,2916,8585],[306,771,7059],[2708,2981,9572],[1596,2695,4950],[1984,2792,3829],[1993,2902,7365],[1679,2357,3440],[1344,1431,3719],[42,1506,4478],[2808,2896,392],[2215,2754,5517],[2615,2702,2173],[1871,2672,1904],[2317,2908,6554],[2984,2994,5957],[2635,2760,1684],[2068,2332,1978],[2498,2964,237],[1338,2455,5259],[2885,2942,9282],[1520,2984,3293],[765,1734,4355],[467,1511,3064],[63,1631,5660],[2368,2667,6797],[32,1632,8342],[777,1365,9666],[1657,2843,779],[1130,1919,2003],[1446,2872,8284],[2753,2812,4686],[2784,2952,5826],[2401,2874,2152],[308,481,2980],[1617,2364,260],[2178,2692,5386],[989,2997,1063],[2445,2918,9007],[525,596,6682],[1624,1814,3062],[1982,2538,5455],[584,717,627],[1909,2606,6600],[1113,1996,6410],[1963,2345,474],[1606,2867,3218],[1732,2964,8557],[1888,2762,6397],[612,1610,2799],[2498,2633,5749],[2579,2955,8947],[1472,2447,2097],[370,1914,5067],[2497,2689,4768],[2317,2602,8995],[2436,2487,3867],[1329,1980,3021],[503,1205,8665],[1497,1520,5887],[1500,2714,3046],[2420,2593,8075],[2983,2986,5881],[1506,1986,8207],[936,1827,8486],[231,499,2208],[306,2011,9823],[677,1177,6113],[2711,2783,5879],[938,1317,4988],[1923,2726,3978],[1133,1297,9195],[1206,1746,3811],[2922,2937,966],[2086,2320,6059],[2394,2628,7138],[268,712,2375],[1537,1867,141],[2786,2974,4187],[501,1363,7041],[101,1982,7160],[2659,2995,4923],[495,2681,5463],[581,1801,1178],[993,1998,6560],[2420,2819,5318],[217,1548,9970],[2273,2290,3934],[2818,2950,2658],[943,2189,5100],[2150,2778,5124],[1869,2327,9668],[295,1586,2213],[666,1740,9857],[1753,2136,5442],[2733,2886,8085],[1721,1905,9128],[797,2052,3064],[1224,2191,9526],[1175,2556,8249],[1515,2732,389],[2612,2997,9042],[2924,2975,248],[2647,2996,9406],[640,2220,8785],[2446,2873,7925],[1847,2557,7215],[1470,2470,7133],[2062,2692,6841],[2842,2947,813],[2948,2970,7712],[1527,1801,7566],[1897,2648,5156],[1209,1923,6041],[450,989,3866],[1656,1833,8356],[558,1428,4548],[2357,2428,6955],[207,1271,7310],[2917,2955,4204],[1189,2932,6606],[2501,2628,4055],[104,2393,4417],[1532,2859,6716],[2965,2995,7695],[1214,2150,9196],[2090,2311,7366],[2873,2942,4320],[1664,1779,3222],[1278,2813,713],[1868,2062,5842],[617,1636,9434],[1209,1817,8898],[1374,1743,5429],[1928,2781,6447],[1918,2480,6710],[2891,2997,3711],[1767,2364,7175],[778,1821,8164],[859,1347,9090],[1498,2407,6254],[2615,2764,4025],[1414,2201,8455],[723,1916,7688],[960,1144,2895],[769,2737,1492],[2304,2349,8366],[1676,2293,9600],[1558,2674,915],[1348,2774,8784],[1627,2537,624],[67,1776,4833],[1443,2038,9835],[2060,2527,8432],[1651,2355,5187],[1160,1959,2740],[1643,2120,8164],[1023,1082,4354],[1586,2007,5139],[1322,2212,6560],[1523,1616,650],[685,2603,7780],[740,1207,41],[2754,2802,9599],[67,2474,351],[2741,2746,7282],[2027,2900,3078],[1902,2224,9742],[2080,2378,7989],[348,682,4200],[955,1473,6136],[2199,2769,2073],[504,2024,5537],[182,719,9919],[194,627,6839],[320,637,1087],[2969,2994,3148],[197,418,1905],[1028,1679,9331],[1189,2016,2709],[505,1158,1734],[2355,2858,8969],[1293,2633,2587],[398,2841,7218],[1932,2921,3909],[2235,2789,996],[1904,2592,3109],[937,2578,6294],[2332,2738,5347],[1451,2720,2227],[2797,2852,8853],[509,1088,3410],[195,1966,702],[1328,2906,362],[763,2139,8536],[1214,2148,9919],[947,1965,1134],[2162,2435,1811],[2734,2959,4998],[1087,1960,5809],[1708,2884,2874],[2294,2801,1567],[766,1237,7893],[1118,2118,8241],[2088,2330,4239],[2603,2637,2827],[1261,2128,962],[2584,2869,692],[1716,1870,6783],[444,2967,2185],[814,2255,349],[737,2298,6381],[2552,2960,5326],[665,2005,2179],[1777,2716,1517],[1571,2376,3790],[711,1341,4747],[2309,2426,3872],[2806,2839,8028],[1428,1690,7353],[2220,2891,4881],[689,1195,3276],[970,2757,7105],[2464,2953,2792],[789,2220,496],[517,1387,9254],[2546,2831,1959],[2717,2805,8765],[2651,2914,1169],[971,2988,3219],[748,2532,8537],[1834,2480,3737],[187,570,6841],[1443,1586,4909],[1423,2521,4955],[2517,2522,4713],[1184,2446,5244],[2834,2987,8122],[332,736,2306],[2481,2718,2136],[1753,2393,1790],[1836,2176,3816],[1143,1282,2462],[1575,1950,9076],[810,2558,9489],[62,2338,114],[2330,2338,7049],[1663,2636,4546],[255,893,2165],[2075,2937,5966],[1507,1792,4693],[2657,2802,9995],[1851,2543,9556],[2088,2757,6449],[2030,2580,43],[946,2529,5202],[2774,2794,5410],[457,1026,8454],[66,160,4793],[2339,2808,9503],[1467,2355,7102],[2497,2758,223],[289,609,915],[2396,2501,6372],[2495,2906,3964],[1064,2052,4698],[1368,1599,4015],[1405,1892,929],[203,838,1731],[1164,2118,403],[1302,1724,4691],[1595,2873,1045],[352,1808,8189],[2241,2758,9400],[2092,2247,4770],[1002,1344,9316],[83,2838,6166],[727,1409,5725],[38,103,8116],[2394,2986,6412],[973,1980,5551],[80,2023,1489],[1772,2504,4674],[2055,2670,5218],[1326,2506,1610],[816,2607,5127],[2425,2971,9826],[1802,2575,1773],[788,2530,3623],[2244,2521,3049],[2015,2043,5408],[1097,2039,7433],[695,1239,1878],[650,2385,5005],[2980,2996,7562],[1742,2681,8347],[651,1117,3053],[2655,2766,2351],[1675,2138,2127],[1925,2722,9485],[1562,2328,7398],[2462,2991,5935],[141,2828,1810],[1045,1344,567],[169,1496,3933],[1867,1892,9431],[2638,2783,5818],[2995,2999,588],[2723,2910,2937],[1868,2467,9074],[2789,2843,2110],[2085,2989,1909],[1186,2740,3961],[2449,2504,6471],[984,2183,8560],[291,778,9056],[804,984,3233],[919,2954,8338],[439,2569,7000],[910,1244,3811],[1224,2413,2331],[1237,2049,3565],[1076,1634,1281],[1243,1246,3754],[2914,2941,5456],[2071,2632,7737],[2796,2824,8288],[2941,2950,2060],[2246,2329,1214],[1413,2594,2581],[1262,1832,367],[1355,2456,5834],[206,470,3914],[2754,2823,1216],[178,2569,8652],[1110,1372,4388],[1349,2143,3118],[562,2049,6491],[2914,2993,4718],[2380,2697,7151],[1189,2584,8035],[1767,2590,6653],[655,2744,3048],[1418,2766,4745],[2421,2874,8039],[1167,2647,7989],[901,2933,4769],[1336,1863,7828],[638,1963,1873],[2106,2479,2202],[1676,2033,9251],[2472,2958,9901],[595,2629,8351],[1256,1967,1545],[2952,2954,6524],[433,1121,7232],[2241,2359,3358],[1683,2464,3885],[2300,2329,7542],[397,2657,6240],[1404,2672,870],[229,2531,4003],[208,1391,879],[535,1534,1151],[63,910,8238],[1493,1731,9531],[79,1845,8252],[2943,2958,1091],[2233,2885,8342],[548,1689,1603],[2195,2649,1426],[2678,2762,3690],[2166,2307,9890],[850,2075,4626],[1529,2154,10],[168,2397,3423],[2696,2972,9262],[2396,2852,7399],[211,469,7222],[2150,2410,7119],[2846,2852,8671],[1999,2034,1037],[1695,2562,3244],[1723,2312,5387],[519,929,919],[2685,2815,8147],[2978,2980,8964],[1542,2560,9019],[972,2681,8914],[964,2399,2409],[2055,2804,7508],[1424,1999,6791],[1618,2706,3016],[2699,2885,6162],[1321,2079,8470],[1918,2336,391],[2024,2294,2092],[728,2223,8887],[312,918,3963],[662,1332,8390],[2561,2969,2761],[1010,1700,6631],[158,907,5600],[740,2684,7970],[1127,1585,8598],[800,1875,1643],[408,1010,2809],[1076,1211,3785],[2297,2442,10000],[900,2264,8200],[2892,2992,964],[97,2445,9703],[2261,2946,7257],[2454,2753,987],[241,1072,3519],[2626,2938,4205],[483,2269,2105],[2803,2977,4564],[2822,2938,7099],[1295,2361,7558],[1459,1833,707],[262,1968,3082],[2191,2825,4960],[2447,2573,2418],[959,2253,8617],[2426,2649,2253],[747,1134,4730],[2348,2927,5700],[1817,2064,6748],[1882,1927,2718],[1971,2593,3802],[343,1737,1551],[540,1435,659],[2326,2832,5059],[95,295,6280],[1106,1838,5765],[2429,2833,6442],[246,658,3489],[2072,2098,9805],[1061,1118,25],[2318,2679,2121],[1476,2326,7730],[1608,2795,9571],[2457,2912,1323],[874,2247,5117],[1791,2696,8158],[999,1752,4071],[2153,2820,6752],[1477,2755,3625],[2413,2754,387],[2022,2597,1739],[350,2520,4488],[2908,2926,4944],[2987,2994,9233],[2287,2732,3298],[105,1083,2553],[1442,2847,4012],[1933,2344,9970],[1240,2368,4106],[1202,1764,1939],[1170,2996,2988],[1159,1598,9503],[116,2155,1299],[1268,1963,6198],[2098,2329,6185],[1177,2568,7556],[546,2332,3446],[380,977,4291],[1941,2004,2674],[2618,2815,7753],[317,933,9975],[1796,2480,8484],[1897,1986,6094],[415,2379,6788],[780,1538,6339],[1874,2786,9624],[1349,1932,2404],[509,1846,1531],[1126,2630,7304],[2966,2981,9117],[911,1003,9116],[847,2086,4322],[1136,2523,8417],[1070,2128,957],[1447,2933,5954],[2653,2903,1061],[513,2462,4827],[981,1215,443],[2257,2805,8645],[1304,2672,8379],[589,1344,9919],[508,2997,8916],[999,2070,8604],[1293,1815,4331],[2595,2634,5856],[2212,2417,1328],[974,1776,6536],[2737,2816,9359],[337,882,2623],[2754,2851,9722],[1986,2149,302],[1834,2179,1286],[1928,2369,1031],[2501,2893,4691],[1013,2502,3164],[122,681,5159],[104,2826,9615],[38,57,1256],[2340,2582,1486],[400,2501,7819],[1317,2560,7768],[1590,1995,5179],[1144,1251,8602],[608,2932,5455],[776,819,5260],[471,1018,794],[1247,2765,4771],[227,1771,3420],[2900,2983,4368],[547,1874,5429],[214,262,8056],[1247,2307,3931],[1422,1971,9686],[928,2658,4187],[1874,1986,4298],[999,1665,2088],[226,2734,777],[1670,2193,8887],[1506,2090,2183],[117,2584,2171],[555,2080,5117],[2060,2312,521],[1369,1904,5254],[1971,2806,3729],[177,2651,6701],[2504,2864,7406],[2016,2204,2016],[2721,2825,3507],[616,1400,1223],[1395,2307,5222],[2372,2519,1430],[616,2235,6512],[1078,1181,1147],[1073,1658,3892],[391,1102,4889],[2824,2825,5664],[1560,2117,7420],[1418,2739,5992],[2299,2361,4862],[2490,2492,9361],[1883,2526,4069],[2513,2899,6466],[2411,2559,9733],[1047,1347,5988],[1782,2692,9013],[2932,2978,3512],[370,886,7658],[2788,2894,3377],[1684,2884,8732],[109,2303,6730],[1760,2491,6376],[2360,2500,1387],[489,2121,1399],[1277,2233,3012],[1278,1894,9359],[370,1196,7504],[842,2858,5707],[2296,2794,3069],[199,871,5869],[2518,2705,233],[733,1666,5743],[750,1591,1185],[2666,2938,758],[1390,2595,6443],[1796,1817,1292],[994,1969,4815],[1300,2629,7526],[2432,2611,9812],[1275,2927,3218],[1515,1946,2796],[601,1768,7519],[1968,2578,1765],[2223,2517,9025],[537,2917,4630],[2261,2828,5618],[2995,2997,3480],[2277,2356,1852],[1368,2245,7883],[347,1071,152],[1561,2095,346],[1485,1860,3212],[2045,2088,2385],[1469,2438,5328],[88,1457,5862],[382,1170,3712],[1139,2748,3283],[551,2185,7520],[1089,1991,1771],[1264,2636,6360],[336,2282,1532],[1792,2893,1302],[327,584,4249],[311,769,5404],[233,2962,5902],[1539,1680,1739],[715,2782,7702],[1417,2651,9917],[2980,2992,6746],[427,1420,2795],[1413,2600,2330],[184,2109,1041],[531,1199,3461],[2716,2978,9284],[1146,1897,833],[2752,2905,8526],[20,65,8438],[2556,2656,8146],[140,224,7185],[1195,2104,1059],[615,1342,7139],[1093,2650,5616],[2610,2611,655],[2870,2968,2082],[1885,1923,6083],[1273,1396,5168],[628,990,3036],[2736,2996,2922],[2647,2847,8693],[2410,2516,3720],[336,2818,2],[260,1871,1381],[2382,2718,2771],[261,1860,1870],[1235,1909,4209],[141,315,152],[1860,2573,3458],[855,1152,5940],[2343,2801,9610],[1297,2400,7803],[1053,2948,7755],[2745,2915,8607],[2294,2875,5326],[782,1225,9036],[905,1674,2524],[1571,1897,2757],[1705,1999,8545],[2874,2950,261],[1078,1607,2170],[551,2453,7678],[1150,1749,3548],[365,757,3140],[2700,2852,9995],[2212,2804,8985],[2393,2606,9350],[1658,2392,5455],[2877,2884,6819],[819,1367,137],[466,853,338],[1671,1823,4657],[1454,2743,844],[2647,2760,6964],[2044,2048,5149],[929,2890,8787],[1353,2527,5202],[280,2931,4365],[1503,2641,5080],[596,1848,2019],[1536,2579,4088],[701,1903,2095],[1261,1686,124],[18,220,4511],[1932,2710,7985],[14,2653,5504],[2854,2925,951],[1535,2812,7069],[743,2897,207],[1803,2124,242],[457,2691,4290],[522,1334,6793],[1145,1416,5343],[70,2636,350],[1621,1844,5951],[2617,2717,9255],[365,802,1105],[477,2232,2835],[2843,2845,4390],[2150,2617,4276],[181,2249,2983],[2630,2967,3122],[2959,2964,8905],[1223,1678,7083],[861,1958,7145],[2799,2894,4299],[932,1812,8555],[1910,2084,4408],[2123,2149,6083],[1474,2787,629],[1387,2447,8772],[10,326,7569],[2186,2340,7415],[549,1088,5353],[2788,2886,9933],[1847,1869,5656],[937,1089,7992],[1338,1641,2241],[687,1197,3234],[2170,2798,9703],[473,2678,2266],[148,2861,3955],[120,2002,2033],[2186,2373,3243],[1513,2013,523],[2648,2768,323],[1003,2735,1136],[704,1376,5008],[2168,2777,418],[2394,2837,303],[2785,2898,4694],[1368,2721,4304],[2707,2747,1480],[254,545,9077],[704,1098,8377],[483,807,4145],[382,1875,8276],[2490,2621,4816],[2242,2467,3264],[2831,2980,9781],[432,2904,3775],[1840,2205,5168],[2833,2962,1761],[1538,1652,5242],[2550,2928,6962],[1186,1782,495],[2714,2994,5971],[1729,2876,8919],[1326,1643,2846],[231,533,2368],[2601,2888,8472],[147,1048,2936],[1768,2516,3368],[249,1256,9627],[2459,2759,9109],[1995,2256,4706],[1875,2119,1475],[962,2449,8335],[2841,2856,7888],[2706,2945,2542],[1624,2513,5782],[1620,2995,1754],[489,2767,4218],[595,919,9427],[1495,2578,327],[1822,2870,352],[952,2310,1275],[1038,2991,8902],[1367,2208,5730],[237,2016,9467],[1059,2145,7925],[1571,1840,6608],[771,2455,2345],[2315,2461,2868],[805,2003,5239],[723,2070,2712],[280,2497,8505],[1670,2722,3109],[1886,2577,2729],[954,1185,6732],[1856,1897,4538],[509,1923,2475],[2791,2981,2958],[1427,1583,6051],[496,2995,7183],[994,2257,4475],[1042,1289,3553],[2947,2977,7407],[1699,2636,9047],[2390,2533,4923],[1418,2472,4379],[1503,2199,8137],[1425,2703,4389],[1071,2100,7653],[2319,2693,5937],[978,2054,7013],[80,2661,2555],[772,2982,1007],[592,1447,1784],[1413,1785,970],[2626,2761,3629],[828,1866,6813],[711,1789,9792],[2325,2818,9929],[1438,2484,5525],[2578,2974,5719],[2112,2656,4352],[2661,2869,4037],[2519,2701,943],[1245,2011,6118],[2242,2273,222],[285,1599,3912],[1441,1974,1886],[2275,2376,1069],[2834,2931,467],[258,2522,3726],[2445,2565,8668],[2225,2851,6858],[2135,2205,6536],[352,2592,9727],[2833,2905,5113],[743,1287,2545],[1928,2393,4592],[2656,2936,3406],[2357,2956,2202],[2336,2403,1316],[2790,2814,6854],[62,1900,9322],[2432,2770,5845],[688,1199,5289],[2149,2683,169],[2099,2584,8965],[719,2889,8359],[1648,2779,4185],[268,2212,3642],[2100,2368,7333],[1926,2076,9901],[1046,2251,8890],[2119,2797,2009],[2962,2986,3905],[2465,2860,1490],[2222,2295,2592],[2753,2911,9208],[781,2278,795],[594,2176,4600],[1983,2402,5126],[1344,2799,5244],[2349,2508,6662],[2419,2658,3257],[998,2718,5444],[2465,2735,5929],[831,982,6951],[1026,1709,7249],[324,943,3094],[1112,1469,2787],[1100,2990,3903],[660,709,51],[105,786,1602],[994,1254,4211],[95,2779,8918],[288,1032,6183],[573,1805,3797],[2330,2475,275],[2052,2501,9531],[2047,2897,3523],[1199,2851,4011],[724,2110,8407],[2565,2637,3469],[513,2318,4594],[342,1940,2681],[2350,2598,4586],[592,1474,2414],[2132,2907,329],[2037,2967,3089],[1577,2911,7041],[2474,2924,4178],[960,2388,8070],[1866,2509,2417],[414,1934,8864],[839,2613,5633],[1032,1601,991],[2586,2814,6980],[2451,2988,7633],[1191,2805,4208],[955,1980,7357],[2501,2707,7969],[2338,2574,9636],[1178,2668,8524],[765,2745,112],[242,2283,813],[1055,2420,6249],[741,2679,7366],[1506,2034,9031],[1001,2685,5527],[674,1547,8529],[2711,2758,3520],[2853,2935,9967],[2022,2430,87],[1613,2931,2232],[2365,2392,7038],[2115,2985,8187],[385,2105,6875],[394,1126,3300],[932,2983,3883],[216,1919,4008],[2340,2449,5590],[1492,2269,2323],[209,330,2798],[2218,2918,384],[1108,2054,9409],[291,2259,6821],[1004,1610,6504],[320,2962,3383],[1540,2230,2546],[515,1809,4436],[189,1065,9171],[429,2694,611],[1498,1521,8168],[857,1109,7293],[1238,1602,199],[1767,2383,5401],[1702,2877,8922],[2810,2993,2312],[2821,2948,4726],[823,1856,4343],[2003,2612,269],[101,2011,8800],[233,1221,7781],[1979,2440,3064],[2253,2643,4663],[1875,2818,6573],[2351,2814,2894],[1480,1970,6051],[969,1638,390],[516,2885,2578],[2833,2898,7307],[2845,2888,3012],[2801,2894,1501],[1440,2177,8168],[2078,2774,4163],[1918,2438,9573],[1056,1197,2627],[885,961,7009],[360,2074,9679],[1443,2978,2150],[1956,2341,4186],[2419,2750,1960],[2600,2634,8687],[1578,2357,4763],[1570,2541,5750],[2069,2136,974],[1551,2792,9641],[2293,2973,7041],[1175,2639,1098],[2555,2795,2376],[1492,2095,6934],[482,2280,3944],[2635,2790,4042],[2368,2370,5566],[895,2725,2403],[2161,2584,752],[508,1151,3618],[1785,2141,894],[1500,2286,143],[624,1129,9447],[508,2106,623],[2031,2162,5963],[2599,2728,5128],[2410,2853,7419],[385,1185,9587],[1663,1832,5874],[2642,2862,4345],[1212,2629,1622],[318,1364,1242],[1318,1862,2727],[2057,2117,5964],[1114,2733,8091],[1630,2484,3019],[338,1969,6366],[1795,1959,5857],[2989,2996,1514],[1048,1095,9119],[800,2543,7097],[1573,1925,5259],[2375,2912,5723],[1544,1816,7489],[1815,2369,7140],[2958,2989,1818],[707,808,2270],[1979,2047,6114],[1128,2375,9435],[1915,2705,7590],[1422,1897,883],[2509,2962,2833],[2523,2921,6053],[2683,2843,1937],[2612,2740,5273],[360,2965,5494],[2492,2673,2437],[667,2145,7655],[630,2780,7763],[1091,1165,9189],[770,1692,7133],[2440,2796,6062],[1460,2626,7521],[306,1631,8042],[1639,1644,760],[1726,2706,3912],[1701,1906,3439],[217,478,5569],[1179,1457,119],[399,2304,7330],[118,1777,9942],[2746,2900,969],[2144,2376,6148],[724,2864,1895],[55,1228,7828],[1529,1768,6657],[703,1116,970],[2819,2956,9051],[2442,2641,9227],[1398,2915,4653],[2066,2650,722],[477,1458,6099],[1780,2446,570],[604,2803,2751],[314,2331,6344],[1481,2357,7130],[2361,2818,9781],[1154,1655,4732],[245,1472,3522],[1721,1887,8094],[70,2515,4398],[1899,2504,3642],[1628,2056,9136],[843,1278,7688],[1935,2379,7071],[1659,2447,9833],[1930,2139,5335],[16,242,7430],[2193,2601,6574],[1903,2585,5406],[2120,2506,8329],[660,2318,773],[94,829,150],[2597,2991,7376],[858,1662,6043],[2803,2871,3222],[2014,2041,9142],[1548,2705,4581],[2777,2924,469],[2955,2974,5043],[1194,2353,5713],[547,1061,9],[1938,2216,5753],[580,700,8035],[138,2737,4095],[986,1678,2084],[2861,2975,3749],[2877,2908,9887],[770,2500,1894],[2471,2611,9844],[871,1617,849],[1535,2451,8922],[2781,2935,6699],[50,451,1012],[1195,2787,5284],[57,679,8659],[2291,2991,1987],[1161,2462,3496],[1859,2076,820],[1982,2053,6791],[2380,2668,4509],[1636,1692,4701],[2953,2978,3272],[2413,2842,5630],[703,1148,1355],[422,2270,507],[24,1080,5739],[2068,2383,9998],[2825,2853,1633],[1166,1332,5175],[1013,1015,6853],[1567,2454,1958],[2670,2974,4002],[1630,2380,1773],[2385,2721,7238],[650,2952,6658],[338,1298,9165],[1168,2592,3604],[2484,2550,6488],[1689,2997,3554],[2449,2626,2209],[279,1783,3524],[1976,2815,6712],[292,500,4882],[1964,2703,4538],[2972,2978,663],[2148,2678,600],[1191,1324,2651],[276,2389,1596],[464,2082,852],[832,1911,1510],[1782,2489,1856],[1669,2075,6828],[57,2736,5859],[2390,2886,202],[2353,2930,6213],[1163,2728,289],[2274,2894,2329],[759,1433,3954],[97,1973,1531],[637,2500,5464],[1580,2026,1701],[636,830,2156],[886,1401,2888],[2120,2351,516],[2111,2456,2326],[352,1344,1484],[2914,2939,3423],[1526,2199,8090],[1792,2703,2415],[2432,2987,2890],[1640,2036,1572],[102,2012,7947],[2858,2919,1153],[454,1567,7886],[2943,2967,3002],[1940,2411,423],[2295,2526,5356],[1710,1968,3490],[1652,2969,8184],[862,1575,6357],[841,2019,3192],[1993,2372,8803],[1169,1973,5667],[2471,2780,9648],[1309,2589,4225],[1127,1136,6787],[2162,2399,6959],[120,1954,6567],[2946,2955,4216],[540,2416,4493],[1009,2309,9873],[699,1032,7575],[2627,2708,8395],[2242,2673,9613],[2310,2557,4723],[936,1460,6103],[2025,2676,6892],[2965,2987,6136],[318,2749,6751],[2256,2776,7005],[635,2436,6132],[2389,2923,4373],[2857,2959,7435],[2198,2953,832],[276,352,4396],[2947,2949,2442],[1001,1951,1480],[1091,1534,2587],[2698,2712,2011],[1690,2932,958],[2237,2890,9383],[1933,2408,1617],[110,1113,8996],[155,2087,8990],[2348,2918,4760],[2220,2702,9537],[2658,2716,1245],[1253,2092,829],[1810,2038,5825],[2443,2838,3540],[2379,2430,7974],[786,795,5585],[450,1933,4577],[1701,2096,6487],[1780,2106,9554],[2415,2962,190],[1903,2154,3175],[1624,1766,3444],[601,1920,5947],[582,1536,8179],[1891,1980,5902],[1518,2247,2593],[1902,1940,8659],[167,2515,5040],[1593,1718,9821],[1205,1851,7270],[1842,1884,1533],[1416,2096,682],[2402,2791,3285],[2367,2604,4745],[179,2562,5348],[1170,2742,6311],[861,2247,7455],[1161,1833,8958],[1363,2519,603],[1673,1955,5578],[686,2209,3368],[2339,2973,2274],[2082,2526,4814],[441,2463,6409],[1344,2186,5460],[440,1096,4283],[2793,2869,8648],[827,1753,7851],[645,939,9930],[786,2598,610],[463,1342,6905],[211,2686,8020],[1505,2289,144],[2772,2911,2087],[1148,1342,1567],[2145,2637,8713],[1545,2026,7675],[292,2767,3672],[481,1758,9308],[153,2694,873],[304,365,1780],[2811,2945,7762],[2850,2974,3627],[2958,2968,1300],[1854,1922,5932],[373,794,7971],[1164,2898,7443],[177,2615,6746],[1169,2090,5822],[1027,2193,5310],[1244,1558,1167],[477,2880,4601],[296,340,3376],[2744,2864,5860],[2674,2955,3936],[2533,2760,1970],[403,2781,1591],[2605,2732,5527],[1569,2772,3088],[240,254,2959],[887,1019,3111],[1867,2178,5007],[1880,2224,5161],[776,962,9332],[1441,1955,8375],[1912,2886,2045],[2383,2667,1139],[2194,2283,1778],[153,2990,9480],[1027,1957,8714],[2599,2917,9167],[1384,2496,4294],[2267,2997,4863],[2359,2396,9256],[246,730,610],[1982,2687,8396],[1307,2968,6009],[1351,2696,9047],[202,940,3501],[1165,2201,2272],[2709,2895,9949],[969,2768,2111],[2636,2897,6063],[2084,2912,5476],[637,2475,8434],[2558,2651,8336],[2221,2850,7185],[1761,2970,7906],[1846,2595,3493],[384,709,2313],[1389,2283,4222],[2805,2924,6098],[2762,2861,5938],[2678,2815,2910],[1746,2625,8439],[2471,2486,6837],[59,2614,8831],[2404,2546,1131],[2873,2874,5406],[1720,2153,2726],[1827,1900,3809],[2289,2813,1990],[2348,2511,9539],[2212,2214,2173],[2041,2597,6503],[1308,2360,6202],[2225,2701,2116],[725,1068,2588],[1916,2596,7939],[1242,2904,4931],[2605,2779,9592],[166,675,3161],[1458,1694,9423],[624,2729,1115],[2928,2981,7774],[1856,2849,5197],[2512,2859,68],[497,1785,2291],[641,2976,2336],[1162,2817,4389],[2546,2845,5624],[350,2663,638],[918,2207,2046],[1415,1492,7006],[32,1948,2962],[1983,2634,4347],[323,2601,1938],[2356,2516,3873],[2807,2888,3572],[2760,2841,1356],[2452,2822,319],[325,2081,1589],[2617,2988,2935],[2117,2338,5327],[2505,2813,1534],[378,1496,1394],[2202,2347,8391],[577,1066,224],[1970,2086,7371],[2040,2971,927],[788,934,6258],[2705,2753,6795],[203,1780,1222],[1998,2203,4663],[1538,1825,9436],[2518,2961,3857],[2316,2351,2482],[1345,2169,4808],[1546,1796,6129],[1909,2330,0],[1963,2449,3828],[1063,1815,3582],[1620,2569,7699],[2752,2993,7372],[2415,2431,5004],[82,2168,9566],[558,2856,800],[186,2710,5597],[42,716,2270],[286,2810,6628],[1009,1084,4103],[2985,2992,2604],[807,1109,1691],[428,2164,2177],[807,2380,8808],[2219,2457,5791],[2787,2852,4834],[2749,2756,6191],[1935,2275,6272],[1747,2448,3418],[583,2194,6349],[1362,1900,6725],[2831,2926,1216],[1192,1849,5606],[855,1704,3550],[2631,2945,339],[1470,1625,6552],[1432,1952,7389],[1975,2317,3291],[957,2430,578],[210,2532,6823],[1974,1988,3262],[310,2215,1297],[2182,2304,6777],[510,1252,3744],[1912,2992,8846],[1195,1784,6934],[554,1820,5708],[277,284,4933],[2658,2814,8003],[2809,2893,6132],[1365,2852,8440],[2710,2809,388],[1545,2948,5559],[465,2415,2556],[1429,2475,8863],[1533,1591,2304],[1163,1210,8324],[2132,2820,648],[1495,2356,8484],[598,1674,7367],[100,2380,7426],[1680,1745,2750],[897,2958,2740],[715,935,9972],[1758,2223,1448],[1555,1880,216],[1407,1708,9782],[739,2622,9991],[1147,1322,1856],[2180,2272,747],[1813,2823,5778],[2530,2779,8833],[1444,2002,1208],[1348,2979,2144],[123,1861,8782],[1758,2215,7194],[328,738,9169],[2763,2922,7034],[2005,2031,6851],[1300,1931,7517],[715,961,294],[1235,2029,9422],[2975,2980,4574],[2446,2479,837],[649,2224,3637],[2043,2826,273],[1416,2862,4655],[1040,2477,7348],[1429,2483,5538],[2918,2946,5007],[154,874,7436],[1042,1107,3777],[2700,2961,2299],[71,1512,12],[620,1682,7752],[32,225,7534],[447,1930,7614],[2534,2974,390],[1159,2120,9317],[1978,2731,7087],[753,795,2708],[2087,2315,6556],[2608,2834,4345],[410,2468,3014],[2178,2820,6221],[1276,1977,7450],[484,907,3321],[138,2869,6152],[2813,2894,286],[1216,2771,9617],[2279,2436,1944],[92,2575,9133],[1435,1448,5160],[2582,2762,3828],[1971,2517,5068],[279,1257,3681],[737,1777,7314],[1841,2871,1523],[781,1164,4624],[2214,2517,2013],[2013,2364,6054],[1853,2448,4629],[2714,2736,7831],[975,2218,3557],[1348,2554,8443],[2354,2675,894],[1019,2175,9568],[2401,2622,4473],[102,387,9293],[167,2787,2207],[2008,2981,5966],[644,1022,1434],[1324,1590,1332],[2128,2742,1922],[895,1733,8668],[2669,2787,5650],[433,639,1126],[1110,1486,8715],[2162,2851,3357],[2112,2679,7739],[397,668,2054],[2141,2842,6791],[1744,2964,1822],[2824,2831,9048],[450,710,5792],[30,2806,5594],[1551,1816,1216],[1988,2881,5107],[1176,2505,4280],[2131,2777,1999],[2294,2979,4986],[425,1306,5145],[1205,2590,4248],[569,2793,1395],[1987,2019,190],[187,1412,8069],[117,2289,3477],[1766,2851,5427],[1371,2431,3753],[472,1100,7071],[1322,1736,3460],[1494,2122,1399],[802,1027,6961],[2924,2928,6479],[1268,2963,1402],[2055,2907,6803],[829,2220,294],[1558,2342,2338],[1218,1857,4457],[2842,2885,5047],[2299,2638,9626],[962,2534,8533],[2569,2677,1683],[271,641,4882],[2160,2997,1194],[36,1530,2701],[1477,2793,2101],[227,962,1196],[1713,2798,8126],[2349,2407,3311],[2570,2758,3650],[758,2600,7351],[2931,2949,3231],[1237,2237,6034],[1633,1639,6723],[354,2896,3720],[596,1830,62],[2936,2944,4066],[973,1544,7250],[2425,2646,9031],[801,2344,9216],[1321,2386,9087],[2517,2768,5947],[1872,2434,5279],[1458,2204,6016],[231,1430,4589],[121,1654,2535],[200,2994,1940],[244,2193,465],[1574,2889,2337],[2429,2798,4330],[1476,2366,3615],[495,847,8956],[2808,2920,7176],[650,1129,3319],[2604,2903,6474],[2700,2989,7999],[2778,2939,7502],[2331,2877,2957],[1957,2994,6038],[0,1113,5098],[1118,2155,9437],[886,2409,2852],[2747,2866,8857],[2709,2954,6326],[2612,2748,1911],[186,2749,958],[2425,2923,8602],[11,786,4353],[1614,1883,9584],[1712,2900,3868],[444,728,5380],[2816,2821,4287],[2463,2690,6682],[2453,2839,3179],[8,1066,2002],[1562,2774,9960],[2166,2489,692],[554,2064,2491],[572,650,1594],[1917,2304,8332],[1388,2378,1516],[2642,2878,6004],[2001,2448,9437],[176,794,3737],[2784,2856,7037],[2356,2477,3247],[662,2520,541],[475,772,6970],[2363,2521,4852],[548,1252,8180],[1486,2171,4593],[1745,2597,7484],[2116,2410,1684],[382,1351,3838],[1673,1793,9238],[2420,2464,2651],[2065,2707,2490],[1762,2266,7391],[1436,1713,1863],[2767,2884,3928],[1546,2255,7050],[202,2561,8711],[6,1343,2465],[254,2621,9390],[525,1131,4514],[956,2295,6590],[2232,2483,8141],[1963,2727,6034],[482,1142,4192],[1571,1575,4806],[393,2564,9117],[558,1732,2981],[367,538,7801],[1853,2264,4424],[2313,2885,2812],[2808,2921,5529],[2658,2750,5982],[1947,1967,9109],[772,1441,2730],[2454,2501,6752],[526,2436,2702],[16,946,2598],[1567,2237,1500],[1788,2545,1229],[2784,2893,880],[1983,2419,5913],[2039,2273,2423],[2087,2186,6544],[2122,2303,4418],[478,720,8265],[2078,2160,7493],[2163,2933,4507],[823,2934,9082],[2843,2884,5132],[2436,2721,550],[1874,2323,5088],[648,1187,748],[830,2864,9898],[2870,2941,3988],[1528,2912,4969],[2439,2548,3113],[2058,2625,351],[2578,2894,1239],[1594,2239,5023],[817,1122,5732],[2930,2951,2784],[1318,2803,5793],[312,2058,3252],[2463,2996,4132],[1470,2767,1181],[2822,2896,9493],[1572,2973,4724],[2021,2581,3341],[1685,2257,4409],[250,716,8743],[1625,1643,5955],[164,2947,269],[1479,2414,4296],[2930,2986,8872],[1676,2599,6801],[2293,2960,6668],[548,1182,6221],[1095,1934,9446],[1845,2779,9826],[710,2874,4839],[673,2257,5744],[2572,2998,70],[846,2692,7712],[2897,2978,9110],[1233,2888,5369],[2194,2710,2194],[1208,2734,2683],[1357,2404,4143],[22,1958,5473],[2118,2779,4100],[2370,2924,9828],[1818,1868,2272],[157,873,7360],[1960,2214,9284],[878,1638,6495],[334,2917,3376],[1054,2331,8421],[912,2007,1280],[2550,2658,6029],[760,2212,9683],[579,2553,935],[759,1605,5071],[1034,2586,8158],[2641,2928,4177],[698,723,4854],[1473,2037,6908],[2691,2791,7834],[1955,1961,790],[2987,2993,9551],[2313,2958,3133],[1323,2678,17],[2349,2840,3345],[2006,2985,9456],[1687,2588,2376],[140,1637,6704],[2889,2945,676],[2841,2988,6401],[2087,2455,2634],[1471,1769,1746],[1372,2395,3731],[665,1672,9526],[1657,1716,6969],[2678,2931,6277],[2090,2491,4739],[211,1169,3100],[1048,2592,3188],[232,1635,862],[1171,1429,134],[2417,2530,4089],[668,2649,8478],[879,1159,2860],[2148,2819,2511],[1808,2384,6806],[2317,2610,7611],[2144,2618,1755],[2874,2964,5843],[350,2236,7692],[2419,2859,8684],[1676,2292,4340],[1163,2130,6326],[2146,2575,8414],[2796,2964,1362],[863,2815,5126],[2707,2756,16],[903,1128,5668],[2554,2555,1860],[1365,2741,9478],[983,1259,5374],[1718,2077,7019],[471,2953,6028],[1701,1931,8990],[1934,2987,7237],[754,2785,4721],[2950,2956,9926],[547,2811,3524],[506,1591,2133],[2202,2228,4890],[2658,2856,2557],[785,2832,9350],[883,2488,2401],[437,2696,4962],[502,1817,218],[2457,2720,7100],[21,1233,9768],[2636,2998,1471],[1410,1472,1860],[2143,2203,2687],[2052,2148,313],[542,1034,1208],[474,2281,1613],[371,2302,2712],[2774,2793,2856],[1079,1331,4440],[1840,2524,6346],[1178,2733,8390],[2045,2170,1329],[323,2308,868],[879,2005,8435],[620,1594,3044],[1709,2315,7940],[1548,2089,4449],[1927,2185,1004],[2531,2791,3594],[2112,2943,5586],[2545,2736,164],[238,380,5614],[349,792,1884],[1344,2322,1610],[783,2166,5897],[2407,2919,4398],[2292,2583,3811],[2183,2954,8568],[232,2807,6396],[237,2249,5183],[1398,2073,7409],[1757,2462,2806],[1482,1709,1970],[2595,2719,2868],[984,1013,1667],[878,2762,9337],[2965,2999,6121],[567,2016,5592],[1180,2962,4342],[2577,2839,2982],[1475,1775,4203],[2335,2448,5134],[287,2675,771],[1683,2113,3786],[699,816,9173],[137,1035,2819],[632,1269,8282],[1506,2212,9031],[197,2841,8749],[1696,2851,6318],[754,1583,2485],[2795,2844,6812],[1712,2691,1768],[1803,2121,8384],[1329,2884,8721],[1889,2769,8612],[2265,2360,142],[1694,2482,657],[844,1961,9080],[2210,2475,5419],[1178,1357,3767],[2157,2964,1305],[1162,2857,6648],[1032,1692,15],[474,1642,232],[574,1363,1427],[2928,2955,1247],[1380,1647,9565],[2644,2733,7420],[2283,2521,578],[2853,2993,4],[2036,2267,8468],[1890,1964,7987],[1440,1578,6906],[1192,2117,588],[1708,1764,9517],[951,1200,5345],[2114,2368,8130],[1107,1773,1941],[2450,2709,9344],[754,2947,9884],[2270,2552,5194],[1205,1581,3057],[1796,2578,7309],[128,531,2898],[1421,1629,8752],[1692,2162,5225],[520,853,7926],[89,2748,2074],[2653,2996,8253],[1964,2353,9892],[1561,2219,9603],[1400,2659,5671],[1844,2903,4402],[924,2974,8769],[358,2273,4159],[2679,2985,7957],[1106,1773,8404],[1680,1718,4367],[2768,2975,8154],[2688,2947,7337],[737,1622,7371],[2990,2996,1490],[503,561,2441],[204,396,6045],[1930,2213,4607],[55,863,4200],[789,1481,8238],[946,1740,3616],[1218,1662,50],[685,2190,8620],[95,1011,1742],[2277,2348,1852],[1227,1936,2986],[902,1893,945],[351,799,4586],[2907,2925,7771],[2939,2993,7043],[2772,2851,940],[721,1523,8982],[2665,2941,4032],[1543,2805,8162],[368,886,8031],[2560,2761,7929],[2889,2956,5],[741,2677,154],[927,2970,8339],[446,532,2736],[1599,2207,427],[890,1999,4985],[1660,2193,9445],[2818,2831,99],[1920,2259,4605],[1213,1681,7753],[521,734,3849],[628,641,2445],[1427,2121,7559],[1399,1630,1513],[2572,2894,6371],[2957,2963,2770],[2253,2685,1796],[190,1006,9908],[1301,1711,7212],[2327,2874,6111],[1221,1715,5054],[367,1414,2595],[2927,2936,5991],[463,1099,2166],[2035,2931,9113],[1816,2881,2227],[823,2115,8811],[719,1234,0],[2217,2483,3659],[2451,2906,6375],[1946,2996,8357],[1777,2758,79],[2978,2986,8726],[2512,2567,1392],[1723,2469,3011],[1018,1648,7159],[2232,2971,3611],[317,2675,7812],[1413,1823,7811],[796,2113,4036],[507,592,1628],[639,2654,3363],[1691,2996,7008],[1610,2952,5387],[1494,1995,7972],[202,484,7046],[388,2971,8819],[1201,1889,2164],[1761,2993,4193],[837,1117,3258],[1903,2985,8055],[1659,2421,1498],[623,2973,8152],[289,1273,1750],[2052,2385,2897],[303,469,6002],[2180,2599,4478],[849,1601,2250],[1015,1204,2696],[2301,2801,6034],[1855,2760,8449],[2922,2999,5650],[1316,2001,3105],[1772,2657,4507],[2356,2387,2948],[1974,2074,6212],[1698,2505,3269],[1868,2602,7687],[2263,2612,6830],[1146,1336,6694],[1347,1785,8246],[33,2246,747],[2035,2169,400],[535,2092,3416],[2419,2678,4009],[1674,1722,3745],[1119,1567,739],[545,992,4667],[779,1468,8549],[1606,2910,609],[1753,2489,289],[1329,2193,9833],[2785,2833,9784],[2762,2978,9848],[1170,1407,1885],[2713,2717,263],[1302,2067,5144],[664,2414,3735],[2437,2596,3784],[1966,2859,3400],[241,2149,9443],[2591,2986,8855],[2418,2654,3575],[565,1579,5214],[2285,2542,2782],[2570,2782,4619],[750,1962,2928],[2685,2795,2803],[484,2506,4634],[1053,2884,6620],[1268,1351,5696],[583,2180,9733],[1948,2850,8220],[2964,2998,3940],[123,2631,930],[2907,2943,3875],[1179,1319,6041],[144,1826,2472],[134,1506,2671],[2832,2839,8237],[2218,2551,6632],[2597,2732,4795],[45,2012,7819],[211,2036,1842],[1371,2482,9507],[750,1499,4475],[2511,2554,8644],[2021,2812,458],[105,1343,2595],[477,788,9170],[1264,1844,5761],[2079,2086,6350],[1211,1841,1037],[2604,2985,881],[542,1934,2883],[2213,2392,5808],[659,2497,5862],[1003,1908,9283],[665,1099,5060],[1825,2492,9898],[756,2005,9139],[1332,1882,4011],[1360,2600,8511],[1833,2443,4462],[1305,1531,1614],[1905,2580,2361],[2061,2635,7069],[182,2588,4290],[2865,2878,5300],[909,2308,4601],[1075,2732,5768],[591,1155,8520],[131,710,5005],[717,1027,6056],[2830,2872,3285],[620,1642,6897],[2027,2175,6871],[411,2068,8417],[763,1658,1912],[610,1811,644],[2358,2841,5006],[2877,2995,3577],[1408,1763,9648],[97,2972,1020],[507,2950,7813],[2137,2288,9570],[166,2769,9617],[1595,2186,6717],[2519,2812,7350],[2188,2716,6922],[455,675,3773],[1119,1774,6877],[571,1109,8418],[1488,1994,4943],[408,1379,449],[663,2033,3771],[641,2002,4089],[518,1241,550],[140,310,4183],[1219,2320,1721],[573,1130,2912],[1913,2062,1053],[1921,2585,8125],[2780,2812,535],[1585,2978,4969],[427,1160,2374],[1618,1831,2088],[1307,2355,375],[1122,2778,6230],[1890,2492,9868],[225,711,1004],[2638,2730,7099],[1269,2529,6328],[2473,2714,5525],[2526,2592,5319],[2370,2748,3564],[2413,2661,8987],[643,2262,1072],[83,346,7149],[894,2218,7994],[346,2528,2929],[1550,1602,4502],[1550,1560,4848],[2038,2544,9345],[679,772,5510],[182,2418,9989],[1648,2046,9132],[750,1668,6240],[2547,2699,8102],[216,1067,4429],[43,2086,1949],[556,2618,1185],[177,1920,5378],[1447,1926,5040],[961,2561,4033],[2479,2517,5923],[1907,2618,5140],[1680,2999,979],[1667,2169,8445],[2187,2646,7185],[1392,1954,2427],[391,2009,6135],[398,1744,596],[9,1792,9119],[801,2554,6064],[2258,2388,4381],[2198,2616,2388],[1431,1893,5567],[1403,2852,1597],[1477,1620,9790],[1567,2650,4064],[2352,2353,6723],[1853,2901,3801],[1874,2097,8842],[2486,2884,3678],[589,2479,4293],[1618,1693,1181],[1607,2626,7752],[2248,2611,8027],[2611,2780,5954],[2355,2458,2601],[1724,1831,6020],[2580,2822,8015],[2613,2892,3514],[1735,1770,5095],[2174,2425,8622],[2504,2578,6924],[2886,2944,3510],[1343,2288,9785],[2893,2913,469],[2349,2764,8962],[1310,1968,2685],[2487,2854,4018],[2672,2987,9509],[1348,2962,3650],[1455,1904,738],[129,2505,930],[1929,2943,9048],[2282,2759,3089],[2130,2490,3526],[2869,2876,8408],[2007,2091,6703],[1768,1910,6107],[526,599,6001],[486,861,678],[1840,2754,1133],[2419,2588,7108],[1094,2305,9986],[508,2484,4331],[1358,1781,2622],[2362,2921,6056],[17,1394,7678],[924,1263,3472],[852,1938,4225],[855,863,5772],[753,2963,8737],[128,1976,5020],[2078,2579,6806],[1900,2725,1386],[2533,2751,4569],[38,1562,7128],[69,2619,8821],[1853,2963,7396],[1936,2943,6434],[910,1802,1699],[1085,1734,3929],[1215,1912,999],[1683,1792,2485],[1686,1862,9036],[2285,2790,6457],[1416,2108,6445],[2953,2976,5402],[631,1775,4535],[1052,2173,4870],[218,2855,3965],[1475,1920,8661],[2136,2425,5577],[2724,2914,5965],[631,681,7325],[220,2530,9823],[2100,2488,7346],[2374,2760,3273],[1813,1969,6879],[2827,2859,6992],[1813,2573,4186],[1039,2124,4450],[1626,1813,3606],[2281,2585,7276],[187,972,4157],[2762,2836,3234],[2156,2186,3201],[362,479,9518],[808,2224,4955],[936,2740,9814],[1145,2394,6009],[1798,2208,2662],[1411,1864,3442],[1927,2629,6275],[2980,2991,7047],[2735,2988,8056],[2220,2254,7971],[197,660,6151],[1973,2056,1382],[2848,2956,2117],[1797,2224,4919],[1977,2482,201],[279,2408,3509],[1929,2469,3296],[202,880,6474],[816,878,4587],[1474,2066,2838],[1964,2079,453],[1452,1778,1694],[1461,2990,2524],[209,1019,8072],[2242,2482,7913],[1231,1711,180],[784,2007,4419],[2318,2454,8400],[89,1705,8065],[481,1190,4697],[1378,2738,4568],[1671,2231,6841],[2473,2478,6558],[680,1655,7125],[530,633,9095],[258,1332,6946],[2343,2775,5641],[1234,2058,793],[702,2400,7611],[1817,2850,3603],[2677,2837,8398],[1863,2976,5995],[1710,2089,5604],[803,2679,9433],[2509,2719,8028],[2918,2976,3663],[1681,2953,6743],[1522,2024,9480],[414,2624,4101],[1426,1828,6954],[1024,2437,3147],[1933,2729,1191],[2479,2891,2830],[1535,2763,993],[1249,1841,2319],[2626,2965,471],[1974,2411,1290],[235,2047,1876],[739,1882,611],[580,876,3685],[593,1295,9563],[939,2971,5472],[1943,2422,7246],[998,1553,7992],[2821,2902,5772],[769,1133,8342],[653,2809,17],[2289,2930,492],[2659,2944,9085],[739,963,9988],[1379,1856,2232],[2259,2647,5659],[2439,2549,3684],[1419,1872,1730],[2323,2392,8615],[1068,1799,3279],[2668,2876,3862],[1193,1809,1862],[1903,2799,1149],[713,1967,2101],[2006,2749,5996],[792,1354,3466],[115,693,1795],[2187,2850,4174],[1445,1658,3463],[965,1565,5716],[650,2126,2672],[2175,2490,3318],[1535,1786,1072],[1855,2962,1960],[365,2354,2201],[218,1531,2410],[961,2253,1389],[594,2351,4730],[2754,2824,8850],[1132,1577,5558],[2834,2835,6529],[172,605,1748],[1637,2256,4115],[1355,1484,9398],[436,2283,9683],[67,434,4594],[2164,2993,9805],[2069,2306,7760],[1396,2914,8799],[1641,1726,3971],[2568,2842,7966],[2196,2403,6634],[2522,2790,4554],[505,742,3552],[2387,2780,9118],[557,2101,1337],[184,943,1716],[941,2465,4524],[2847,2997,1954],[156,1841,4879],[86,204,704],[948,2199,4971],[2669,2780,5315],[2322,2623,7001],[2882,2908,7874],[1368,2326,8746],[2781,2921,8315],[2643,2878,3128],[856,1614,2668],[1450,2144,8743],[2362,2674,829],[2602,2918,2561],[2130,2257,8282],[1526,2964,2720],[898,2183,2791],[2739,2976,5358],[1300,2918,2063],[530,865,9512],[1104,1661,1112],[1682,2362,8124],[1594,2499,7606],[1284,1673,9623],[2225,2691,6194],[342,2206,7672],[2830,2987,580],[97,2753,9105],[1599,1994,2298],[2298,2667,7183],[1108,1903,9240],[546,2745,2850],[1154,1530,1475],[2500,2656,8439],[1056,1992,1552],[2022,2755,4356],[364,698,6747],[2033,2295,8716],[1832,2569,4502],[2334,2684,9108],[2368,2799,2019],[2875,2911,2996],[1696,2990,264],[1987,2587,5092],[1889,2920,8159],[1959,2642,6446],[194,959,5139],[1197,1582,8434],[1371,1991,6151],[1805,2313,5138],[724,2196,6886],[2958,2982,9657],[832,2425,6651],[2566,2764,7200],[2930,2960,2337],[1896,2745,264],[210,535,8887],[1565,2252,7627],[2957,2991,7258],[828,2397,6901],[1768,2529,6010],[2267,2391,4193],[1287,2802,9288],[1512,2867,8292],[1529,2836,6433],[1131,2924,6277],[2948,2978,6409],[566,2018,6636],[879,2186,3662],[951,2177,6656],[640,2389,1397],[1060,1084,5748],[1334,2888,969],[1353,1633,3374],[2526,2566,577],[2650,2866,2215],[2198,2755,9724],[1360,1615,9971],[2245,2276,634],[1173,2863,1420],[2639,2712,8762],[360,2522,9663],[1596,1712,7871],[2173,2233,3843],[1111,2085,901],[494,1350,1209],[1117,1349,4636],[2522,2851,843],[1084,1279,2770],[1370,2563,9112],[341,2944,6883],[2646,2882,8771],[508,1117,7981],[2229,2345,6417],[2758,2892,1159],[1776,1825,8523],[205,634,4702],[2522,2883,5836],[2934,2973,5188],[624,2819,3020],[1525,2367,5022],[939,2898,7827],[469,2766,3439],[2029,2034,5286],[867,2966,9118],[460,1362,8679],[1479,2644,7653],[135,734,5514],[1639,2392,6293],[1287,1933,6875],[2262,2537,2190],[379,1438,7072],[168,766,1591],[384,1086,4010],[1928,2384,9864],[1864,2261,6485],[2347,2410,6551],[637,802,7095],[1502,2762,8838],[1787,1888,592],[799,1978,3711],[1853,2653,9907],[1183,1344,3337],[1344,2222,8119],[1517,2914,8634],[959,1723,1942],[388,1487,8793],[1038,1579,3779],[323,661,7853],[194,796,2297],[2410,2640,2438],[1066,1668,1115],[2923,2997,5179],[1103,1632,9231],[2569,2751,3467],[2834,2946,8116],[1844,2864,7095],[310,1027,9716],[392,2635,2825],[2952,2999,5220],[1000,1330,7727],[196,855,151],[269,2666,9378],[1495,2246,4976],[1760,2922,9033],[1277,1435,4851],[749,2370,2518],[1306,2708,1137],[826,1008,2394],[2703,2999,247],[1755,2173,619],[2297,2299,2526],[1354,1851,9939],[2282,2558,606],[2122,2595,6979],[2430,2493,3046],[1174,2142,3663],[2012,2408,5899],[1649,2076,4041],[1672,1826,6326],[728,2965,5190],[2643,2939,8055],[2496,2823,8926],[2631,2755,1819],[2859,2892,5879],[47,1132,220],[2228,2474,9316],[468,1555,3534],[1514,2224,955],[1143,1661,8208],[1057,1313,2254],[1415,2357,9812],[455,1998,1759],[1413,2241,840],[123,356,6633],[1840,2201,3604],[2155,2611,2505],[2438,2642,8092],[835,2102,8779],[1281,1873,5604],[1916,2449,3452],[1658,1820,7927],[1039,1584,7231],[1171,1385,539],[502,1914,5321],[1858,1981,7062],[1901,2630,7915],[222,355,2569],[1544,1616,4905],[735,2173,9198],[1211,1414,3892],[1406,2677,5382],[2435,2929,676],[2362,2959,2711],[1276,2413,8418],[2817,2963,6938],[1933,2214,5887],[2701,2717,5430],[409,1699,637],[48,2488,1380],[1817,1840,825],[2092,2713,9953],[1190,1483,5359],[1055,2484,5664],[834,1310,2442],[804,2037,8074],[2517,2923,1320],[148,593,3262],[2598,2994,7349],[491,615,801],[1205,2924,915],[37,435,5735],[2203,2873,1632],[1627,2896,8628],[2523,2654,3210],[2793,2963,9592],[143,1012,2762],[1084,2828,5056],[126,903,9539],[497,734,8180],[1645,2108,331],[1166,2094,4065],[1932,2156,5243],[1830,2593,3018],[1482,2393,945],[411,1987,1577],[1519,2165,8426],[2241,2333,5260],[881,947,2563],[2621,2669,2982],[397,556,9177],[2427,2748,208],[102,1926,3681],[2638,2996,3316],[1930,1986,7642],[2252,2655,9136],[2747,2760,1763],[99,1204,9018],[410,2663,3105],[2126,2388,8590],[91,1940,7685],[2378,2737,780],[2003,2154,8443],[20,914,8820],[977,1351,7782],[927,1342,1226],[206,1177,9525],[892,1650,8537],[2663,2675,2394],[1331,2994,4171],[1704,2294,5537],[1206,2029,6423],[2812,2894,826],[447,1537,3511],[1322,1815,9038],[1934,2140,3901],[2174,2885,3384],[2550,2876,5283],[1535,2137,5379],[2896,2944,2801],[1837,2821,1144],[983,1655,3190],[2497,2760,9147],[1561,2856,7511],[130,2932,1358],[1597,1985,1182],[169,609,155],[748,1364,759],[2631,2970,1834],[210,1454,9744],[1189,2564,7553],[23,2185,1655],[45,2605,1160],[835,1343,1494],[515,898,4824],[1718,2690,2831],[218,1196,3473],[766,1056,2422],[2536,2667,53],[1091,1537,1522],[2278,2791,9597],[2472,2659,8572],[1438,1783,5282],[2238,2396,8145],[359,1766,7779],[36,1223,7023],[2611,2640,6063],[2346,2763,5409],[2114,2523,8558],[624,2149,1974],[1576,2035,1246],[2843,2944,3300],[2566,2847,8539],[2717,2900,697],[2520,2925,4255],[222,2415,2874],[328,470,8533],[1960,2459,9552],[1977,2787,6238],[1672,1842,2586],[2643,2990,7633],[990,2881,5058],[700,1688,7245],[1024,1724,5066],[2387,2440,3196],[850,1750,3009],[1321,1916,3820],[1410,2183,6326],[591,871,5126],[1867,2584,687],[863,2354,5603],[2109,2327,3760],[1344,2153,1165],[2757,2798,6621],[1545,2660,8013],[1311,2330,4742],[1358,2456,3427],[232,1049,4303],[2659,2669,1079],[637,2473,4516],[2871,2939,6544],[624,1985,8283],[209,2072,140],[1168,1977,7252],[798,1158,1048],[586,1749,867],[1291,2964,2298],[8,695,7496],[1404,2776,6423],[1411,2566,3127],[235,1221,2702],[947,996,973],[2836,2964,3812],[1009,2322,5011],[276,2717,4628],[2720,2876,9598],[2941,2968,5802],[300,478,8513],[305,969,2228],[1251,2246,3737],[679,2548,6689],[2567,2751,4145],[584,1150,7369],[2033,2587,6785],[1462,2411,4023],[2272,2435,7179],[1009,1857,463],[1530,1878,8860],[345,1272,9824],[1739,2761,137],[921,1443,9808],[2560,2925,8363],[695,2108,3977],[2427,2909,5147],[638,857,7004],[1222,1981,4279],[1713,2768,4293],[1571,1601,6822],[838,2900,7617],[2818,2959,8653],[2364,2974,2444],[453,526,9393],[2798,2932,6878],[2038,2740,2252],[1776,1939,2552],[855,1046,8635],[2766,2825,9085],[2971,2989,6731],[118,1913,6351],[1041,1741,448],[2525,2626,1198],[1595,2279,7794],[1968,2424,3123],[1128,1278,6689],[1297,2612,1517],[2695,2716,3944],[509,1370,9678],[2970,2988,6356],[377,1621,1469],[839,1367,1923],[2166,2247,6773],[2231,2288,4706],[1164,1981,4913],[1346,1389,1265],[1910,2328,4953],[2219,2407,9966],[2566,2864,6478],[2412,2730,6451],[54,792,264],[549,1978,1261],[1860,2010,3343],[1601,2735,6428],[992,2894,3603],[1048,1318,9479],[823,1694,3924],[1142,1480,7450],[1623,2643,2337],[502,2348,5346],[2651,2818,5997],[711,2955,1595],[217,1212,6503],[349,974,8303],[917,2275,7355],[2252,2676,7266],[552,2776,6835],[2990,2992,6153],[1094,2354,5265],[1968,2563,7690],[488,2586,6092],[1785,2236,7200],[2983,2991,7261],[2938,2952,2637],[2884,2908,9298],[802,2115,255],[2142,2308,2465],[1607,2741,9187],[2793,2925,6506],[2696,2952,5767],[2664,2838,6694],[1770,2604,2475],[128,2455,8302],[2208,2388,1749],[2847,2915,3076],[2159,2658,8942],[1690,2328,9615],[2528,2939,6085],[1816,1825,4257],[712,1798,137],[362,1151,6475],[2134,2581,4644],[1067,2494,6436],[814,1580,1777],[1852,2546,2486],[1684,2320,1167],[186,1839,4602],[1420,2746,2509],[1396,1905,59],[857,1274,6883],[615,1219,8470],[2061,2655,5824],[476,2011,5472],[1775,2000,6811],[1336,1722,7387],[933,945,9808],[242,2661,3438],[117,1680,1871],[2579,2931,2582],[974,2136,2919],[1666,2590,3777],[141,2148,2374],[198,2338,5202],[558,2428,4322],[1364,1892,2989],[338,1636,4735],[2073,2855,9945],[833,992,3455],[128,1584,1126],[2329,2582,7873],[523,2526,4816],[2915,2968,7288],[2826,2898,617],[907,1140,4866],[523,2642,4078],[1590,2550,3668],[1804,2346,9140],[1137,1739,8735],[2017,2133,1252],[634,2271,7167],[1142,1932,9068],[234,2358,7211],[2701,2960,7817],[2158,2377,6741],[1923,2795,563],[2050,2252,9452],[356,2328,9772],[1064,2293,6521],[826,1229,3912],[1031,2516,7734],[2325,2923,313],[1461,2149,7301],[1041,1071,3573],[2179,2968,24],[225,1064,8380],[574,1748,6542],[564,1932,8788],[1692,2783,7405],[426,2868,9553],[1886,2958,8579],[1558,2456,174],[1088,1193,81],[476,2145,5583],[2911,2928,6809],[2741,2916,5041],[1277,1882,4131],[2963,2991,9511],[815,2668,2230],[1278,1906,4268],[1911,2044,9272],[2898,2989,4966],[2769,2829,1475],[1156,2333,6449],[1548,2856,5014],[851,1898,9265],[363,762,7141],[2630,2821,3799],[2136,2470,8157],[1216,1343,5379],[2345,2804,8538],[2387,2792,6494],[204,413,7294],[718,1039,8883],[747,1122,306],[2828,2976,9001],[2593,2701,7861],[1274,2713,4058],[2839,2987,6482],[2378,2811,3978],[2786,2966,1202],[2139,2488,6213],[1044,1627,8750],[2142,2926,7411],[383,1831,3141],[2740,2742,7685],[959,2684,9911],[341,2778,4243],[2549,2932,5362],[2847,2982,4044],[2595,2760,422],[2474,2488,5759],[2998,2999,807],[992,2737,8886],[1390,1562,1114],[459,744,2768],[1641,2580,7692],[2704,2761,7189],[439,1790,4600],[2614,2674,3260],[2506,2824,1935],[309,1642,4796],[851,903,3492],[2265,2908,6959],[1082,1577,9154],[2409,2444,9261],[2180,2690,1030],[1265,2488,7536],[358,884,5702],[289,2190,85],[1792,2024,9397],[699,1518,2575],[1555,2239,1730],[1308,2897,7732],[1777,1963,3381],[2944,2987,7421],[1649,2707,7318],[527,1445,5687],[1100,1888,4451],[283,2300,3103],[496,1419,4327],[969,1646,6160],[167,2400,4160],[2184,2798,4065],[471,2528,4303],[642,2572,7766],[1611,1883,7286],[2029,2302,5396],[2575,2887,8750],[122,2282,7217],[796,1352,7426],[393,1300,2736],[2861,2994,7299],[502,669,3819],[1220,2115,9634],[2499,2536,104],[2932,2949,7026],[467,2497,4268],[2043,2197,359],[2018,2395,5183],[608,704,2194],[1410,2075,2362],[677,2769,1111],[1903,2719,1794],[2045,2105,991],[1630,2325,7600],[2842,2975,5776],[938,1722,2861],[1714,2473,7737],[987,1132,6033],[2202,2602,2893],[2397,2922,4320],[1011,1702,4438],[1923,2361,5668],[1923,2851,3864],[1365,1580,764],[1553,2832,658],[1887,2737,2447],[642,755,4269],[1111,1592,3381],[1728,2104,52],[1384,1948,3672],[1768,2019,5717],[878,1812,267],[916,2745,5729],[1502,2745,742],[2230,2430,5373],[714,1313,6307],[1800,2619,1778],[2653,2902,3045],[1992,2049,825],[2761,2840,606],[152,840,7012],[2690,2832,2519],[1316,1564,9576],[416,1377,7611],[2272,2942,7905],[2465,2805,4836],[1642,2379,3529],[2291,2925,8718],[32,528,5414],[1128,2191,7330],[1286,1781,9686],[1113,2709,96],[2891,2952,7309],[1368,1908,5987],[131,2120,9070],[1970,2564,5233],[1596,1653,1541],[1392,2741,4559],[1141,1393,9608],[972,2191,7194],[2315,2705,6687],[1267,2855,8324],[2349,2701,5632],[2132,2262,2714],[2032,2443,6191],[748,1669,4686],[474,762,1766],[580,747,646],[1590,1801,136],[415,1345,9385],[1739,2954,1837],[1136,1202,4975],[2155,2232,1912],[2279,2642,2095],[1642,2738,9631],[136,2884,6099],[2228,2347,2495],[2105,2427,5313],[1042,2925,8749],[251,1640,9347],[2647,2885,3584],[2722,2861,963],[1857,2843,4797],[1874,2546,6457],[1887,2486,1207],[2341,2990,59],[1522,2859,2719],[298,1491,3710],[703,2817,6971],[2977,2991,2513],[681,2161,4331],[1503,2621,6934],[1933,2240,3290],[139,1342,7425],[1211,2586,2088],[2539,2818,5638],[2553,2769,6462],[1761,1766,1119],[2192,2692,929],[1242,1858,2795],[82,2764,3675],[843,1685,1216],[249,1749,2103],[943,1846,8280],[1328,1913,341],[2820,2969,7142],[2142,2912,9339],[313,2633,5970],[1999,2312,1812],[1802,2382,8580],[2192,2901,5647],[2316,2590,6699],[1977,2803,5343],[2675,2682,9598],[2911,2952,3738],[1512,2581,530],[651,1448,3665],[2508,2722,8798],[2282,2551,8172],[2431,2957,2786],[1761,2320,1305],[107,2931,8210],[1995,2333,1619],[2509,2664,1551],[2127,2347,2349],[2131,2790,2916],[2241,2967,301],[421,1401,7298],[318,1977,5792],[1085,2041,7190],[510,2375,4537],[2385,2607,2657],[2959,2978,7497],[483,600,4233],[2465,2910,5155],[432,2464,5122],[2625,2719,6149],[2648,2701,1054],[1209,1232,6728],[56,1985,6655],[2075,2486,1691],[1750,2297,6356],[2619,2842,8882],[2143,2290,7770],[2634,2924,683],[209,709,8167],[1130,2669,7208],[2085,2187,7091],[687,1811,4858],[199,1364,9399],[1121,2209,3172],[876,1653,8939],[126,1068,8174],[629,2717,6946],[2123,2751,2208],[1099,1587,3632],[2379,2601,906],[680,1011,8355],[1639,2231,9493],[1753,2279,3338],[1777,2482,3211],[2984,2995,9995],[2358,2679,1900],[1715,1981,2645],[789,2476,6066],[1739,2900,5194],[2401,2856,3633],[2509,2992,7046],[1185,2652,6391],[2787,2918,4523],[1949,2164,8359],[1118,1744,7263],[1476,2343,5537],[1493,2758,9117],[1266,1638,4511],[1255,1774,2555],[323,2199,3187],[1132,2066,890],[276,1801,5507],[2753,2942,3643],[2666,2996,9605],[2315,2635,3027],[2928,2949,3933],[2029,2912,561],[1106,1770,7399],[1042,2963,5576],[1859,2509,8005],[2754,2815,8974],[614,2339,5131],[625,1930,188],[457,2924,9016],[610,2715,9753],[2676,2819,3314],[2274,2721,1003],[1720,2046,573],[1854,2410,9640],[2395,2687,3355],[2227,2338,2881],[725,944,1971],[5,1110,4027],[1101,2852,9185],[1023,2560,4838],[1672,1993,8482],[1081,2611,2630],[992,2645,2787],[407,2857,452],[1673,1820,6463],[1116,2223,9550],[1109,2017,6199],[1151,2965,2488],[1252,2565,3390],[1100,2628,6],[1743,1884,2575],[323,791,9261],[598,1831,7283],[242,1350,5171],[1073,1565,7748],[1041,2161,8942],[2235,2650,2273],[58,1444,3599],[106,832,8585],[1376,2840,6527],[268,425,2625],[2474,2878,2769],[1395,2774,1262],[124,2714,543],[262,862,5688],[795,1958,6310],[1522,1829,8536],[1212,2786,632],[337,801,6094],[2360,2868,1409],[372,1744,4799],[2260,2797,9824],[1162,2867,9315],[2947,2954,7508],[1234,2379,4319],[1412,1651,5685],[1217,2265,4114],[2511,2518,2975],[2216,2768,6924],[2012,2790,7232],[1756,2246,2682],[367,1276,4570],[24,2084,194],[158,1845,9504],[1542,1944,9751],[917,2626,3683],[1431,1761,7896],[2000,2222,5524],[2065,2724,1234],[2710,2795,8698],[2426,2501,4007],[2752,2849,79],[2610,2794,7585],[408,1766,8908],[2821,2901,6208],[2423,2624,3702],[1265,1352,9242],[53,848,574],[1338,2079,2227],[194,1142,1694],[677,812,6549],[2934,2988,8648],[1820,2317,6723],[2249,2429,8061],[1482,2001,8544],[1358,2562,8175],[1778,1922,9618],[1970,2266,8832],[1090,2577,5717],[1793,1873,3840],[2637,2961,7905],[2636,2755,4911],[2917,2994,1719],[2220,2871,1149],[2779,2847,6557],[738,2822,2385],[2461,2599,867],[2579,2615,1241],[1986,2550,4002],[2351,2369,5471],[442,2476,2030],[1817,2880,7005],[2488,2834,8874],[763,1740,3787],[2987,2995,4802],[1676,2021,441],[1757,2978,3245],[511,2227,8555],[2741,2833,2702],[2723,2867,4268],[2969,2970,3964],[2513,2860,3199],[2113,2970,8821],[2368,2704,1253],[1186,2566,8279],[375,1455,7329],[1714,1907,4426],[929,2124,2292],[157,2338,4314],[1778,2684,2245],[0,1850,414],[1177,1375,5633],[1259,2841,8575],[1105,2738,5726],[153,1512,7070],[2020,2370,1193],[1940,2577,7042],[1044,2662,2666],[2283,2311,9519],[2415,2743,9323],[175,1087,3204],[205,469,7607],[2652,2653,1270],[2048,2153,9791],[1266,1951,3322],[2922,2935,6286],[2703,2874,1033],[2264,2290,5742],[1066,1838,837],[2953,2962,5164],[1201,1405,5575],[2199,2611,1296],[2799,2856,4757],[1691,2634,5819],[2382,2597,6656],[1465,2506,2363],[2502,2918,8210],[1159,2569,3663],[130,2117,2595],[200,488,9200],[301,2052,9404],[2061,2515,7312],[205,2134,8562],[194,1891,4723],[1639,2186,1342],[101,1052,4591],[1524,2597,4679],[1901,1957,6296],[471,1191,5455],[1,2936,4286],[2554,2998,7687],[2865,2956,8349],[2449,2548,8304],[2009,2335,2630],[2092,2980,1080],[2790,2817,8095],[2387,2735,5791],[2985,2987,5798],[2261,2841,2776],[210,2247,9421],[336,1844,9958],[1221,1540,5396],[1262,2569,5837],[170,247,1140],[2523,2672,8131],[1709,1834,4391],[1453,2288,2396],[979,1634,7508],[192,980,108],[2351,2778,8157],[2364,2928,4271],[499,2528,5752],[2150,2544,2604],[1146,2012,2222],[2493,2994,535],[141,539,3424],[1149,1216,4348],[1776,2217,3922],[699,1142,6229],[799,2736,5249],[710,2779,8047],[2667,2977,107],[2166,2625,9446],[2788,2916,6365],[2182,2997,8955],[1867,2418,8990],[2965,2971,652],[978,1426,3269],[2772,2950,5855],[2733,2956,2244],[846,2965,9561],[2496,2840,3000],[343,2802,5650],[1348,2779,3132],[1145,1494,264],[1606,2483,539],[1976,2418,4588],[2919,2977,5247],[1741,1977,1557],[1940,2537,1450],[992,1832,5153],[2966,2983,1626],[45,833,4498],[2019,2551,220],[1199,1814,5291],[1269,2251,6161],[797,2480,9930],[1553,1731,507],[1725,2840,5737],[1492,2390,336],[2605,2710,6665],[1216,2627,5347],[1539,2166,2186],[1274,1426,1049],[2322,2461,623],[465,870,3539],[2082,2768,3829],[1174,2467,4491],[581,2426,9825],[314,986,4169],[2410,2975,4165],[443,938,7345],[2728,2735,269],[1102,2080,796],[383,708,5874],[1395,2900,4466],[2386,2820,7720],[1846,2095,2276],[2446,2507,7615],[1341,1836,1210],[859,2764,6333],[2194,2943,798],[545,1331,2164],[1381,2578,441],[2926,2971,1459],[2409,2449,2621],[396,753,2758],[2425,2979,899],[2966,2973,724],[2198,2717,1255],[2952,2995,8546],[326,1006,428],[2969,2971,7206],[1343,2633,8931],[1745,1948,8634],[1206,1565,6899],[2225,2760,7754],[70,1533,1579],[1895,2688,6589],[66,2754,497],[2104,2938,9846],[2412,2423,5669],[1710,2344,1368],[2656,2798,3483],[1278,1479,2126],[1891,2481,546],[2129,2318,2569],[347,400,2367],[2604,2647,7737],[927,1986,5586],[968,2410,7733],[772,2048,9966],[1250,2027,2994],[1115,1646,5184],[857,2343,4671],[1066,1548,698],[2058,2536,7692],[2959,2972,2898],[2770,2859,1160],[1131,2136,453],[1548,1579,6313],[2341,2958,8528],[1841,2822,2639],[685,1993,9548],[2988,2991,9971],[2633,2778,860],[2581,2864,4450],[649,2000,5701],[1282,2211,4702],[534,2400,2922],[2401,2539,5354],[622,1132,8054],[517,730,1769],[131,982,2580],[235,648,3713],[125,1002,9363],[362,566,8735],[387,752,7823],[2974,2979,9724],[993,2867,6102],[1792,2807,7942],[2755,2756,5797],[1231,2855,2761],[2145,2482,844],[2549,2960,6977],[1877,2483,4175],[250,1046,2627],[2663,2928,8898],[2669,2833,57],[2467,2656,6382],[978,2581,3936],[1965,2044,2840],[153,2374,250],[2634,2788,1286],[2970,2978,9426],[1425,2470,7197],[1863,2232,8748],[1231,1764,5863],[100,406,9336],[1034,1458,2088],[2105,2477,3760],[1121,1217,630],[2575,2897,4338],[149,1480,9227],[1505,1825,1748],[323,1723,5030],[2078,2333,4814],[2560,2797,8275],[1646,2903,1568],[2469,2495,8592],[1973,2996,3012],[1840,1933,9746],[1890,2214,6996],[1621,2025,2666],[205,374,487],[897,1699,8169],[399,1364,7337],[320,813,1666],[994,1563,8310],[658,665,4372],[280,2893,777],[614,2244,3196],[705,1372,4350],[1947,2049,2060],[1621,1898,4427],[463,1601,9922],[2306,2422,5784],[432,1583,6419],[2008,2610,2961],[1450,2290,8140],[1374,1646,8713],[2283,2685,7190],[1142,1521,766],[556,633,3549],[2504,2675,4805],[749,2133,6923],[625,1418,6703],[237,1383,405],[1175,2989,1073],[851,952,294],[28,1785,183],[63,749,1984],[747,782,8725],[2021,2375,8979],[1793,2022,2557],[2407,2560,6568],[1995,2343,7840],[849,971,4704],[956,1811,6127],[1881,2128,4891],[1858,2705,9163],[243,1385,9935],[2761,2984,9268],[904,2278,7245],[2798,2988,8298],[1039,1877,8410],[2627,2694,3114],[1660,2758,1292],[2861,2915,576],[2884,2957,4338],[705,1596,8626],[1718,2414,3050],[2014,2223,9777],[1654,2005,5050],[961,1936,1958],[2412,2544,7647],[2896,2979,1685],[618,1219,3452],[2885,2888,1096],[697,2331,7946],[1002,2200,3482],[180,2458,1632],[671,1280,7877],[2921,2936,8684],[1836,2971,6481],[648,2241,9697],[615,1541,9128],[2762,2831,8039],[2388,2988,2952],[651,820,1238],[121,1571,2085],[1565,2418,4478],[2474,2515,3793],[1537,2988,5986],[2969,2993,2916],[287,640,2183],[1176,2181,8445],[1545,1768,5347],[322,1536,3738],[109,2464,3143],[109,2971,9169],[443,624,2769],[741,2079,1065],[1921,2870,941],[2559,2930,8449],[2715,2983,3868],[81,817,6211],[1670,2177,4],[2361,2754,5064],[654,1092,5952],[367,2718,5062],[194,2035,2525],[935,2098,167],[2270,2466,8102],[906,1985,8953],[2964,2999,6433],[2359,2833,3616],[1618,2980,8810],[2747,2862,8643],[2078,2479,8742],[1659,1972,9183],[2864,2921,188],[2396,2685,905],[2425,2583,945],[1305,1333,9309],[2246,2490,2031],[2595,2845,3216],[2212,2474,3748],[2890,2898,8743],[1554,2653,2718],[2444,2692,6015],[112,1191,384],[2245,2831,7233],[611,2597,5937],[1918,2894,3806],[139,1611,2149],[93,519,9413],[2273,2851,6721],[1161,2277,5475],[1195,1871,4143],[172,2693,4078],[1486,2494,5811],[787,1057,2301],[220,2146,6257],[2469,2752,1296],[417,1114,1690],[520,2475,2696],[1091,1749,5854],[1922,2086,3106],[194,716,2295],[2215,2224,9914],[204,610,7533],[1674,2563,3443],[2694,2912,2120],[450,1629,1628],[201,1419,3245],[1801,2479,7379],[419,2163,4194],[625,1511,1101],[62,2755,3316],[2407,2824,1468],[2715,2950,8005],[287,452,4083],[2570,2804,9532],[2453,2675,9243],[2444,2645,6988],[2083,2624,57],[855,1646,3539],[542,1918,8535],[2559,2926,5619],[2982,2999,7916],[868,1602,9572],[902,2236,7132],[571,584,152],[4,2336,9099],[1035,2547,9782],[1469,2143,1526],[162,2818,6473],[2156,2823,4209],[2108,2873,4331],[1873,2272,623],[1392,1481,1049],[624,2284,6960],[306,353,8792],[645,665,2575],[1417,2060,710],[1814,2642,6709],[1630,2852,8409],[1311,1797,6861],[40,722,9268],[934,1924,8350],[625,1724,1170],[1864,1944,5747],[1978,2147,441],[889,955,7652],[2674,2812,3500],[2545,2948,1597],[116,2033,9223],[168,2826,8830],[828,863,2713],[1719,2353,7311],[148,2529,2939],[852,2057,891],[437,2650,4274],[931,1824,2435],[2285,2995,5323],[2898,2931,363],[2669,2841,6658],[49,2785,2093],[341,531,882],[1169,1278,4099],[1116,1143,2374],[1967,2600,8699],[2144,2921,4801],[582,2842,778],[196,887,6569],[46,117,7977],[932,2056,9879],[1831,2728,4450],[1908,2939,7664],[879,2470,1626],[2946,2996,9926],[589,1752,4739],[851,1792,7105],[400,1394,4758],[1964,2750,9705],[1302,2658,7238],[688,1904,5612],[2234,2717,8652],[2146,2937,781],[2121,2473,7683],[260,1548,9158],[1875,2065,7502],[464,995,9113],[2537,2623,6750],[2832,2838,3981],[66,1051,3519],[2883,2885,8653],[2471,2669,2590],[1759,2650,98],[1721,2322,699],[2700,2788,7679],[530,1060,6321],[729,2242,366],[1166,1495,244],[2547,2966,3344],[1399,2585,93],[1685,2711,1850],[812,1162,8241],[723,916,1715],[2714,2861,1809],[1702,2119,9447],[497,1317,5961],[2108,2129,9556],[339,1710,8164],[1197,1403,3026],[228,2560,4728],[2860,2947,517],[774,2529,5040],[2597,2945,9252],[197,1703,423],[2228,2637,8313],[2520,2972,3965],[87,1938,624],[1716,2828,8011],[1821,2342,5],[2669,2793,3304],[762,2154,8098],[2096,2153,438],[996,1231,4159],[2428,2822,4909],[476,1573,9073],[715,2602,3415],[1695,2457,4875],[54,2600,9900],[121,1683,920],[2472,2965,8085],[2023,2468,1264],[2902,2961,8724],[2016,2772,7927],[2440,2590,3006],[1652,1912,3828],[1533,1537,1539],[466,2109,1401],[2937,2991,8642],[2195,2480,2508],[1848,2599,7392],[54,2309,336],[1839,2203,3214],[1015,1911,5887],[1621,1854,6433],[2579,2729,5823],[2699,2771,5892],[1926,2662,3667],[1660,2174,6163],[1071,2581,7763],[1987,2317,3101],[1662,2967,2014],[2671,2972,8261],[2751,2834,9281],[1351,2520,3615],[1846,2220,5307],[1551,2746,260],[1632,2549,4884],[1168,2164,2741],[1606,2310,9678],[2698,2947,4514],[1751,2424,5320],[276,1933,3020],[529,2243,533],[54,2778,3083],[2755,2956,7400],[1144,1668,9630],[1856,2206,855],[1178,1372,9367],[2788,2991,1919],[1640,2017,515],[1787,2953,5694],[1737,2367,2136],[1443,2993,7644],[2110,2478,9430],[145,1436,4155],[673,1152,6833],[375,879,1898],[1612,2312,2031],[1871,2033,6268],[1491,1816,9291],[1273,1282,1033],[663,2682,4620],[72,527,3174],[1077,2180,7002],[455,2487,5317],[1804,2682,2422],[400,2601,771],[2738,2846,1744],[512,2795,331],[473,2891,2859],[1648,2747,1679],[1023,2790,1350],[2726,2929,3316],[483,2109,7323],[2495,2942,9908],[45,1799,5934],[198,2228,9809],[1215,2769,4771],[803,818,345],[2594,2679,4177],[2749,2847,1233],[2666,2856,6355],[545,2912,7137],[1892,2666,5395],[232,2858,4403],[2330,2932,3098],[2749,2913,8067],[2440,2736,6746],[647,1945,5943],[1689,2570,8369],[1230,2661,3819],[1865,2221,8443],[2368,2600,4156],[924,1449,3276],[1303,2885,7115],[2542,2696,8117],[2560,2915,376],[609,1424,1145],[343,1498,9970],[2800,2818,2690],[1722,2968,5330],[485,2163,7274],[191,1878,4881],[2889,2993,8898],[412,2743,1504],[727,2737,5788],[116,300,4149],[2219,2542,4724],[2747,2992,5353],[419,2218,4351],[1854,2404,2875],[2458,2853,6411],[2087,2506,5174],[1032,1289,3418],[631,2370,1448],[1170,1619,7568],[1194,1755,3469],[2094,2603,1491],[975,2374,3739],[728,1596,8271],[2196,2203,5916],[2445,2970,5451],[2741,2966,6969],[12,1725,2364],[1563,2717,538],[329,706,5063],[737,932,3613],[2600,2641,866],[1587,1845,6607],[2223,2484,7628],[710,1546,8407],[808,992,3351],[1004,1642,8633],[1321,2738,4441],[153,924,1070],[1880,1920,26],[2714,2811,8274],[2087,2796,3283],[1749,1806,2708],[2007,2546,4586],[1307,1846,9182],[761,1005,1394],[1936,2102,438],[1807,1915,9933],[543,2369,9156],[2160,2999,1030],[2570,2955,7500],[506,1740,2079],[588,854,1634],[1151,1582,7403],[1987,2360,5343],[686,1302,7745],[1692,1938,8644],[1440,1764,9774],[2484,2680,3401],[2699,2812,9176],[232,1675,6102],[2171,2361,2259],[209,948,7114],[2801,2857,6545],[2635,2968,6159],[2616,2987,7087],[680,1608,7585],[2286,2951,6280],[2916,2942,4278],[2417,2874,6978],[1761,1938,9963],[1334,2442,7207],[2115,2514,8063],[1582,1876,377],[1480,2802,5811],[116,2255,7384],[146,249,9341],[898,923,1521],[1713,2585,56],[104,819,1252],[2664,2925,5899],[990,2192,1744],[345,569,227],[395,2861,5827],[373,790,2281],[1182,1831,7654],[2136,2620,8340],[1334,2641,6199],[971,1636,6768],[2523,2751,9199],[674,1351,3876],[708,1250,4050],[1564,1715,5307],[209,1729,4740],[1546,2947,2374],[2034,2246,6675],[2148,2805,9696],[2383,2778,1641],[1642,2605,485],[2496,2808,8221],[2889,2965,1508],[1227,1830,7785],[587,746,3331],[666,1815,862],[2928,2978,5688],[1917,2882,196],[358,1045,7183],[434,2752,3655],[2175,2635,2357],[432,2525,1431],[419,702,2860],[453,1675,6821],[1070,1446,4773],[2756,2996,6101],[420,1808,3325],[1472,2324,1056],[2804,2952,7112],[503,964,5160],[257,2324,9979],[311,2226,4821],[2376,2661,1821],[947,1624,8781],[415,2116,2809],[2289,2856,7541],[2032,2065,9746],[424,865,8087],[113,1117,1118],[2983,2996,4325],[1687,1972,9014],[1683,2198,4444],[709,2735,5359],[50,2780,7829],[2040,2825,7348],[535,1403,6177],[468,2681,2326],[1767,2110,3648],[1628,1790,4292],[1918,2039,2289],[2276,2613,4680],[135,798,6023],[1101,2851,1825],[2071,2774,1270],[1753,1911,8611],[1427,1667,9009],[2567,2927,8707],[92,873,7740],[1145,2312,5951],[91,195,7500],[1084,1611,7790],[1363,2778,9227],[2576,2788,5697],[484,1943,5280],[2481,2516,9271],[1986,2269,9234],[1043,2136,8946],[1769,2097,3145],[1899,1970,5532],[234,1414,1283],[833,877,9631],[1026,2729,8973],[1988,2009,6288],[322,430,7473],[1768,2203,2392],[1423,1533,4468],[2006,2237,6370],[64,993,8439],[1901,2266,8980],[2692,2729,387],[2702,2843,3292],[1037,1942,6047],[546,2267,5075],[2021,2788,829],[496,2722,8166],[1545,2784,5628],[1745,2113,7315],[824,2929,5612],[1871,2424,7544],[2991,2993,992],[1942,2328,3325],[2040,2220,6160],[2800,2989,3453],[1092,1109,397],[719,2299,6472],[1754,1769,4422],[459,1030,8970],[2217,2516,813],[2859,2897,8320],[562,2554,1988],[667,775,7272],[419,1543,3348],[2620,2803,8493],[2107,2135,3888],[895,1437,4699],[2247,2369,7798],[2767,2877,2583],[48,1912,423],[2542,2662,937],[1510,2393,468],[2570,2651,1086],[1145,2810,3900],[2531,2561,4669],[1602,2787,8500],[2801,2906,3297],[414,1533,5262],[872,2109,6108],[506,1971,4162],[1240,1416,4907],[809,928,3531],[2023,2856,9316],[2655,2907,7769],[291,667,8326],[1936,2837,3269],[1345,2775,7906],[342,788,3305],[1402,2946,4379],[2721,2740,4790],[356,2726,7129],[2416,2614,2091],[1740,2146,654],[210,526,6824],[1725,2922,5539],[1249,1872,4466],[2008,2186,5926],[1980,2206,4093],[1156,2762,4697],[1745,2512,4034],[142,539,4598],[889,2262,4120],[1086,2374,708],[2004,2656,5321],[2541,2599,33],[408,539,5594],[100,1535,9410],[2321,2682,657],[1308,1927,4570],[544,1048,3434],[451,1392,9565],[81,2011,9185],[2410,2969,9129],[117,518,1611],[236,2248,3633],[2042,2406,4155],[859,1961,3995],[2250,2524,1346],[2890,2903,3538],[949,1905,5827],[948,2929,3266],[172,828,9588],[1840,2561,1404],[1556,2458,8819],[715,2046,6241],[1121,2074,3009],[1252,2386,4206],[1696,2504,4824],[1983,2438,5665],[691,738,2116],[384,902,8970],[2055,2303,4084],[1023,2147,5034],[1907,2613,3840],[1204,2896,9788],[1366,2757,3870],[2064,2902,3307],[2937,2946,3111],[1819,2584,7711],[2052,2405,8671],[2898,2974,6864],[2935,2988,4341],[2496,2727,3513],[978,1218,1824],[1543,2450,6927],[582,768,6178],[2346,2432,6252],[818,2069,8198],[2218,2928,6141],[2540,2702,4466],[437,2830,5188],[1155,1852,2958],[518,1282,2091],[1111,2250,9508],[2634,2794,9167],[843,2556,9493],[1066,1085,6837],[489,1595,332],[276,2570,5995],[2638,2910,4620],[468,2296,7270],[54,1130,5272],[1325,1808,7947],[2002,2536,3843],[552,1722,2377],[1628,2303,9471],[2441,2461,5319],[2591,2748,8133],[2574,2885,3904],[880,1707,3592],[1634,1938,1324],[2725,2888,2874],[1853,2547,679],[1721,1775,3102],[2105,2388,1696],[893,1633,8207],[1776,2011,954],[2750,2839,8318],[949,972,5265],[813,1084,5112],[2552,2729,2669],[924,2135,4803],[2248,2681,8916],[2874,2879,2487],[1868,2881,2211],[2264,2769,8157],[1050,2746,1168],[53,207,62],[619,1564,4281],[701,2193,608],[2619,2939,4926],[2134,2647,865],[108,1102,9489],[236,739,5305],[282,745,658],[521,1639,3744],[1489,1532,374],[362,983,3599],[126,188,5922],[810,1299,5723],[257,1516,7714],[1785,2129,4827],[2018,2489,9334],[1208,1616,4653],[2215,2450,6608],[2695,2883,6247],[155,1271,9835],[1958,2283,8676],[986,2465,6567],[1549,2433,7498],[1049,2124,832],[1648,2073,4138],[223,2574,5744],[1066,1878,3818],[933,2167,6308],[2269,2704,4899],[2286,2524,8305],[340,1449,3369],[124,846,1771],[1009,1860,949],[623,1086,4386],[220,1888,6040],[185,361,3972],[1659,1841,255],[819,1939,1587],[732,2302,6479],[1616,2004,335],[1221,2481,8265],[2132,2618,9136],[552,1971,1401],[1677,2560,535],[2197,2687,4429],[17,1617,6254],[1255,2090,5676],[1443,2867,4838],[2633,2743,6616],[2758,2768,3071],[2913,2977,7669],[802,1507,7866],[1110,1427,1053],[1293,2013,9067],[1418,2004,2667],[617,2802,5360],[2615,2809,8675],[872,1407,5565],[1434,1461,7468],[1028,2159,9030],[1446,2118,2270],[1159,2123,8688],[2612,2626,6636],[1940,2014,6161],[2789,2931,3735],[1229,2016,738],[780,1496,6083],[1539,2390,8253],[1468,1818,1275],[315,368,1432],[1135,1876,9405],[2744,2995,7944],[2295,2774,5830],[239,2403,9369],[1825,2245,2810],[331,394,7375],[1696,1966,1117],[2860,2940,5827],[2154,2974,189],[151,1856,7889],[1788,2426,3245],[1576,2927,8990],[1178,2093,4402],[1180,1380,3390],[520,1687,4723],[1158,1300,8411],[2199,2729,7186],[819,2311,2633],[1969,2976,2181],[2607,2954,1703],[1535,1947,9420],[387,844,5351],[1464,2712,3575],[2295,2339,3937],[32,1785,3285],[1654,2619,5938],[1505,1721,2097],[2241,2423,6325],[2516,2578,5070],[188,1245,8532],[1447,1821,9818],[782,2227,5362],[673,2039,5579],[1514,1911,8706],[2771,2835,6766],[2984,2998,6270],[1675,1730,6838],[1804,2929,4967],[2526,2644,4673],[2385,2558,8528],[2813,2827,2239],[2252,2821,8020],[1307,1567,894],[1835,1923,4050],[532,2372,1116],[1362,2300,8381],[1679,2065,9848],[1955,2714,1148],[2036,2689,6686],[729,1637,2622],[2877,2943,4906],[1251,2254,196],[1939,2279,3478],[2588,2678,4408],[1604,2080,8073],[532,712,5514],[2617,2674,1191],[2212,2489,6933],[2846,2873,4603],[2584,2747,5246],[812,1120,4514],[620,679,2341],[2922,2967,3113],[1813,2664,704],[2334,2579,5587],[1817,1942,9716],[256,1945,604],[144,2244,3358],[2733,2905,2683],[2358,2994,9130],[1203,2463,2418],[670,2314,2417],[826,932,6717],[1721,2654,1560],[15,2815,6577],[920,2685,1115],[2421,2909,4905],[2899,2944,8061],[2002,2871,6680],[163,1580,913],[499,2992,1891],[777,828,5319],[1145,2934,6061],[2505,2855,4298],[1445,2557,7479],[1582,1757,3683],[85,664,979],[2874,2937,8652],[1928,2819,5677],[962,2425,5608],[2098,2249,7609],[1054,2873,4905],[562,1354,5874],[2368,2843,7141],[1320,2746,3941],[463,2418,3479],[1265,2368,8885],[168,1675,3496],[439,534,4549],[328,2097,5587],[638,2619,433],[2928,2999,5737],[1197,2934,9754],[638,2624,4520],[1485,1744,7762],[293,2483,8138],[2709,2829,550],[825,2236,8397],[1011,1262,5442],[797,1933,3252],[210,1428,1728],[1875,2647,3400],[1150,1797,90],[296,1956,4749],[1709,2019,1342],[1697,2889,2859],[724,2035,4595],[2267,2728,8940],[1556,2004,4145],[1885,2424,3115],[385,2841,4031],[2217,2802,5290],[1916,1971,6372],[1886,2136,6040],[2174,2865,264],[2163,2246,4944],[2340,2478,8598],[82,227,1757],[568,600,8427],[2626,2793,3794],[508,1209,4433],[1655,1956,7684],[2937,2969,1008],[1316,2174,9642],[705,1995,379],[1286,1493,9743],[1719,2544,9011],[219,2658,8388],[1560,2064,3186],[2175,2686,5333],[843,1254,9731],[1095,2931,5637],[1107,1382,3327],[601,2182,5731],[1926,2741,8841],[572,2182,3004],[2988,2998,9209],[2809,2999,4303],[440,768,9728],[2747,2813,3113],[1709,2557,7784],[1401,1767,3201],[591,1236,2829],[1624,2647,2167],[2942,2967,7074],[707,2290,4946],[2830,2993,2955],[2131,2942,3579],[2387,2511,947],[1331,2715,6573],[1204,2576,5359],[2416,2689,5128],[921,1799,1654],[1526,2187,1726],[2865,2867,7128],[2981,2994,6092],[2421,2947,9029],[2256,2329,9610],[2848,2855,7040],[1438,1727,9342],[1758,2381,8959],[2233,2552,8026],[1392,1439,6272],[2910,2978,5425],[590,734,9485],[2735,2889,4753],[334,781,9248],[1945,2384,3359],[1196,2567,7500],[1744,2715,114],[160,481,3301],[2616,2881,6936],[449,2647,688],[2320,2776,1306],[474,2687,9657],[95,1310,8581],[1781,2796,1133],[2226,2811,4959],[2072,2671,7483],[1849,2302,284],[2534,2587,9349],[2287,2322,7807],[2649,2741,2207],[1932,2928,1349],[2982,2989,4559],[2758,2911,3800],[2986,2997,8137],[810,1529,3744],[163,2095,489],[1369,2503,635],[1556,2761,8344],[1378,2507,5906],[2517,2526,6806],[1489,2774,8994],[1143,2280,8413],[2476,2968,9306],[2822,2878,410],[2058,2445,9034],[2202,2223,6239],[2961,2976,4021],[446,625,8976],[1289,2925,637],[2320,2598,3960],[2389,2536,4872],[574,1693,4083],[2953,2975,4392],[2112,2452,5468],[2516,2751,4057],[2574,2998,2490],[1920,2518,6716],[2586,2928,8779],[2422,2965,7404],[846,1801,7861],[519,2111,4333],[2443,2803,1684],[814,1726,2261],[662,2247,9748],[2469,2708,7366],[1655,2740,9399],[994,1626,2156],[2056,2518,7905],[911,2763,2390],[307,416,9027],[1533,2170,5429],[1892,2191,3916],[884,2347,2339],[58,318,5828],[1609,1963,3826],[2487,2781,5849],[1650,1940,4677],[538,2896,2351],[2787,2810,2891],[713,908,5172],[1787,2444,8247],[2418,2448,8652],[574,2431,8894],[995,1854,9270],[2768,2924,5190],[2162,2265,7839],[2755,2869,9871],[760,1524,754],[2478,2513,8130],[2146,2936,987],[628,2685,4563],[311,1063,4203],[2599,2929,466],[1504,2844,1454],[2559,2620,5403],[2380,2941,2469],[2834,2934,8387],[146,1902,3645],[892,2892,9869],[115,2629,8147],[2734,2945,5436],[70,1313,7975],[1274,2850,9593],[456,578,8927],[1904,2040,4259],[2456,2545,4762],[2232,2406,7579],[221,2700,6268],[1241,1802,7412],[1060,2254,1060],[1915,2137,442],[912,1191,5648],[770,1833,3643],[2391,2496,5098],[2420,2610,1773],[709,1733,4110],[301,309,6604],[353,1177,2912],[7,1947,842],[386,2440,5514],[1562,1582,5696],[1239,1926,4440],[1821,2689,3238],[2503,2780,6279],[1228,2248,2966],[2238,2797,3982],[204,886,8725],[1199,2379,5472],[2073,2893,2799],[1674,2439,8470],[168,1151,4593],[2918,2981,3300],[557,1224,6185],[2778,2805,3356],[1878,2244,325],[547,1030,6428],[2067,2087,2487],[1535,2743,4727],[2398,2794,6120],[11,2402,3001],[1551,1825,2796],[2361,2466,8084],[2756,2962,8425],[718,2595,3991],[99,1353,1012],[1720,1744,3002],[1368,2800,618],[2278,2755,1077],[852,2001,8198],[1287,1544,1221],[28,623,7024],[1867,2209,4582],[1662,2714,734],[2512,2819,146],[1760,2383,3983],[2508,2665,3489],[1110,2679,820],[509,1579,1306],[2938,2945,7312],[2201,2458,7816],[1701,2150,9748],[121,935,4774],[2615,2950,7056],[2130,2662,9328],[1153,1232,8889],[725,1034,8978],[1089,1228,2474],[2264,2525,4419],[1618,2579,8574],[2499,2564,5159],[953,2445,1361],[1770,2968,8583],[1411,2885,2455],[2582,2734,7035],[969,2253,3151],[39,520,9178],[168,550,4159],[600,2619,5908],[767,2612,5760],[687,2330,7622],[1008,2819,4358],[2616,2864,1517],[2771,2964,5661],[65,414,2627],[2135,2173,7350],[151,1353,2241],[574,1354,8118],[2452,2863,3175],[2083,2452,4131],[262,2357,7219],[1745,2968,1570],[768,1860,4758],[555,2249,6647],[835,2471,6726],[580,2446,8604],[861,1498,6179],[937,2392,5679],[1612,2812,6936],[628,1802,7719],[493,1081,3842],[2687,2821,4376],[1518,1680,2632],[2125,2678,6671],[303,1706,3783],[2810,2977,5882],[895,1642,2552],[1667,2451,6419],[1216,1665,2976],[1661,1983,5718],[1941,2076,8492],[2035,2172,6214],[2844,2890,5695],[116,2015,1267],[1553,1664,4299],[771,2511,3979],[393,851,8331],[2438,2439,4092],[2344,2633,9731],[1539,2978,1737],[2653,2825,9769],[322,1207,2085],[1535,2923,3624],[506,635,7016],[1638,2126,4963],[780,1026,8306],[1096,1140,860],[285,301,8226],[695,1563,318],[1275,1572,9047],[1441,1861,2209],[50,2083,1278],[754,1271,4351],[2967,2981,693],[2733,2964,7572],[2127,2924,2140],[1008,1025,9526],[14,1042,2151],[1713,2660,6189],[2030,2355,7166],[2506,2837,5817],[993,2315,4052],[2223,2490,8818],[648,1504,8845],[852,1058,5315],[1200,2012,6335],[2845,2979,1539],[623,752,6465],[2057,2156,3985],[68,415,7310],[2694,2745,4594],[2020,2702,1617],[796,1986,8121],[482,624,6817],[2170,2238,9194],[95,457,9887],[2783,2957,3586],[946,1234,7189],[451,2819,3941],[2531,2570,2920],[962,2654,6080],[2007,2864,9629],[1570,2617,9580],[2807,2889,2185],[2132,2327,185],[1298,1708,6559],[709,1152,8608],[263,1963,6989],[391,1649,882],[2086,2591,7928],[1912,2594,8464],[1463,2113,4501],[1791,2779,4092],[2560,2691,2149],[777,1216,6671],[362,693,1835],[2413,2829,737],[2666,2861,5544],[1404,2988,6629],[991,1706,6340],[1048,1885,5346],[381,1786,1874],[263,2583,4460],[1108,1518,2626],[1162,1834,1290],[765,1464,7491],[1194,2280,3538],[983,2974,5581],[1651,2957,6929],[944,1100,119],[1130,2619,3964],[1079,1250,557],[2141,2396,6788],[227,2206,5599],[2592,2756,6859],[517,1632,2917],[1393,2493,9242],[1235,2773,8785],[2774,2845,147],[2903,2957,8724],[2434,2631,4127],[2977,2988,2835],[582,2483,7521],[864,1289,6223],[992,1383,9087],[67,1367,4011],[428,2682,2321],[2777,2789,1848],[2085,2986,8023],[2021,2888,5537],[1959,2617,4917],[2420,2521,2585],[448,727,5113],[1985,2975,3934],[1861,2752,6614],[855,2875,2575],[937,2980,5306],[1219,2682,7191],[1846,2656,3193],[1523,2470,4456],[2385,2756,6319],[614,712,7564],[893,2947,9936],[1319,1358,3986],[2476,2975,2291],[2950,2994,8964],[2577,2835,3466],[851,1953,3033],[2806,2958,3186],[787,1695,1434],[2683,2766,9953],[2452,2812,9865],[2040,2543,1267],[2018,2047,1401],[2261,2902,8123],[587,2330,2779],[2319,2991,9931],[1324,2542,3834],[2388,2727,4496],[690,1681,3291],[1505,1909,8187],[304,981,9635],[2889,2910,8354],[2530,2838,1796],[1901,2342,6580],[341,1493,3120],[1746,2818,9346],[591,2062,5952],[2703,2787,3161],[1924,2631,8447],[2091,2131,9122],[2795,2935,3854],[2293,2710,4802],[949,2173,2449],[1720,1758,6024],[2802,2961,4804],[2221,2323,1946],[2793,2897,6782],[1134,1485,3982],[1864,2110,9095],[202,713,1556],[1995,2898,4827],[1060,2968,1674],[2696,2775,2582],[1331,2632,6286],[2347,2406,2595],[2642,2774,4807],[2832,2939,3865],[1379,1915,15],[1979,1986,8664],[2008,2236,2784],[1614,2917,6707],[584,2587,5728],[1972,2489,1435],[1089,1968,8448],[503,2802,3131],[66,1452,5125],[1583,2513,1542],[2806,2932,7098],[1656,2055,9287],[1698,2098,1539],[1088,1325,1135],[976,2049,5959],[702,2103,304],[795,2505,7020],[131,2006,3167],[556,1980,8336],[2329,2850,9473],[77,1912,8060],[502,1464,8704],[1121,2926,1433],[1550,2428,652],[262,1108,9152],[2621,2679,547],[2200,2284,1110],[1666,2892,8470],[2428,2644,4757],[1664,2307,2602],[400,753,9473],[573,1182,278],[195,1048,1343],[1904,2102,1917],[2591,2954,2938],[1540,1788,7053],[2380,2448,7298],[418,2105,4101],[1910,2798,822],[1009,1670,9000],[1872,2488,9733],[1963,2175,4599],[1883,2533,8596],[978,2194,6523],[1387,2638,5390],[1509,2390,2220],[1926,1976,5102],[1304,1514,2424],[2519,2845,7519],[331,2036,4582],[968,1837,3377],[927,1557,5200],[72,764,9987],[1134,2932,5587],[1879,2742,7676],[2335,2516,1914],[2220,2680,5665],[2741,2804,640],[1434,2863,3185],[456,1007,1056],[1953,2176,6568],[2755,2792,3128],[983,1290,3235],[1738,2450,1065],[2632,2853,5390],[1229,1647,9223],[1792,2415,3846],[1513,2627,8976],[8,2584,3248],[2871,2929,4804],[1072,2975,6022],[1364,2550,6028],[363,1629,4550],[2562,2943,2177],[369,1373,2380],[2328,2784,2784],[1897,2338,1209],[2090,2991,3724],[2432,2835,9641],[775,2394,2702],[128,375,172],[794,1286,4663],[2904,2923,2691],[2519,2549,413],[981,1793,375],[927,2867,5712],[595,1255,3998],[2708,2846,3774],[2021,2537,6773],[2822,2956,9988],[820,1326,9270],[1788,2627,590],[2688,2891,8589],[2035,2056,8652],[2472,2775,9375],[2155,2683,2507],[298,2587,9780],[1973,2865,4116],[1104,2816,94],[483,1815,496],[1648,2390,4094],[375,1318,2695],[1986,2289,1135],[966,1311,8769],[806,1265,878],[303,1123,7649],[646,1619,9821],[1499,2745,9934],[216,612,5711],[1637,1812,2727],[780,2438,4604],[2611,2775,5961],[489,1121,6545],[525,1235,8163],[2239,2912,4381],[1363,2857,8831],[1906,2294,6029],[892,2228,1276],[2041,2668,1865],[2518,2681,8801],[1176,1396,7485],[361,1650,5673],[908,1358,751],[900,1452,4594],[457,2411,4319],[409,610,2889],[2020,2864,1432],[2633,2746,6712],[89,2863,1859],[2299,2928,4105],[2675,2936,623],[2964,2967,7887],[340,653,8026],[2050,2815,5030],[1811,1905,6485],[218,1795,2388],[2185,2459,8810],[1731,2242,1287],[2676,2788,8257],[1032,1089,7606],[1510,2951,5807],[1435,1575,1310],[2684,2945,8565],[1168,1419,8650],[1647,1781,6773],[1674,2345,1926],[1061,2650,9432],[2602,2607,9013],[2940,2959,4798],[80,1811,5306],[69,956,463],[1684,2599,2604],[2725,2977,771],[90,765,2104],[1232,2506,6782],[187,476,5889],[2676,2733,6262],[179,1020,2662],[2208,2995,1132],[1011,2913,2397],[702,2039,2663],[1505,2781,5580],[540,1579,3230],[2280,2300,2472],[2050,2422,6005],[2049,2159,6505],[2931,2957,5978],[1024,1984,9237],[1834,2263,58],[231,1423,9974],[2899,2978,1574],[928,1260,8964],[2708,2852,1246],[2891,2928,4689],[58,1758,9823],[2145,2684,9983],[2463,2879,2358],[92,1807,7465],[2156,2705,2932],[1726,1968,244],[1974,2271,6191],[2532,2845,3365],[2342,2906,8651],[1421,2102,5883],[1572,2817,8278],[2517,2586,2347],[1759,2393,719],[2363,2805,7342],[1662,2110,2033],[2241,2662,2746],[1956,2964,5208],[1272,1972,5624],[265,2371,2317],[35,2450,7182],[1106,2696,4866],[2795,2928,1671],[376,858,9673],[200,2693,6734],[2447,2908,2947],[1298,1353,3857],[2364,2584,5705],[2837,2928,2956],[2741,2776,3424],[996,2975,4332],[2029,2427,6497],[2536,2651,1879],[2605,2795,5117],[944,2543,9919],[1663,2296,294],[1378,2637,7013],[377,1503,7700],[373,2765,4276],[2771,2850,904],[873,2408,1501],[1751,1752,2662],[1652,2932,9823],[724,2611,8126],[2482,2661,9361],[2705,2745,9822],[797,1556,2495],[2111,2913,5745],[2206,2452,1294],[1201,1957,53],[2589,2877,6981],[1734,2967,9083],[2,1815,7674],[2774,2775,7490],[2181,2633,9271],[684,1674,3011],[2719,2732,8552],[1071,1562,4507],[720,1847,5520],[1327,2289,4621],[1614,1647,2769],[931,1786,5852],[93,279,184],[274,489,5998],[2403,2421,1688],[2930,2932,9888],[931,962,8369],[1868,2578,4979],[1699,2870,5545],[344,2076,8409],[2714,2900,6198],[2087,2432,5868],[268,906,8501],[2830,2962,1479],[1561,1925,8953],[631,1921,9982],[929,1347,7718],[366,2011,8167],[1305,1642,2892],[2171,2582,8147],[377,2656,7486],[1924,2440,7736],[2723,2809,7731],[191,2916,1054],[1843,2231,3739],[565,1721,2731],[1911,2217,3374],[1892,2314,9892],[409,2503,659],[2287,2694,164],[2820,2983,2208],[865,2080,290],[2571,2883,6987],[974,1817,274],[83,2790,1594],[2651,2983,5778],[199,2687,3504],[2289,2465,8369],[975,2888,58],[368,628,881],[1622,1826,7955],[547,1346,9120],[1809,2474,2074],[1879,1947,6463],[2464,2480,7254],[310,2516,7025],[902,2376,3954],[367,488,152],[1952,2899,7926],[1747,2803,7525],[689,2862,7019],[1037,1617,2041],[1752,2239,1656],[2313,2319,6821],[633,1526,5604],[2947,2960,6585],[1398,1824,1487],[613,2412,1938],[1836,2820,2890],[2962,2977,2716],[500,2483,6612],[920,2630,796],[734,1440,1691],[337,1754,9342],[428,1950,5058],[408,2728,7725],[637,1594,5263],[157,355,6875],[98,1611,9309],[1942,2130,9807],[937,2775,9196],[2731,2938,3612],[211,2161,936],[2708,2854,4977],[1935,2569,6027],[2213,2673,8643],[2818,2917,1096],[479,509,8808],[1719,2846,7703],[80,2142,7551],[2257,2721,3086],[1000,2739,768],[1448,2897,4083],[2003,2950,6304],[1016,1935,9021],[1256,1514,697],[2655,2741,7065],[2057,2747,6287],[578,2357,4181],[2212,2880,6397],[2198,2984,3170],[1568,2618,3650],[2484,2787,4565],[716,1929,5636],[308,359,3118],[2149,2937,2526],[1188,2754,9349],[2338,2756,7880],[72,1139,2166],[1477,1774,2371],[679,2799,6194],[1566,2743,9366],[2439,2784,2563],[1013,1928,8079],[992,2909,9521],[652,2586,706],[1996,2655,6385],[1308,1765,4118],[2130,2802,4984],[1915,2163,4016],[1782,2806,4119],[2033,2880,7993],[2244,2824,8087],[2015,2892,9948],[2529,2738,8436],[532,2156,5553],[1467,2457,5492],[1721,2006,8851],[2368,2699,4211],[2951,2961,7435],[582,2971,2739],[1027,2697,9595],[2789,2831,2155],[211,412,3361],[2374,2638,7521],[50,232,3007],[1055,2523,9771],[1660,2898,7481],[2221,2435,9117],[151,706,5078],[69,2136,1079],[2192,2476,739],[876,1140,8324],[2077,2811,2208],[537,1049,3189],[850,1428,2691],[2812,2817,5426],[2607,2678,1223],[221,1151,9583],[699,2686,2471],[1059,1938,7354],[1639,2149,2073],[1248,2837,4668],[774,2279,7078],[452,2749,5814],[1838,2382,6097],[901,2047,8202],[2598,2934,2882],[304,337,6046],[1033,1279,1416],[483,2967,5256],[2993,2999,5723],[643,2793,6932],[2077,2720,1432],[2542,2724,2735],[225,1298,8108],[2481,2710,1119],[2664,2898,7630],[2116,2166,6661],[1827,2737,5557],[979,1069,2096],[1228,1994,2106],[1181,1256,1868],[205,2756,1229],[2975,2979,1665],[1550,1658,7238],[2994,2997,7501],[2053,2894,4118],[2662,2726,5197],[2134,2484,2612],[652,2090,9600],[1444,2099,7096],[2696,2971,7748],[1509,2742,1700],[2928,2970,9583],[2877,2957,2849],[1341,2865,4014],[2543,2741,2184],[482,1494,5522],[953,2517,8165],[2518,2919,3576],[1490,2264,6143],[349,879,5149],[664,2658,751],[2653,2834,2190],[1010,1944,2930],[649,1197,7723],[278,1173,4106],[1231,1444,223],[1960,2243,3641],[1608,2426,4640],[1969,2075,9143],[2376,2853,9665],[2609,2814,399],[1661,1912,7618],[1125,1781,3524],[2986,2995,982],[846,1521,4133],[1488,1758,9580],[447,637,2346],[1532,2037,5042],[42,2003,9414],[893,2698,7682],[1870,1941,3799],[90,2848,3035],[1168,1182,8230],[2528,2881,4072],[2920,2972,9268],[445,2592,509],[672,2268,3520],[1628,2535,4444],[2972,2990,6578],[1615,2800,8971],[99,1588,4252],[1357,1620,4009],[1141,1474,7133],[1288,2917,8364],[403,806,7471],[1750,1873,843],[2155,2967,4835],[2324,2973,8455],[320,1333,3967],[2174,2716,7616],[268,926,5579],[361,2263,3229],[1281,1621,7375],[1495,2726,8753],[1612,2337,8651],[1444,2345,2241],[1279,1595,7082],[673,1628,7883],[2296,2939,6431],[1281,2649,6175],[937,1048,1755],[663,2277,1785],[2525,2801,2558],[2619,2982,9703],[1787,2528,4668],[426,810,6569],[223,1807,5998],[2024,2258,8747],[1362,2475,6733],[2573,2594,4376],[2117,2410,1230],[2791,2827,9481],[602,697,5498],[1019,1755,9512],[831,2642,9254],[594,1694,9233],[1346,2072,4654],[1569,2126,4096],[1803,1884,789],[511,2358,3173],[2072,2801,4478],[900,2906,7365],[2931,2952,3501],[1613,2026,3900],[2406,2777,4533],[2409,2551,9483],[562,602,2652],[2444,2795,3985],[1107,2581,1587],[1169,1809,4493],[278,1429,3309],[358,652,6868],[1090,1239,6941],[127,507,7091],[839,2474,9253],[2890,2928,9530],[2372,2947,2111],[1894,2753,5060],[1003,1466,5013],[965,1371,3873],[738,2549,1043],[2735,2865,6496],[897,2396,6044],[2336,2505,4399],[1613,2977,8711],[898,2556,7361],[2003,2286,9359],[1190,1934,8431],[432,503,690],[2241,2642,3559],[103,925,3510],[1595,2721,4720],[2992,2996,507],[1508,2627,8719],[1747,2844,9651],[1814,1835,626],[2795,2890,7822],[511,2712,86],[263,1210,9792],[289,676,4040],[2634,2650,1524],[2695,2880,1045],[902,2373,5330],[425,1122,9305],[309,405,920],[1977,2190,1684],[534,2808,853],[2000,2685,2234],[1935,2136,3227],[867,2248,5969],[2205,2368,7468],[1842,2611,7483],[1337,1580,4026],[2135,2385,6081],[1237,2955,33],[164,1502,812],[1190,1631,2336],[251,560,5293],[2331,2449,5627],[804,1204,4838],[2959,2968,8645],[1478,2731,1591],[2880,2933,1866],[1805,2473,4244],[1735,2476,9538],[218,1947,8277],[2951,2959,8845],[1794,2602,8720],[989,2603,655],[1862,2037,3949],[2448,2813,4993],[837,1582,8651],[93,1606,2747],[1401,2613,2742],[2270,2405,2981],[224,2705,874],[2518,2990,1496],[2851,2994,6141],[1457,2843,6302],[2938,2963,2598],[1395,2979,7763],[292,2886,450],[2361,2780,3347],[1408,2382,3502],[755,1961,4844],[123,532,5580],[745,1631,6482],[70,2025,7014],[2193,2649,9160],[913,1236,4058],[808,2894,670],[850,2728,9984],[2389,2538,9916],[2875,2883,4846],[2055,2660,7771],[96,656,874],[1289,1708,8065],[1565,1657,7289],[2274,2906,1774],[1921,2805,2911],[2169,2759,129],[1665,1807,5850],[2468,2821,5764],[976,2000,1569],[715,2010,9713],[1336,2850,3487],[2306,2626,1275],[297,1696,4746],[2115,2915,128],[2244,2336,6756],[2968,2975,8140],[615,2910,8956],[1538,1935,944],[764,1162,1023],[1713,2594,107],[920,2901,303],[1787,2703,4405],[2411,2881,1660],[2899,2917,6042],[598,2814,8673],[1797,2752,1068],[2154,2444,5694],[2196,2685,7865],[445,2793,3899],[736,1083,5165],[2255,2644,6857],[2985,2988,3975],[726,1032,8069],[672,1018,5027],[278,1982,4972],[1687,2843,5723],[2935,2989,7643],[776,2710,6206],[712,2024,7692],[2897,2931,8810],[836,2890,1794],[2986,2989,7316],[2661,2917,6577],[1837,2202,6003],[1151,1754,9732],[1835,2812,1903],[1219,2388,7892],[2617,2727,3733],[2345,2718,4357],[6,872,9327],[1137,1751,6339],[271,1304,7082],[897,1951,370],[441,584,2337],[1369,2608,5721],[873,2185,9015],[1299,2796,7358],[2087,2401,3002],[2755,2758,4097],[2713,2728,9787],[134,2087,9860],[2765,2976,7958],[290,1316,4240],[401,2309,5289],[212,1064,7202],[1940,2560,6036],[1234,2659,8847],[1718,2614,2787],[2285,2668,1010],[2130,2247,9387],[1185,2255,7324],[197,2898,7594],[135,2082,2806],[2696,2810,3738],[2093,2919,2334],[1700,1935,1780],[2683,2995,3167],[126,1241,3089],[52,2097,6283],[1915,2864,1256],[1518,1941,7360],[2039,2601,9959],[377,1553,6891],[1220,2126,9181],[2117,2481,6935],[702,1317,7995],[2681,2734,5952],[450,2758,3306],[2045,2610,6559],[2585,2754,1809],[2751,2808,1129],[2233,2500,6790],[2169,2789,8471],[2563,2895,1382],[1587,1852,6345],[842,2701,6228],[1295,2419,5988],[1818,1895,9935],[505,1915,3542],[2660,2982,5740],[2447,2964,3244],[1608,1785,6362],[644,1364,6333],[60,1993,2201],[2057,2798,7577],[378,2141,1768],[2332,2732,8240],[779,1356,5911],[2372,2375,8576],[1313,2753,226],[1613,2522,8791],[1090,1882,6440],[1656,2779,993],[1345,1860,1037],[1466,2313,8752],[2007,2465,4261],[2183,2947,2922],[1766,2047,2685],[1681,2224,9986],[1002,2731,8112],[2929,2994,716],[1982,2101,9636],[19,1780,9666],[1045,1918,5515],[596,1318,2986],[1756,2495,5983],[564,1107,5756],[761,1663,4096],[1923,2856,5682],[1959,2057,3464],[185,2569,3510],[2858,2889,1524],[1739,2932,607],[2815,2916,3387],[2961,2965,9023],[567,1016,3654],[2324,2565,7749],[2204,2641,7112],[1998,2669,5430],[2115,2516,6720],[2730,2747,662],[1400,2102,413],[778,1033,6039],[2909,2973,4344],[1599,1963,3816],[2695,2795,968],[2192,2271,9419],[6,2073,6665],[2919,2920,3707],[1266,1892,3969],[2873,2989,273],[1513,2511,98],[2063,2266,8432],[1899,1938,9848],[851,2946,3968],[1917,2000,9285],[2105,2971,824],[2256,2665,3747],[423,1581,5264],[2624,2797,7371],[1332,2839,4083],[625,2778,4191],[789,2660,7320],[1609,2150,9015],[871,2763,5794],[2290,2450,4451],[2721,2943,4428],[1485,1997,5463],[486,2783,9929],[43,1946,6658],[123,159,6186],[1686,2239,6247],[530,1352,6151],[2616,2800,1134],[1027,1031,6791],[2506,2768,2425],[945,1345,2749],[1937,2920,9708],[1466,2436,4964],[253,2904,2058],[211,423,5365],[707,1501,392],[733,2549,4983],[1826,2402,8488],[1314,2221,5420],[2260,2970,4652],[1594,2390,1660],[2215,2400,1317],[2903,2983,395],[1592,2800,8821],[814,1117,3274],[1928,2123,951],[1528,2344,4846],[971,2204,9785],[442,481,4465],[1989,2440,3729],[2610,2962,3517],[1052,2963,9867],[24,2750,3188],[2778,2987,7545],[1530,2014,8194],[2230,2596,4649],[921,1496,9800],[246,1083,7216],[609,2288,9705],[577,2988,1545],[1543,2066,5450],[89,808,3527],[2917,2966,3655],[479,1270,470],[162,1457,5134],[895,948,2872],[437,1075,2012],[2203,2736,5492],[2630,2799,6898],[497,817,8583],[2582,2587,234],[373,2555,5446],[560,1208,7408],[2880,2940,924],[2162,2916,5651],[2663,2883,3301],[1659,2372,2220],[1813,2017,6340],[1616,1878,5011],[2739,2776,1423],[2540,2857,9614],[2464,2851,9853],[638,2837,8075],[2676,2848,4027],[1945,2581,5675],[2496,2900,7576],[2216,2917,2145],[2766,2907,201],[2662,2894,911],[2248,2275,8529],[405,2049,8830],[2965,2974,5759],[1429,2712,7873],[2880,2927,1227],[2297,2826,1126],[2106,2768,2325],[1468,2803,6807],[2233,2634,5842],[55,805,3431],[1679,2024,6222],[520,1855,6907],[134,2345,9009],[2824,2980,8535],[2818,2897,8424],[1114,1311,5717],[778,973,1889],[642,2997,4326],[704,1762,6197],[906,1192,6631],[63,2087,329],[1771,2891,1769],[1477,1516,5989],[2979,2992,264],[1204,1841,9229],[6,550,8273],[2687,2844,6699],[864,2386,6102],[2976,2985,1060],[2857,2954,3798],[12,639,144],[644,2473,5784],[1865,2281,8821],[172,1051,861],[2726,2838,5455],[342,2926,376],[1940,2129,6837]]
M=25059196
N=3000
so.reachableNodes(edges,M,N)

50102076

In [103]:
import collections
import heapq
class Solution(object):
    def reachableNodes(self, edges, M, N):
        graph = collections.defaultdict(dict)
        for u, v, w in edges:
            graph[u][v] = graph[v][u] = w

        pq = [(0, 0)]
        dist = {0: 0}
        used = {}
        ans = 0

        while pq:
            d, node = heapq.heappop(pq)
            if d > dist[node]: continue
            # Each node is only visited once.  We've reached
            # a node in our original graph.
            ans += 1

            for nei in graph[node]:
                # M - d is how much further we can walk from this node;
                # weight is how many new nodes there are on this edge.
                # v is the maximum utilization of this edge.
                weight = graph[node][nei]
                v = min(weight, M - d)
                used[node, nei] = v

                # d2 is the total distance to reach 'nei' (neighbor) node
                # in the original graph.
                d2 = d + weight + 1
                if d2 < dist.get(nei, M+1):
                    heapq.heappush(pq, (d2, nei))
                    dist[nei] = d2

        # At the end, each edge (u, v, w) can be used with a maximum
        # of w new nodes: a max of used[u, v] nodes from one side,
        # and used[v, u] nodes from the other.
        for u, v, w in edges:
            ans += min(w, used.get((u, v), 0) + used.get((v, u), 0))

        return ans

In [104]:
ss = Solution()

In [105]:
ss.reachableNodes(edges,M,N)

50102076